In [ ]:
import os.path as osp
import sys
import random

path = osp.dirname(osp.abspath(''))
sys.path.append(path)
sys.path.append(osp.join(path, "./open_biomed"))
path

In [ ]:
import torch
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(device)

In [ ]:

def read_text(path=None):
    with open(path, 'r') as f:

        text_list = []
        for index, line in enumerate(f):
            text = line.strip()
            text_list.append(text)
    return text_list
path='./extracted_chemistry_keywords.txt'
extracted_text=read_text(path=path)
path='./text_after_structure_filter.txt'
texts=read_text(path=path)

# path='./smis_after_structure_filter.txt'
# smis=read_text(path=path)

In [ ]:
def structure_isomers(path=''):
    sim_list=[]
    with open(path, 'r') as f:
        for index, line in enumerate(f):

            isomers_list = line.strip().split("\t")
            sim_list.append(isomers_list[0])
    return sim_list
structure_isomers=structure_isomers(path='./structure_ismers.txt')

smis=structure_isomers
print((len(texts),len(smis)))

In [ ]:
len(extracted_text),len(texts),len(smis)

In [ ]:
# 线性核函数，用 PyTorch 实现
def linear_kernel(X):
    return torch.mm(X, X.T)  # 使用矩阵乘法

# 居中核矩阵
def center_kernel(K):
    n = K.shape[0]
    H = torch.eye(n, device=K.device) - torch.ones((n, n), device=K.device) / n  # 中心化矩阵 H
    return torch.mm(H, torch.mm(K, H))

# CKA 实现，计算两个张量 X 和 Y 的 CKA
def CKA(X, Y):
    # 计算线性核矩阵
    K = linear_kernel(X)
    L = linear_kernel(Y)

    # 居中核矩阵
    Kc = center_kernel(K)
    Lc = center_kernel(L)

    # 计算 HSIC 值
    hsic_XY = torch.trace(torch.mm(Kc, Lc))
    hsic_XX = torch.trace(torch.mm(Kc, Kc))
    hsic_YY = torch.trace(torch.mm(Lc, Lc))

    # 计算 CKA 值
    cka_value = hsic_XY / torch.sqrt(hsic_XX * hsic_YY)
    return cka_value

In [ ]:
from metrics import cka,gram_linear
# structure_feats_np = structure_feats[indes].cpu().detach().numpy()
# text_feats_np = text_feats[indes].cpu().detach().numpy()
# # Calculate Gram matrices for structure_feats and text_feats
def de_biased_cka(structure_feats,text_feats):
    structure_feats_np = structure_feats.cpu().detach().numpy()
    text_feats_np = text_feats.cpu().detach().numpy()
    gram_structure = gram_linear(structure_feats_np)
    gram_text = gram_linear(text_feats_np)

    # Calculate CKA between the two Gram matrices
    cka_value = cka(gram_structure, gram_text, debiased=True)

    return cka_value

In [ ]:
import torch.nn.functional as F

def retrieve(A, B, K=1):
    """
    检索任务函数，输入两个 tensor A 和 B，返回 A 和 B 匹配的索引值、匹配的分数和正确匹配的条目数。

    参数：
    - A: torch.Tensor，待匹配的 query tensor，形状 (n, d)。
    - B: torch.Tensor，待匹配的 target tensor，形状 (n, d)。
    - K: int，检索 top K 匹配。

    返回：
    - indices: 每个 A 对应的最相似 B 的索引值，形状 (n, K)。
    - scores: 每个 A 对应的最相似 B 的分数，形状 (n, K)。
    - correct_total: 匹配正确的总条目数。
    """

    # 计算 A 和 B 之间的余弦相似度 (n, n)
    similarities = F.cosine_similarity(A.unsqueeze(1), B.unsqueeze(0), dim=-1)
    # 获取相似度最高的前 K 个条目的索引和分数
    topk_scores, topk_indices = torch.topk(similarities, K, dim=1)
    # 计算总的正确匹配数量
    # 正确的定义：A[i] 和 B[i] 相匹配，意味着第 i 个 A 应该匹配第 i 个 B
    correct_total = 0
    n = A.size(0)
    for i in range(n):
        # 检查 top K 的索引是否包含正确匹配
        if i in topk_indices[i]:
            correct_total += 1
    return topk_indices, topk_scores, correct_total

In [ ]:
def overleap(indexA,indexB ):
    C=[]
    D=[]
    for a, b in zip(indexA, indexB):
        overlap = list(set(a) & set(b))
        C.append(overlap)
        D.append([len(overlap)])

    # Print results
    print("Overlap C:", C)
    print("Number of overlaps D:", D)

    sum=0
    for i in C:
        sum=sum+len(i)
    print(sum)

In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

def KM(embedding=None,num_clusters=5):
    num_clusters = num_clusters  # You can set this based on your requirement

    # Initialize and fit the KMeans model
    kmeans = KMeans(n_clusters=num_clusters, random_state=42)
    kmeans.fit(embedding)

    return kmeans

In [ ]:
from sklearn.neighbors import NearestNeighbors

def NN(embedding=None,num=10):
    # Initialize the Nearest Neighbors model
    num_neighbors = num  # Number of nearest neighbors to find
    nn_model = NearestNeighbors(n_neighbors=num_neighbors)

    # Fit the model to your data
    try:
        nn_model.fit(embedding.cpu().numpy())
    except:
        nn_model.fit(embedding.detach().cpu().numpy())

    return nn_model

In [ ]:
len(extracted_text),len(smis),len(texts)

In [ ]:
# new_compute_list=[]
# with open('compute_list.txt','r') as f:
#     for index, line in enumerate(f):
#         i = line.strip()
#         new_compute_list.append(int(i))

# new_pass_list=[]
# with open('pass_list.txt','r') as f:
#     for index, line in enumerate(f):
#         i = line.strip()
#         new_pass_list.append(int(i))

# len(new_compute_list),len(new_pass_list)

In [ ]:
# len(extracted_text),len(smis),len(texts)

In [ ]:
# extracted_text=[extracted_text[i] for i in new_compute_list]
# smis=[smis[i] for i in new_compute_list]
# texts=[texts[i] for i in new_compute_list]
# len(extracted_text),len(smis),len(texts)

## aligned model

In [ ]:
# prepare data
import json
from open_biomed.utils.data_utils import DataProcessorFast
from open_biomed.utils import fix_path_in_config

config = json.load(open("./../configs/mtr/molfm.json", "r"))
# config['network']['bert_config_path']='../../configs/encoders/multimodal/molfm_bert_config.json'
# config['data']['mol']['featurizer']['text']['model_name_or_path']='../../ckpts/text_ckpts/scibert_scivocab_uncased'
fix_path_in_config(config, path)
print("Config: ", config)
processor = DataProcessorFast(entity_type="molecule", config=config["data"]["mol"])
processor.featurizer.set_mol2text_dict(dict(zip(smis, texts)))
mols = processor(smis)

In [ ]:
import torch
from open_biomed.models.multimodal import MolFM
model = MolFM(config["network"])
state_dict = torch.load("./../ckpts/fusion_ckpts/MolFM.pth", map_location="cpu")["model"]
model.load_state_dict(state_dict, strict=False)
model.eval()


In [ ]:
model.to(device)
with torch.no_grad():
    structure_feats = model.encode_mol(mols["structure"].to(device), proj=True)
    text_feats = model.encode_text(mols["text"].to(device))

### with original data 

In [ ]:
structure_feats.shape,text_feats.shape

### CKA

In [ ]:
CKA(structure_feats,text_feats)

In [ ]:
de_biased_cka(structure_feats,text_feats)

### match

In [ ]:
A =structure_feats
B = text_feats
indices_k1, scores_k1, correct_total_k1 = retrieve(A.to(device), B.to(device), K=1)
# 打印结果
# print("Top K 索引：", indices_k1)
# print("Top K 分数：", scores_k1)
print("正确匹配的条目数：", correct_total_k1)

In [ ]:
A =structure_feats
B = text_feats
indices, scores, correct_total = retrieve(A.to(device), B.to(device), K=10)
# 打印结果
# print("Top K 索引：", indices)
# print("Top K 分数：", scores)
print("正确匹配的条目数：", correct_total)

### text NN

In [ ]:
texts_NN=NN(embedding=B)
texts_NN_distance,texts_NN_indes=texts_NN.kneighbors(B.cpu().detach())
overleap(indices.cpu().numpy(), texts_NN_indes)

### SMILES NN

In [ ]:
smis_NN=NN(embedding=A)
smis_NN_distance,smis_NN_indes=smis_NN.kneighbors(A.cpu().detach())
overleap(indices.cpu().numpy(), smis_NN_indes)

### K-means
### texts

In [ ]:
# K-means
texts_KM=KM(B.cpu().numpy())
cls_index=texts_KM.predict(B.cpu().numpy())

C=B[indices_k1.cpu()].reshape(B.shape).cpu().numpy()
match_index=texts_KM.predict(C)
x=cls_index==match_index
np.sum(x)

##### smis

In [ ]:
# A =structure_feats
# B = text_feats

smis_KM=KM(A.cpu().numpy())

smis_index=smis_KM.predict(A.cpu().numpy())

C=A[indices_k1.cpu()].reshape(A.shape).cpu().numpy()
match_index=smis_KM.predict(C)

x=smis_index==match_index

np.sum(x)

## MoLFormer

In [ ]:
# # success_sim
# # success_text
# import torch
# from transformers import AutoModel, AutoTokenizer

# # device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# model = AutoModel.from_pretrained("ibm/MoLFormer-XL-both-10pct", deterministic_eval=True, trust_remote_code=True)
# tokenizer = AutoTokenizer.from_pretrained("ibm/MoLFormer-XL-both-10pct", trust_remote_code=True)

# # smiles = ["Cn1c(=O)c2c(ncn2C)n(C)c1=O", "CC(=O)Oc1ccccc1C(=O)O"]
# inputs = tokenizer(smis, padding=True, return_tensors="pt")
# with torch.no_grad():
#     sim_embeddings = model(**inputs)
# # sim_embeddings.pooler_output


In [ ]:
# sim_embeddings.pooler_output.shape

In [ ]:
# torch.save(sim_embeddings,'./savedpt/Structure_sim_embeddings.pt')

In [ ]:
sim_embeddings=torch.load('./savedpt/Structure_sim_embeddings.pt')

In [ ]:
sim_embeddings.pooler_output.shape

## SCI-BERT

### texts

In [ ]:
from transformers import AutoTokenizer,AutoModel

def SCIBERT(texts=texts):

    tokenizer = AutoTokenizer.from_pretrained('allenai/scibert_scivocab_uncased')
    model = AutoModel.from_pretrained('allenai/scibert_scivocab_uncased')
    model = model.to(device)

    # 将句子 token 化，并将输入转移到 GPU
    inputs = tokenizer(texts, return_tensors='pt', truncation=True, padding=True, max_length=512)
    inputs = {key: value.to(device) for key, value in inputs.items()}  # 转移到 GPU

    # 获取BERT的输出
    with torch.no_grad():  # 禁用梯度计算，节省内存
        outputs = model(**inputs)

    # BERT的最后隐藏层的输出 [batch_size, seq_len, hidden_size]
    last_hidden_states = outputs.last_hidden_state

    # 取 [CLS] token 的 embedding（句子嵌入）
    cls_embedding = last_hidden_states[:, 0, :]

    # 也可以取所有 token 的平均值作为句子嵌入
    mean_embedding = torch.mean(last_hidden_states, dim=1)

    print(f"CLS token embedding size: {cls_embedding.shape}")
    print(f"Mean token embedding size: {mean_embedding.shape}")

    return cls_embedding,mean_embedding

In [ ]:
SCIB_cls_embedding,SCIB_mean_embedding=SCIBERT(texts)

### extracted_text

In [ ]:
SCIB_extracted_text_cls_embedding,SCIB_extracted_text_mean_embedding=SCIBERT(extracted_text)

### CKA

In [ ]:
de_biased_cka(SCIB_cls_embedding,sim_embeddings.pooler_output),de_biased_cka(SCIB_mean_embedding,sim_embeddings.pooler_output),de_biased_cka(SCIB_extracted_text_cls_embedding,sim_embeddings.pooler_output),de_biased_cka(SCIB_extracted_text_mean_embedding,sim_embeddings.pooler_output)

In [ ]:
CKA(SCIB_cls_embedding.cpu(),sim_embeddings.pooler_output),CKA(SCIB_mean_embedding.cpu(),sim_embeddings.pooler_output),CKA(SCIB_extracted_text_cls_embedding.cpu(),sim_embeddings.pooler_output),CKA(SCIB_extracted_text_mean_embedding.cpu(),sim_embeddings.pooler_output)

### Now, we have cls mean embedding of texts and extracted_text
cls_embedding,

mean_embedding,

extracted_text_cls_embedding,

extracted_text_mean_embedding,

#### text NN

In [ ]:
SCIB_texts_cls_NN=NN(embedding=SCIB_cls_embedding)
SCIB_texts_mean_NN=NN(embedding=SCIB_mean_embedding)
SCIB_extracted_text_cls_NN=NN(embedding=SCIB_extracted_text_cls_embedding)
SCIB_extracted_text_mean_NN=NN(embedding=SCIB_extracted_text_mean_embedding)

In [ ]:
SCIB_texts_cls_NN_distance,SCIB_texts_cls_NN_indes=SCIB_texts_cls_NN.kneighbors(sim_embeddings.pooler_output)
SCIB_texts_mean_NN_distance,SCIB_texts_mean_NN_indes=SCIB_texts_mean_NN.kneighbors(sim_embeddings.pooler_output)
SCIB_extracted_text_cls_NN_distance,SCIB_extracted_text_cls_NN_indes=SCIB_extracted_text_cls_NN.kneighbors(sim_embeddings.pooler_output)
SCIB_extracted_text_mean_NN_distance,SCIB_extracted_text_mean_NN_indes=SCIB_extracted_text_mean_NN.kneighbors(sim_embeddings.pooler_output)

In [ ]:
SCIB_texts_match_indices, SCIB_texts_match_scores, SCIB_texts_match_correct_total = retrieve(sim_embeddings.pooler_output, SCIB_cls_embedding.cpu().detach(), K=10)
SCIB_texts_mean_match_indices, SCIB_texts_mean_match_scores, SCIB_texts_mean_match_correct_total = retrieve(sim_embeddings.pooler_output, SCIB_mean_embedding.cpu().detach(), K=10)

# 打印结果
# print("Top K 索引：", indices)
# print("Top K 分数：", scores)
print("正确匹配的条目数：", SCIB_texts_match_correct_total)
print("正确匹配的条目数：", SCIB_texts_mean_match_correct_total)
SCIB_extracted_text_match_indices, SCIB_extracted_text_match_scores, SCIB_extracted_text_match_correct_total = retrieve(sim_embeddings.pooler_output, SCIB_extracted_text_cls_embedding.cpu().detach(), K=10)
SCIB_extracted_text_mean_match_indices, SCIB_extracted_text_mean_match_scores, SCIB_extracted_text_mean_match_correct_total = retrieve(sim_embeddings.pooler_output, SCIB_extracted_text_mean_embedding.cpu().detach(), K=10)

print("正确匹配的条目数：", SCIB_extracted_text_match_correct_total)
print("正确匹配的条目数：", SCIB_extracted_text_mean_match_correct_total)

In [ ]:
SCIB_texts_k1_match_indices, SCIB_texts_k1_match_scores, SCIB_texts_k1_match_correct_total = retrieve(sim_embeddings.pooler_output.cpu().detach(), SCIB_cls_embedding.cpu().detach(), K=1)
SCIB_texts_K1_mean_match_indices, SCIB_texts_k1_mean_match_scores, SCIB_texts_k1_mean_match_correct_total = retrieve(sim_embeddings.pooler_output.cpu().detach(), SCIB_mean_embedding.cpu().detach(), K=1)

# 打印结果
# print("Top K 索引：", indices)
# print("Top K 分数：", scores)
print("正确匹配的条目数：", SCIB_texts_k1_match_correct_total)
print("正确匹配的条目数：", SCIB_texts_k1_mean_match_correct_total)
SCIB_extracted_k1_text_match_indices, SCIB_extracted_k1_text_match_scores, SCIB_extracted_k1_text_match_correct_total = retrieve(sim_embeddings.pooler_output.cpu().detach(), SCIB_extracted_text_cls_embedding.cpu().detach(), K=1)
SCIB_extracted_k1_text_mean_match_indices, SCIB_extracted_k1_text_mean_match_scores, SCIB_extracted_text_k1_mean_match_correct_total = retrieve(sim_embeddings.pooler_output.cpu().detach(), SCIB_extracted_text_mean_embedding.cpu().detach(), K=1)

print("正确匹配的条目数：", SCIB_extracted_k1_text_match_correct_total)
print("正确匹配的条目数：", SCIB_extracted_text_k1_mean_match_correct_total)

In [ ]:
overleap(SCIB_texts_match_indices.cpu().numpy(), SCIB_texts_cls_NN_indes)

overleap(SCIB_texts_mean_match_indices.cpu().numpy(), SCIB_texts_mean_NN_indes)
#_____________cross should get low score_____________

overleap(SCIB_texts_match_indices.cpu().numpy(), SCIB_texts_mean_NN_indes)
overleap(SCIB_texts_mean_match_indices.cpu().numpy(), SCIB_texts_cls_NN_indes)
print('='*20)
overleap(SCIB_extracted_text_match_indices.cpu().numpy(), SCIB_extracted_text_cls_NN_indes)
overleap(SCIB_extracted_text_mean_match_indices.cpu().numpy(), SCIB_extracted_text_mean_NN_indes)
#_____________cross should get low score_____________

overleap(SCIB_extracted_text_mean_match_indices.cpu().numpy(), SCIB_extracted_text_cls_NN_indes)
overleap(SCIB_extracted_text_match_indices.cpu().numpy(), SCIB_extracted_text_mean_NN_indes)

#### SMILES NN

In [ ]:
SCIBSMIS_texts_cls_NN=NN(embedding=sim_embeddings.pooler_output)
SCIBSMIS_texts_mean_NN=NN(embedding=sim_embeddings.pooler_output)
SCIBSMIS_extracted_text_cls_NN=NN(embedding=sim_embeddings.pooler_output)
SCIBSMIS_extracted_text_mean_NN=NN(embedding=sim_embeddings.pooler_output)

In [ ]:
SCIBSMIS_texts_cls_NN_distance,SCIBSMIS_texts_cls_NN_indes=SCIBSMIS_texts_cls_NN.kneighbors(SCIB_cls_embedding.cpu().numpy())
SCIBSMIS_texts_mean_NN_distance,SCIBSMIS_texts_mean_NN_indes=SCIBSMIS_texts_mean_NN.kneighbors(SCIB_mean_embedding.cpu().numpy())

SCIBSMIS_extracted_text_cls_NN_distance,SCIBSMIS_extracted_text_cls_NN_indes=SCIBSMIS_extracted_text_cls_NN.kneighbors(SCIB_extracted_text_cls_embedding.cpu().numpy())
SCIBSMIS_extracted_text_mean_NN_distance,SCIBSMIS_extracted_text_mean_NN_indes=SCIBSMIS_extracted_text_mean_NN.kneighbors(SCIB_extracted_text_mean_embedding.cpu().numpy())

In [ ]:
overleap(SCIB_texts_match_indices.cpu().numpy(), SCIBSMIS_texts_cls_NN_indes)

overleap(SCIB_texts_mean_match_indices.cpu().numpy(), SCIBSMIS_texts_mean_NN_indes)
#_____________cross should get low score_____________

overleap(SCIB_texts_match_indices.cpu().numpy(), SCIBSMIS_texts_mean_NN_indes)
overleap(SCIB_texts_mean_match_indices.cpu().numpy(), SCIBSMIS_texts_cls_NN_indes)
print('='*20)
overleap(SCIB_extracted_text_match_indices.cpu().numpy(), SCIBSMIS_extracted_text_cls_NN_indes)
overleap(SCIB_extracted_text_mean_match_indices.cpu().numpy(), SCIBSMIS_extracted_text_mean_NN_indes)

#_____________cross should get low score_____________
overleap(SCIB_extracted_text_mean_match_indices.cpu().numpy(), SCIBSMIS_extracted_text_cls_NN_indes)
overleap(SCIB_extracted_text_match_indices.cpu().numpy(), SCIBSMIS_extracted_text_mean_NN_indes)


### text K-MEANS

In [ ]:
SCIB_cls_KM=KM(SCIB_cls_embedding.cpu().numpy())
SCIB_mean_KM=KM(SCIB_mean_embedding.cpu().numpy())
SCIB_extracted_text_cls_KM=KM(SCIB_extracted_text_cls_embedding.cpu().numpy())
SCIB_extracted_text_mean_KM=KM(SCIB_extracted_text_mean_embedding.cpu().numpy())

In [ ]:
SCIB_cls_KM_index=SCIB_cls_KM.predict(SCIB_cls_embedding.cpu().numpy())
SCIB_mean_KM_index=SCIB_mean_KM.predict(SCIB_mean_embedding.cpu().numpy())
SCIB_extracted_text_cls_KM_index=SCIB_extracted_text_cls_KM.predict(SCIB_extracted_text_cls_embedding.cpu().numpy())
SCIB_extracted_text_mean_KM_index=SCIB_extracted_text_mean_KM.predict(SCIB_extracted_text_mean_embedding.cpu().numpy())

In [ ]:
SCIB_cls_match_index=SCIB_cls_KM.predict(SCIB_cls_embedding[SCIB_texts_k1_match_indices].reshape(SCIB_cls_embedding.shape).cpu().numpy())
SCIB_mean_match_index=SCIB_mean_KM.predict(SCIB_mean_embedding[SCIB_texts_K1_mean_match_indices].reshape(SCIB_cls_embedding.shape).cpu().numpy())
SCIB_et_cls_match_index=SCIB_extracted_text_cls_KM.predict(SCIB_extracted_text_cls_embedding[SCIB_extracted_k1_text_match_indices].reshape(SCIB_cls_embedding.shape).cpu().numpy())
SCIB_et_mean_match_index=SCIB_extracted_text_mean_KM.predict(SCIB_extracted_text_mean_embedding[SCIB_extracted_k1_text_mean_match_indices].reshape(SCIB_cls_embedding.shape).cpu().numpy())

In [ ]:
SCIB_x_cls=SCIB_cls_match_index==SCIB_cls_KM_index
SCIB_x_mean=SCIB_mean_KM_index==SCIB_mean_match_index
SCIB_x_et_cls=SCIB_extracted_text_cls_KM_index==SCIB_et_cls_match_index
SCIB_x_et_mean=SCIB_et_mean_match_index==SCIB_extracted_text_mean_KM_index

In [ ]:
np.sum(SCIB_x_cls),np.sum(SCIB_x_mean),np.sum(SCIB_x_et_cls),np.sum(SCIB_x_et_mean)

##### Smis K-MEANS

In [ ]:
SCIBSMIS_cls_KM=KM(sim_embeddings.pooler_output.cpu().numpy().astype(np.float32))
SCIBSMIS_mean_KM=KM(sim_embeddings.pooler_output.cpu().numpy().astype(np.float32))
SCIBSMIS_extracted_text_cls_KM=KM(sim_embeddings.pooler_output.cpu().numpy().astype(np.float32))
SCIBSMIS_extracted_text_mean_KM=KM(sim_embeddings.pooler_output.cpu().numpy().astype(np.float32))

In [ ]:
SCIBSMIS_cls_KM_index=SCIBSMIS_cls_KM.predict(sim_embeddings.pooler_output.cpu().numpy().astype(np.float32))
SCIBSMIS_mean_KM_index=SCIBSMIS_mean_KM.predict(sim_embeddings.pooler_output.cpu().numpy().astype(np.float32))
SCIBSMIS_extracted_text_cls_KM_index=SCIBSMIS_extracted_text_cls_KM.predict(sim_embeddings.pooler_output.cpu().numpy().astype(np.float32))
SCIBSMIS_extracted_text_mean_KM_index=SCIBSMIS_extracted_text_mean_KM.predict(sim_embeddings.pooler_output.cpu().numpy().astype(np.float32))

In [ ]:
SCIBSMIS_cls_match_index=SCIBSMIS_cls_KM.predict(sim_embeddings.pooler_output[SCIB_texts_k1_match_indices].reshape(SCIB_cls_embedding.shape).cpu().numpy())
SCIBSMIS_mean_match_index=SCIBSMIS_mean_KM.predict(sim_embeddings.pooler_output[SCIB_texts_K1_mean_match_indices].reshape(SCIB_cls_embedding.shape).cpu().numpy())
SCIBSMIS_et_cls_match_index=SCIBSMIS_extracted_text_cls_KM.predict(sim_embeddings.pooler_output[SCIB_extracted_k1_text_match_indices].reshape(SCIB_cls_embedding.shape).cpu().numpy())
SCIBSMIS_et_mean_match_index=SCIBSMIS_extracted_text_mean_KM.predict(sim_embeddings.pooler_output[SCIB_extracted_k1_text_mean_match_indices].reshape(SCIB_cls_embedding.shape).cpu().numpy())

In [ ]:
SCIBSMIS_x_cls=SCIBSMIS_cls_match_index==SCIBSMIS_cls_KM_index
SCIBSMIS_x_mean=SCIBSMIS_mean_KM_index==SCIBSMIS_mean_match_index
SCIBSMIS_x_et_cls=SCIBSMIS_extracted_text_cls_KM_index==SCIBSMIS_et_cls_match_index
SCIBSMIS_x_et_mean=SCIBSMIS_et_mean_match_index==SCIBSMIS_extracted_text_mean_KM_index

In [ ]:
np.sum(SCIBSMIS_x_cls),np.sum(SCIBSMIS_x_mean),np.sum(SCIBSMIS_x_et_cls),np.sum(SCIBSMIS_x_et_mean)

## Pure BERT

In [ ]:
import torch
from transformers import BertTokenizer, BertModel
def pureBert(texts=texts):
    # 检查是否有可用的 GPU
    device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
    # device = torch.device('cuda:1')
    # 加载预训练的BERT模型和tokenizer
    model_name = 'bert-base-uncased'  # 你可以使用其他的BERT模型，例如 'bert-large-uncased'
    tokenizer = BertTokenizer.from_pretrained(model_name)
    model = BertModel.from_pretrained(model_name)

    # 将模型转移到 GPU
    model = model.to(device)

    # 将句子 token 化，并将输入转移到 GPU
    inputs = tokenizer(texts, return_tensors='pt', truncation=True, padding=True)
    inputs = {key: value.to(device) for key, value in inputs.items()}  # 转移到 GPU

    # 获取BERT的输出
    with torch.no_grad():  # 禁用梯度计算，节省内存
        outputs = model(**inputs)

    # BERT的最后隐藏层的输出 [batch_size, seq_len, hidden_size]
    last_hidden_states = outputs.last_hidden_state

    # 取 [CLS] token 的 embedding（句子嵌入）
    cls_embedding = last_hidden_states[:, 0, :]

    # 也可以取所有 token 的平均值作为句子嵌入
    mean_embedding = torch.mean(last_hidden_states, dim=1)

    print(f"CLS token embedding size: {cls_embedding.shape}")
    print(f"Mean token embedding size: {mean_embedding.shape}")

    return cls_embedding,mean_embedding


In [ ]:
PB_cls_embedding,PB_mean_embedding=pureBert(texts)

In [ ]:
PB_extracted_text_cls_embedding,PB_extracted_text_mean_embedding=pureBert(extracted_text)

### CKA

In [ ]:
de_biased_cka(PB_cls_embedding,sim_embeddings.pooler_output),de_biased_cka(PB_mean_embedding,sim_embeddings.pooler_output),de_biased_cka(PB_extracted_text_cls_embedding,sim_embeddings.pooler_output),de_biased_cka(PB_extracted_text_mean_embedding,sim_embeddings.pooler_output)

# (0.21898326, 0.15394284, 0.18666576, 0.16397773)

In [ ]:
CKA(PB_cls_embedding.cpu(),sim_embeddings.pooler_output),CKA(PB_mean_embedding.cpu(),sim_embeddings.pooler_output),CKA(PB_extracted_text_cls_embedding.cpu(),sim_embeddings.pooler_output),CKA(PB_extracted_text_mean_embedding.cpu(),sim_embeddings.pooler_output)

### Now, we have cls mean embedding of texts and extracted_text
cls_embedding,

mean_embedding,

extracted_text_cls_embedding,

extracted_text_mean_embedding,

#### exact match

In [ ]:
PB_texts_match_indices, PB_texts_match_scores, PB_texts_match_correct_total = retrieve(sim_embeddings.pooler_output, PB_cls_embedding.cpu().detach(), K=10)
PB_texts_mean_match_indices, PB_texts_mean_match_scores, PB_texts_mean_match_correct_total = retrieve(sim_embeddings.pooler_output, PB_mean_embedding.cpu().detach(), K=10)

# 打印结果
# print("Top K 索引：", indices)
# print("Top K 分数：", scores)
print("正确匹配的条目数：", PB_texts_match_correct_total)
print("正确匹配的条目数：", PB_texts_mean_match_correct_total)
PB_extracted_text_match_indices, PB_extracted_text_match_scores, PB_extracted_text_match_correct_total = retrieve(sim_embeddings.pooler_output, PB_extracted_text_cls_embedding.cpu().detach(), K=10)
PB_extracted_text_mean_match_indices, PB_extracted_text_mean_match_scores, PB_extracted_text_mean_match_correct_total = retrieve(sim_embeddings.pooler_output, PB_extracted_text_mean_embedding.cpu().detach(), K=10)

print("正确匹配的条目数：", PB_extracted_text_match_correct_total)
print("正确匹配的条目数：", PB_extracted_text_mean_match_correct_total)

In [ ]:
PB_texts_k1_match_indices, PB_texts_k1_match_scores, PB_texts_k1_match_correct_total = retrieve(sim_embeddings.pooler_output.cpu().detach(), PB_cls_embedding.cpu().detach(), K=1)
texts_K1_mean_match_indices, texts_k1_mean_match_scores, texts_k1_mean_match_correct_total = retrieve(sim_embeddings.pooler_output.cpu().detach(), PB_mean_embedding.cpu().detach(), K=1)

# 打印结果
# print("Top K 索引：", indices)
# print("Top K 分数：", scores)
print("正确匹配的条目数：", PB_texts_k1_match_correct_total)
print("正确匹配的条目数：", texts_k1_mean_match_correct_total)
PB_extracted_k1_text_match_indices, PB_extracted_k1_text_match_scores, PB_extracted_k1_text_match_correct_total = retrieve(sim_embeddings.pooler_output.cpu().detach(), PB_extracted_text_cls_embedding.cpu().detach(), K=1)
PB_extracted_k1_text_mean_match_indices, PB_extracted_k1_text_mean_match_scores, PB_extracted_text_k1_mean_match_correct_total = retrieve(sim_embeddings.pooler_output.cpu().detach(), PB_extracted_text_mean_embedding.cpu().detach(), K=1)

print("正确匹配的条目数：", PB_extracted_k1_text_match_correct_total)
print("正确匹配的条目数：", PB_extracted_text_k1_mean_match_correct_total)

#### text NN

In [ ]:
PB_texts_cls_NN=NN(embedding=PB_cls_embedding)
PB_texts_mean_NN=NN(embedding=PB_mean_embedding)
PB_extracted_text_cls_NN=NN(embedding=PB_extracted_text_cls_embedding)
PB_extracted_text_mean_NN=NN(embedding=PB_extracted_text_mean_embedding)

In [ ]:
PB_texts_cls_NN_distance,PB_texts_cls_NN_indes=PB_texts_cls_NN.kneighbors(sim_embeddings.pooler_output)
PB_,PB_texts_mean_NN_indes=PB_texts_mean_NN.kneighbors(sim_embeddings.pooler_output)

PB_extracted_text_cls_NN_distance,PB_extracted_text_cls_NN_indes=PB_extracted_text_cls_NN.kneighbors(sim_embeddings.pooler_output)
PB_extracted_text_mean_NN_distance,PB_extracted_text_mean_NN_indes=PB_extracted_text_mean_NN.kneighbors(sim_embeddings.pooler_output)

In [ ]:
overleap(PB_texts_match_indices.cpu().numpy(), PB_texts_cls_NN_indes)

overleap(PB_texts_mean_match_indices.cpu().numpy(), PB_texts_mean_NN_indes)
#_____________cross should get low score_____________

overleap(PB_texts_match_indices.cpu().numpy(), PB_texts_mean_NN_indes)
overleap(PB_texts_mean_match_indices.cpu().numpy(), PB_texts_cls_NN_indes)
print('='*30)
overleap(PB_extracted_text_match_indices.cpu().numpy(), PB_extracted_text_cls_NN_indes)
overleap(PB_extracted_text_mean_match_indices.cpu().numpy(), PB_extracted_text_mean_NN_indes)
#_____________cross should get low score_____________

overleap(PB_extracted_text_mean_match_indices.cpu().numpy(), PB_extracted_text_cls_NN_indes)
overleap(PB_extracted_text_match_indices.cpu().numpy(), PB_extracted_text_mean_NN_indes)


#### SMILES NN

In [ ]:
PBSMIS_texts_cls_NN=NN(embedding=sim_embeddings.pooler_output)
PBSMIS_texts_mean_NN=NN(embedding=sim_embeddings.pooler_output)
PBSMIS_extracted_text_cls_NN=NN(embedding=sim_embeddings.pooler_output)
PBSMIS_extracted_text_mean_NN=NN(embedding=sim_embeddings.pooler_output)

In [ ]:
PBSMIS_texts_cls_NN_distance,PBSMIS_texts_cls_NN_indes=PBSMIS_texts_cls_NN.kneighbors(PB_cls_embedding.cpu().numpy())
PBSMIS_texts_mean_NN_distance,PBSMIS_texts_mean_NN_indes=PBSMIS_texts_mean_NN.kneighbors(PB_mean_embedding.cpu().numpy())

PBSMIS_extracted_text_cls_NN_distance,PBSMIS_extracted_text_cls_NN_indes=PBSMIS_extracted_text_cls_NN.kneighbors(PB_extracted_text_cls_embedding.cpu().numpy())
PBSMIS_extracted_text_mean_NN_distance,PBSMIS_extracted_text_mean_NN_indes=PBSMIS_extracted_text_mean_NN.kneighbors(PB_extracted_text_mean_embedding.cpu().numpy())

In [ ]:
overleap(PB_texts_match_indices.cpu().numpy(), PBSMIS_texts_cls_NN_indes)

overleap(PB_texts_mean_match_indices.cpu().numpy(), PBSMIS_texts_mean_NN_indes)
#_____________cross should get low score_____________

overleap(PB_texts_match_indices.cpu().numpy(), PBSMIS_texts_mean_NN_indes)
overleap(PB_texts_mean_match_indices.cpu().numpy(), PBSMIS_texts_cls_NN_indes)
print('='*30)
overleap(PB_extracted_text_match_indices.cpu().numpy(), PBSMIS_extracted_text_cls_NN_indes)
overleap(PB_extracted_text_mean_match_indices.cpu().numpy(), PBSMIS_extracted_text_mean_NN_indes)

#_____________cross should get low score_____________
overleap(PB_extracted_text_mean_match_indices.cpu().numpy(), PBSMIS_extracted_text_cls_NN_indes)
overleap(PB_extracted_text_match_indices.cpu().numpy(), PBSMIS_extracted_text_mean_NN_indes)

### text K-MEANS

In [ ]:
PB_cls_KM=KM(PB_cls_embedding.cpu().numpy())
PB_mean_KM=KM(PB_mean_embedding.cpu().numpy())

PB_extracted_text_cls_KM=KM(PB_extracted_text_cls_embedding.cpu().numpy())
PB_extracted_text_mean_KM=KM(PB_extracted_text_mean_embedding.cpu().numpy())

In [ ]:
PB_cls_KM_index=PB_cls_KM.predict(PB_cls_embedding.cpu().numpy())
PB_mean_KM_index=PB_mean_KM.predict(PB_mean_embedding.cpu().numpy())

PB_extracted_text_cls_KM_index=PB_extracted_text_cls_KM.predict(PB_extracted_text_cls_embedding.cpu().numpy())
PB_extracted_text_mean_KM_index=PB_extracted_text_mean_KM.predict(PB_extracted_text_mean_embedding.cpu().numpy())

In [ ]:
PB_cls_match_index=PB_cls_KM.predict(PB_cls_embedding[PB_texts_k1_match_indices].reshape(PB_cls_embedding.shape).cpu().numpy())
PB_mean_match_index=PB_mean_KM.predict(PB_mean_embedding[texts_K1_mean_match_indices].reshape(PB_cls_embedding.shape).cpu().numpy())
PB_et_cls_match_index=PB_extracted_text_cls_KM.predict(PB_extracted_text_cls_embedding[PB_extracted_k1_text_match_indices].reshape(PB_cls_embedding.shape).cpu().numpy())
PB_et_mean_match_index=PB_extracted_text_mean_KM.predict(PB_extracted_text_mean_embedding[PB_extracted_k1_text_mean_match_indices].reshape(PB_cls_embedding.shape).cpu().numpy())

In [ ]:
PB_x_cls=PB_cls_match_index==PB_cls_KM_index
PB_x_mean=PB_mean_KM_index==PB_mean_match_index
PB_x_et_cls=PB_extracted_text_cls_KM_index==PB_et_cls_match_index
PB_x_et_mean=PB_et_mean_match_index==PB_extracted_text_mean_KM_index

In [ ]:
np.sum(PB_x_cls),np.sum(PB_x_mean),np.sum(PB_x_et_cls),np.sum(PB_x_et_mean)

##### Smis K-MEANS

In [ ]:
PBSMIS_cls_KM=KM(sim_embeddings.pooler_output.cpu().numpy().astype(np.float32))
PBSMIS_mean_KM=KM(sim_embeddings.pooler_output.cpu().numpy().astype(np.float32))

PBSMIS_extracted_text_cls_KM=KM(sim_embeddings.pooler_output.cpu().numpy().astype(np.float32))
PBSMIS_extracted_text_mean_KM=KM(sim_embeddings.pooler_output.cpu().numpy().astype(np.float32))

In [ ]:
PBSMIS_cls_KM_index=PBSMIS_cls_KM.predict(sim_embeddings.pooler_output.cpu().numpy().astype(np.float32))
PBSMIS_mean_KM_index=PBSMIS_mean_KM.predict(sim_embeddings.pooler_output.cpu().numpy().astype(np.float32))

PBSMIS_extracted_text_cls_KM_index=PBSMIS_extracted_text_cls_KM.predict(sim_embeddings.pooler_output.cpu().numpy().astype(np.float32))
PBSMIS_extracted_text_mean_KM_index=PBSMIS_extracted_text_mean_KM.predict(sim_embeddings.pooler_output.cpu().numpy().astype(np.float32))

In [ ]:
# c = sim_embeddings.pooler_output
# print("Dtype after pooler output:", c.dtype)  # Should be torch.float32 if using PyTorch
# c = c[texts_k1_match_indices].reshape(cls_embedding.shape).cpu().numpy()
# print("Dtype after conversion to numpy:", c.dtype)  # Should be float64 initially
# c = c.astype(np.float32)
# print("Dtype after astype(float32):", c.dtype)  # Confirm it is now float32
# cls_match_index = cls_KM.predict(c)
# # extracted_k1_text_match_indices

In [ ]:
PBSMIS_cls_match_index=PBSMIS_cls_KM.predict(sim_embeddings.pooler_output[PB_texts_k1_match_indices].reshape(PB_cls_embedding.shape).cpu().numpy())
PBSMIS_mean_match_index=PBSMIS_mean_KM.predict(sim_embeddings.pooler_output[texts_K1_mean_match_indices].reshape(PB_cls_embedding.shape).cpu().numpy())
PBSMIS_et_cls_match_index=PBSMIS_extracted_text_cls_KM.predict(sim_embeddings.pooler_output[PB_extracted_k1_text_match_indices].reshape(PB_cls_embedding.shape).cpu().numpy())
PBSMIS_et_mean_match_index=PBSMIS_extracted_text_mean_KM.predict(sim_embeddings.pooler_output[PB_extracted_k1_text_mean_match_indices].reshape(PB_cls_embedding.shape).cpu().numpy())

In [ ]:
PBSMIS_x_cls=PBSMIS_cls_match_index==PBSMIS_cls_KM_index
PBSMIS_x_mean=PBSMIS_mean_KM_index==PBSMIS_mean_match_index
PBSMIS_x_et_cls=PBSMIS_extracted_text_cls_KM_index==PBSMIS_et_cls_match_index
PBSMIS_x_et_mean=PBSMIS_et_mean_match_index==PBSMIS_extracted_text_mean_KM_index

In [ ]:
np.sum(PBSMIS_x_cls),np.sum(PBSMIS_x_mean),np.sum(PBSMIS_x_et_cls),np.sum(PBSMIS_x_et_mean)

## Sentence BERT

In [ ]:
torch.cuda.empty_cache()

In [ ]:
from sentence_transformers import SentenceTransformer

# 1. Load a pretrained Sentence Transformer model
model = SentenceTransformer("multi-qa-mpnet-base-dot-v1").to(device)
# 2. Calculate SB_embeddings by calling model.encode()
SB_embeddings = model.encode(texts)
print('SB_embeddings',SB_embeddings.shape)

SB_extracted_embeddings = model.encode(extracted_text)
print('SB_extracted_embeddings',SB_extracted_embeddings.shape)

### CKA

In [ ]:
de_biased_cka(torch.from_numpy(SB_embeddings),sim_embeddings.pooler_output),de_biased_cka(torch.from_numpy(SB_extracted_embeddings),sim_embeddings.pooler_output)

In [ ]:
CKA(torch.from_numpy(SB_embeddings),sim_embeddings.pooler_output),CKA(torch.from_numpy(SB_extracted_embeddings),sim_embeddings.pooler_output)

### exactly match

In [ ]:
K=10
SB_texts_match_indices, SB_texts_match_scores, SB_texts_match_correct_total = retrieve(sim_embeddings.pooler_output, torch.from_numpy(SB_embeddings).cpu().detach(), K=K)
SB_extracted_match_indices, SB_extracted_match_scores, SB_extracted_match_correct_total = retrieve(sim_embeddings.pooler_output, torch.from_numpy(SB_extracted_embeddings).cpu().detach(), K=K)

print(SB_texts_match_correct_total,SB_extracted_match_correct_total)

In [ ]:
K=1

SB_texts_k1_match_indices, SB_texts_k1_match_scores, SB_texts_k1_match_correct_total = retrieve(sim_embeddings.pooler_output.cpu().detach(), torch.from_numpy(SB_embeddings).cpu().detach(), K=1)
SB_extracted_k1_match_indices, SB_extracted_k1_match_scores, SB_extracted_k1_match_correct_total = retrieve(sim_embeddings.pooler_output.cpu().detach(), torch.from_numpy(SB_extracted_embeddings).cpu().detach(), K=1)

print(SB_texts_k1_match_correct_total,SB_extracted_k1_match_correct_total)

### text NN

In [ ]:
SB_texts_mean_NN=NN(embedding=torch.from_numpy(SB_embeddings))
SB_extracted_text_mean_NN=NN(embedding=torch.from_numpy(SB_extracted_embeddings))

In [ ]:
SB_emb_distance,SB_emb_index=SB_texts_mean_NN.kneighbors(sim_embeddings.pooler_output)
SB_et_distance,SB_et_index=SB_extracted_text_mean_NN.kneighbors(sim_embeddings.pooler_output)

In [ ]:
overleap(SB_texts_match_indices.cpu().numpy(), SB_emb_index)

overleap(SB_extracted_match_indices.cpu().numpy(), SB_et_index)

### SMILES NN

In [ ]:
SBSMIS_texts_mean_NN=NN(embedding=sim_embeddings.pooler_output)
SBSMIS_extracted_text_mean_NN=NN(embedding=sim_embeddings.pooler_output)

In [ ]:
SBSMIS_emb_distance,SBSMIS_emb_index=SBSMIS_texts_mean_NN.kneighbors(SB_embeddings)
SBSMIS_et_distance,SBSMIS_et_index=SBSMIS_extracted_text_mean_NN.kneighbors(SB_extracted_embeddings)

In [ ]:
overleap(SB_texts_match_indices.cpu().numpy(), SBSMIS_emb_index)

overleap(SB_extracted_match_indices.cpu().numpy(), SBSMIS_et_index)

### K-means
#### text K-means

In [ ]:
SB_cls_KM=KM(SB_embeddings)

SB_extracted_text_KM=KM(SB_extracted_embeddings)
SB_cls=SB_cls_KM.predict(SB_embeddings)
SB_extracted_text_cls=SB_extracted_text_KM.predict(SB_extracted_embeddings)
SB_cls_index=SB_cls_KM.predict(SB_embeddings[SB_texts_k1_match_indices].reshape(SB_embeddings.shape))
SB_et_index=SB_extracted_text_KM.predict(SB_extracted_embeddings[SB_extracted_k1_match_indices].reshape(SB_extracted_embeddings.shape))
SB_x_cls=SB_cls_index==SB_cls
SB_et=SB_et_index==SB_extracted_text_cls
np.sum(SB_x_cls),np.sum(SB_et)


#### SMIS

In [ ]:
#### SMIS
SBSMIS_cls_KM=KM(sim_embeddings.pooler_output.cpu().numpy().astype(np.float32))

SBSMIS_extracted_text_KM=KM(sim_embeddings.pooler_output.cpu().numpy().astype(np.float32))

SBSMIS_cls=SBSMIS_cls_KM.predict(sim_embeddings.pooler_output.cpu().numpy().astype(np.float32))
SBSMIS_extracted_text_cls=SBSMIS_extracted_text_KM.predict(sim_embeddings.pooler_output.cpu().numpy().astype(np.float32))

SBSMIS_cls_index=SBSMIS_cls_KM.predict(sim_embeddings.pooler_output[SB_texts_k1_match_indices].reshape(SB_embeddings.shape).cpu().numpy())
SBSMIS_et_index=SBSMIS_extracted_text_KM.predict(sim_embeddings.pooler_output[SB_extracted_k1_match_indices].reshape(SB_embeddings.shape).cpu().numpy())

SBSMIS_x_cls=SBSMIS_cls_index==SBSMIS_cls
SBSMIS_et=SBSMIS_et_index==SBSMIS_extracted_text_cls
np.sum(SBSMIS_x_cls),np.sum(SBSMIS_et)

## GPT

In [ ]:
from transformers import AutoTokenizer, OpenAIGPTModel
import torch
def get_GPT_embedding(texts=texts):
    # Load tokenizer and model
    tokenizer = AutoTokenizer.from_pretrained("openai-community/openai-gpt")
    model = OpenAIGPTModel.from_pretrained("openai-community/openai-gpt")

    # Set the padding token
    if tokenizer.pad_token is None:
        tokenizer.add_special_tokens({'pad_token': '[PAD]'})
        model.resize_token_embeddings(len(tokenizer))

    # Move model to GPU if available
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    # Tokenize input texts
    inputs = tokenizer(texts, return_tensors="pt", truncation=True, padding=True)

    # Move inputs to the same device as the model
    inputs = {key: value.to(device) for key, value in inputs.items()}

    # Run model to get hidden states
    with torch.no_grad():  # Disable gradient calculation for inference
        outputs = model(**inputs)

    # Access last hidden states
    last_hidden_states = outputs.last_hidden_state
    print(last_hidden_states.shape)

    # 获取最后的隐藏状态 [batch_size, seq_len, hidden_size]
    last_hidden_states = outputs.last_hidden_state

    # 获取第一个 token (<s> token) 的 embedding（相当于 CLS token 的句子嵌入）
    cls_embedding = last_hidden_states[:, 0, :]

    # 也可以取所有 token 的平均值作为句子嵌入
    mean_embedding = torch.mean(last_hidden_states, dim=1)

    print(f"CLS token embedding size: {cls_embedding.shape}")
    print(f"Mean token embedding size: {mean_embedding.shape}")
    return cls_embedding,mean_embedding


In [ ]:
# GPT_cls_embedding,GPT_mean_embedding=get_GPT_embedding(texts)

GPT_cls_embedding1,GPT_mean_embedding1=get_GPT_embedding(texts[:1000])
GPT_cls_embedding2,GPT_mean_embedding2=get_GPT_embedding(texts[1000:2000])
GPT_cls_embedding3,GPT_mean_embedding3=get_GPT_embedding(texts[2000:3000])
GPT_cls_embedding4,GPT_mean_embedding4=get_GPT_embedding(texts[3000:])
GPT_cls_embedding=torch.cat((GPT_cls_embedding1,GPT_cls_embedding2,GPT_cls_embedding3,GPT_cls_embedding4))
GPT_mean_embedding=torch.cat((GPT_mean_embedding1,GPT_mean_embedding2,GPT_mean_embedding3,GPT_mean_embedding4))
GPT_cls_embedding.shape,GPT_mean_embedding.shape

In [ ]:
GPT_extracted_text_cls_embedding1,GPT_extracted_text_mean_embedding1=get_GPT_embedding(extracted_text[:1000])
GPT_extracted_text_cls_embedding2,GPT_extracted_text_mean_embedding2=get_GPT_embedding(extracted_text[1000:2000])
GPT_extracted_text_cls_embedding3,GPT_extracted_text_mean_embedding3=get_GPT_embedding(extracted_text[2000:3000])
GPT_extracted_text_cls_embedding4,GPT_extracted_text_mean_embedding4=get_GPT_embedding(extracted_text[3000:])
GPT_extracted_text_cls_embedding=torch.cat((GPT_extracted_text_cls_embedding1,GPT_extracted_text_cls_embedding2,GPT_extracted_text_cls_embedding3,GPT_extracted_text_cls_embedding4))
GPT_extracted_text_mean_embedding=torch.cat((GPT_extracted_text_mean_embedding1,GPT_extracted_text_mean_embedding2,GPT_extracted_text_mean_embedding3,GPT_extracted_text_mean_embedding4))
GPT_extracted_text_cls_embedding.shape,GPT_extracted_text_mean_embedding.shape

### CKA

In [ ]:
de_biased_cka(GPT_cls_embedding,sim_embeddings.pooler_output),de_biased_cka(GPT_mean_embedding,sim_embeddings.pooler_output),de_biased_cka(GPT_extracted_text_cls_embedding,sim_embeddings.pooler_output),de_biased_cka(GPT_extracted_text_mean_embedding,sim_embeddings.pooler_output)

In [ ]:
CKA(GPT_cls_embedding.cpu(),sim_embeddings.pooler_output),CKA(GPT_mean_embedding.cpu(),sim_embeddings.pooler_output),CKA(GPT_extracted_text_cls_embedding.cpu(),sim_embeddings.pooler_output),CKA(GPT_extracted_text_mean_embedding.cpu(),sim_embeddings.pooler_output)

### Now, we have cls mean embedding of texts and extracted_text
cls_embedding,

mean_embedding,

extracted_text_cls_embedding,

extracted_text_mean_embedding,

#### exact match

In [ ]:
GPT_texts_match_indices, GPT_texts_match_scores, GPT_texts_match_correct_total = retrieve(sim_embeddings.pooler_output, GPT_cls_embedding.cpu().detach(), K=10)
GPT_texts_mean_match_indices, GPT_texts_mean_match_scores, GPT_texts_mean_match_correct_total = retrieve(sim_embeddings.pooler_output, GPT_mean_embedding.cpu().detach(), K=10)

# 打印结果
# print("Top K 索引：", indices)
# print("Top K 分数：", scores)
print("正确匹配的条目数：", GPT_texts_match_correct_total)
print("正确匹配的条目数：", GPT_texts_mean_match_correct_total)
GPT_extracted_text_match_indices, GPT_extracted_text_match_scores, GPTextracted_text_match_correct_total = retrieve(sim_embeddings.pooler_output, GPT_extracted_text_cls_embedding.cpu().detach(), K=10)
GPT_extracted_text_mean_match_indices, GPT_extracted_text_mean_match_scores, GPT_extracted_text_mean_match_correct_total = retrieve(sim_embeddings.pooler_output, GPT_extracted_text_mean_embedding.cpu().detach(), K=10)

print("正确匹配的条目数：", GPTextracted_text_match_correct_total)
print("正确匹配的条目数：", GPT_extracted_text_mean_match_correct_total)

In [ ]:
GPT_texts_k1_match_indices, GPT_texts_k1_match_scores, GPT_texts_k1_match_correct_total = retrieve(sim_embeddings.pooler_output.cpu().detach(), GPT_cls_embedding.cpu().detach(), K=1)
GPT_texts_K1_mean_match_indices, GPT_texts_k1_mean_match_scores, GPT_texts_k1_mean_match_correct_total = retrieve(sim_embeddings.pooler_output.cpu().detach(), GPT_mean_embedding.cpu().detach(), K=1)

# 打印结果
# print("Top K 索引：", indices)
# print("Top K 分数：", scores)
print("正确匹配的条目数：", GPT_texts_k1_match_correct_total)
print("正确匹配的条目数：", GPT_texts_k1_mean_match_correct_total)
GPT_extracted_k1_text_match_indices, GPT_extracted_k1_text_match_scores, GPT_extracted_k1_text_match_correct_total = retrieve(sim_embeddings.pooler_output.cpu().detach(), GPT_extracted_text_cls_embedding.cpu().detach(), K=1)
GPT_extracted_k1_text_mean_match_indices, GPT_extracted_k1_text_mean_match_scores, GPT_extracted_text_k1_mean_match_correct_total = retrieve(sim_embeddings.pooler_output.cpu().detach(), GPT_extracted_text_mean_embedding.cpu().detach(), K=1)

print("正确匹配的条目数：", GPT_extracted_k1_text_match_correct_total)
print("正确匹配的条目数：", GPT_extracted_text_k1_mean_match_correct_total)

#### text NN

In [ ]:
GPT_texts_cls_NN=NN(embedding=GPT_cls_embedding)
GPT_texts_mean_NN=NN(embedding=GPT_mean_embedding)
GPT_extracted_text_cls_NN=NN(embedding=GPT_extracted_text_cls_embedding)
GPT_extracted_text_mean_NN=NN(embedding=GPT_extracted_text_mean_embedding)

In [ ]:
CPT_texts_cls_NN_distance,GPT_texts_cls_NN_indes=GPT_texts_cls_NN.kneighbors(sim_embeddings.pooler_output)
GPT_texts_mean_NN_distance,GPT_texts_mean_NN_indes=GPT_texts_mean_NN.kneighbors(sim_embeddings.pooler_output)

GPT_extracted_text_cls_NN_distance,GPT_extracted_text_cls_NN_indes=GPT_extracted_text_cls_NN.kneighbors(sim_embeddings.pooler_output)
GPT_extracted_text_mean_NN_distance,GPT_extracted_text_mean_NN_indes=GPT_extracted_text_mean_NN.kneighbors(sim_embeddings.pooler_output)

In [ ]:
overleap(GPT_texts_match_indices.cpu().numpy(), GPT_texts_cls_NN_indes)

overleap(GPT_texts_mean_match_indices.cpu().numpy(), GPT_texts_mean_NN_indes)
#_____________cross should get low score_____________

overleap(GPT_texts_match_indices.cpu().numpy(), GPT_texts_mean_NN_indes)
overleap(GPT_texts_mean_match_indices.cpu().numpy(), GPT_texts_cls_NN_indes)
print('='*10)
overleap(GPT_extracted_text_match_indices.cpu().numpy(), GPT_extracted_text_cls_NN_indes)
overleap(GPT_extracted_text_mean_match_indices.cpu().numpy(), GPT_extracted_text_mean_NN_indes)
#_____________cross should get low score_____________

overleap(GPT_extracted_text_mean_match_indices.cpu().numpy(), GPT_extracted_text_cls_NN_indes)
overleap(GPT_extracted_text_match_indices.cpu().numpy(), GPT_extracted_text_mean_NN_indes)

#### SMILES NN

In [ ]:
GPTSMIS_texts_cls_NN=NN(embedding=sim_embeddings.pooler_output)
GPTSMIS_texts_mean_NN=NN(embedding=sim_embeddings.pooler_output)
GPTSMIS_extracted_text_cls_NN=NN(embedding=sim_embeddings.pooler_output)
GPTSMIS_extracted_text_mean_NN=NN(embedding=sim_embeddings.pooler_output)

In [ ]:
CPTSMIS_texts_cls_NN_distance,GPTSMIS_texts_cls_NN_indes=GPTSMIS_texts_cls_NN.kneighbors(GPT_cls_embedding.cpu().numpy())
GPTSMIS_texts_mean_NN_distance,GPTSMIS_texts_mean_NN_indes=GPTSMIS_texts_mean_NN.kneighbors(GPT_mean_embedding.cpu().numpy())

GPTSMIS_extracted_text_cls_NN_distance,GPTSMIS_extracted_text_cls_NN_indes=GPTSMIS_extracted_text_cls_NN.kneighbors(GPT_extracted_text_cls_embedding.cpu().numpy())
GPTSMIS_extracted_text_mean_NN_distance,GPTSMIS_extracted_text_mean_NN_indes=GPTSMIS_extracted_text_mean_NN.kneighbors(GPT_extracted_text_mean_embedding.cpu().numpy())

In [ ]:
overleap(GPT_texts_match_indices.cpu().numpy(), GPTSMIS_texts_cls_NN_indes)

overleap(GPT_texts_mean_match_indices.cpu().numpy(), GPTSMIS_texts_mean_NN_indes)

#_____________cross should get low score_____________

overleap(GPT_texts_match_indices.cpu().numpy(), GPTSMIS_texts_mean_NN_indes)
overleap(GPT_texts_mean_match_indices.cpu().numpy(), GPTSMIS_texts_cls_NN_indes)
print('='*30)
overleap(GPT_extracted_text_match_indices.cpu().numpy(), GPTSMIS_extracted_text_cls_NN_indes)
overleap(GPT_extracted_text_mean_match_indices.cpu().numpy(), GPTSMIS_extracted_text_mean_NN_indes)

#_____________cross should get low score_____________
overleap(GPT_extracted_text_mean_match_indices.cpu().numpy(), GPTSMIS_extracted_text_cls_NN_indes)
overleap(GPT_extracted_text_match_indices.cpu().numpy(), GPTSMIS_extracted_text_mean_NN_indes)

### text K-MEANS

In [ ]:
GPT_cls_KM=KM(GPT_cls_embedding.cpu().numpy())
GPT_mean_KM=KM(GPT_mean_embedding.cpu().numpy())

GPT_extracted_text_cls_KM=KM(GPT_extracted_text_cls_embedding.cpu().numpy())
GPT_extracted_text_mean_KM=KM(GPT_extracted_text_mean_embedding.cpu().numpy())

In [ ]:
GPT_cls_KM_index=GPT_cls_KM.predict(GPT_cls_embedding.cpu().numpy())
GPT_mean_KM_index=GPT_mean_KM.predict(GPT_mean_embedding.cpu().numpy())

GPT_extracted_text_cls_KM_index=GPT_extracted_text_cls_KM.predict(GPT_extracted_text_cls_embedding.cpu().numpy())
GPT_extracted_text_mean_KM_index=GPT_extracted_text_mean_KM.predict(GPT_extracted_text_mean_embedding.cpu().numpy())

In [ ]:
GPT_cls_match_index=GPT_cls_KM.predict(GPT_cls_embedding[GPT_texts_k1_match_indices].reshape(GPT_cls_embedding.shape).cpu().numpy())
GPT_mean_match_index=GPT_mean_KM.predict(GPT_mean_embedding[GPT_texts_K1_mean_match_indices].reshape(GPT_cls_embedding.shape).cpu().numpy())
GPT_et_cls_match_index=GPT_extracted_text_cls_KM.predict(GPT_extracted_text_cls_embedding[GPT_extracted_k1_text_match_indices].reshape(GPT_cls_embedding.shape).cpu().numpy())
GPT_et_mean_match_index=GPT_extracted_text_mean_KM.predict(GPT_extracted_text_mean_embedding[GPT_extracted_k1_text_mean_match_indices].reshape(GPT_cls_embedding.shape).cpu().numpy())

In [ ]:
GPT_x_cls=GPT_cls_match_index==GPT_cls_KM_index
GPT_x_mean=GPT_mean_KM_index==GPT_mean_match_index
GPT_x_et_cls=GPT_extracted_text_cls_KM_index==GPT_et_cls_match_index
GPT_x_et_mean=GPT_et_mean_match_index==GPT_extracted_text_mean_KM_index
np.sum(GPT_x_cls),np.sum(GPT_x_mean),np.sum(GPT_x_et_cls),np.sum(GPT_x_et_mean)

##### Smis K-MEANS

In [ ]:
GPTSMIS_cls_KM=KM(sim_embeddings.pooler_output.cpu().numpy().astype(np.float32))
GPTSMIS_mean_KM=KM(sim_embeddings.pooler_output.cpu().numpy().astype(np.float32))

GPTSMIS_extracted_text_cls_KM=KM(sim_embeddings.pooler_output.cpu().numpy().astype(np.float32))
GPTSMIS_extracted_text_mean_KM=KM(sim_embeddings.pooler_output.cpu().numpy().astype(np.float32))

In [ ]:
GPTSMIS_cls_KM_index=GPTSMIS_cls_KM.predict(sim_embeddings.pooler_output.cpu().numpy().astype(np.float32))
GPTSMIS_mean_KM_index=GPTSMIS_mean_KM.predict(sim_embeddings.pooler_output.cpu().numpy().astype(np.float32))

GPTSMIS_extracted_text_cls_KM_index=GPTSMIS_extracted_text_cls_KM.predict(sim_embeddings.pooler_output.cpu().numpy().astype(np.float32))
GPTSMIS_extracted_text_mean_KM_index=GPTSMIS_extracted_text_mean_KM.predict(sim_embeddings.pooler_output.cpu().numpy().astype(np.float32))

In [ ]:
GPTSMIS_cls_match_index=GPTSMIS_cls_KM.predict(sim_embeddings.pooler_output[GPT_texts_k1_match_indices].reshape(GPT_cls_embedding.shape).cpu().numpy())
GPTSMIS_mean_match_index=GPTSMIS_mean_KM.predict(sim_embeddings.pooler_output[GPT_texts_K1_mean_match_indices].reshape(GPT_cls_embedding.shape).cpu().numpy())
GPTSMIS_et_cls_match_index=GPTSMIS_extracted_text_cls_KM.predict(sim_embeddings.pooler_output[GPT_extracted_k1_text_match_indices].reshape(GPT_cls_embedding.shape).cpu().numpy())
GPTSMIS_et_mean_match_index=GPTSMIS_extracted_text_mean_KM.predict(sim_embeddings.pooler_output[GPT_extracted_k1_text_mean_match_indices].reshape(GPT_cls_embedding.shape).cpu().numpy())

In [ ]:
GPTSMIS_x_cls=GPTSMIS_cls_match_index==GPTSMIS_cls_KM_index
GPTSMIS_x_mean=GPTSMIS_mean_KM_index==GPTSMIS_mean_match_index
GPTSMIS_x_et_cls=GPTSMIS_extracted_text_cls_KM_index==GPTSMIS_et_cls_match_index
GPTSMIS_x_et_mean=GPTSMIS_et_mean_match_index==GPTSMIS_extracted_text_mean_KM_index
np.sum(GPTSMIS_x_cls),np.sum(GPTSMIS_x_mean),np.sum(GPTSMIS_x_et_cls),np.sum(GPTSMIS_x_et_mean)

## last word GPT

In [ ]:
import torch
from transformers import AutoTokenizer, OpenAIGPTModel

def Last_GPTembedding(texts):

    # Check if GPU is available and set the device
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

    # Load pre-trained model and tokenizer and move the model to the GPU
    tokenizer = AutoTokenizer.from_pretrained("openai-community/openai-gpt")
    model = OpenAIGPTModel.from_pretrained("openai-community/openai-gpt").to(device)
    
    if tokenizer.pad_token is None:
        tokenizer.add_special_tokens({'pad_token': '[PAD]'})
        model.resize_token_embeddings(len(tokenizer))
    # Input sentences
    # texts = ['Your sentence here1', 'Your sentence here2']

    # Tokenize input sentence and move input tensors to the GPU

    template = 'This_sentence_:_"The_molecule_is_a_nitrile_that_is_acetonitrile_where_one_of_the_methyl_hydrogens_is_substituted_by_a_2-methylphenyl_group."_means_in_one_word:"Acetonitrile".This_sentence_:_"*sent_0*"_means_in_one_word:"'
    inputs = tokenizer([template.replace('*sent_0*', i).replace('_', ' ') for i in texts], padding=True, truncation=True,  max_length=512,return_tensors="pt")['input_ids'].to(device)
    with torch.no_grad():
        embeddings = model(inputs, output_hidden_states=True, return_dict=True).hidden_states[-1][:, -1, :]
    print(embeddings.shape)
    return embeddings



In [ ]:
# LastGPT_embeddings=Last_GPTembedding(texts)
LastGPT_embeddings1=Last_GPTembedding(texts[:1000])
LastGPT_embeddings2=Last_GPTembedding(texts[1000:2000])
LastGPT_embeddings3=Last_GPTembedding(texts[2000:3000])
LastGPT_embeddings4=Last_GPTembedding(texts[3000:])

LastGPT_embeddings=torch.cat((LastGPT_embeddings1,LastGPT_embeddings2,LastGPT_embeddings3,LastGPT_embeddings4))

In [ ]:
# LastGPT_et_embeddings=Last_GPTembedding(extracted_text)
LastGPT_et_embeddings1=Last_GPTembedding(extracted_text[:1000])
LastGPT_et_embeddings2=Last_GPTembedding(extracted_text[1000:2000])
LastGPT_et_embeddings3=Last_GPTembedding(extracted_text[2000:3000])
LastGPT_et_embeddings4=Last_GPTembedding(extracted_text[3000:])

LastGPT_et_embeddings=torch.cat((LastGPT_et_embeddings1,LastGPT_et_embeddings2,LastGPT_et_embeddings3,LastGPT_et_embeddings4))

In [ ]:
LastGPT_embeddings.shape,LastGPT_et_embeddings.shape

In [ ]:
de_biased_cka(LastGPT_embeddings,sim_embeddings.pooler_output),de_biased_cka(LastGPT_et_embeddings,sim_embeddings.pooler_output)

In [ ]:
CKA(LastGPT_embeddings,sim_embeddings.pooler_output.to(device)),CKA(LastGPT_et_embeddings,sim_embeddings.pooler_output.to(device))

In [ ]:
K=10

LastGPT_texts_match_indices, LastGPT_texts_match_scores, LastGPT_texts_match_correct_total = retrieve(sim_embeddings.pooler_output.cpu().detach(), LastGPT_embeddings.cpu().detach(), K=K)

# 打印结果
# print("Top K 索引：", indices)
# print("Top K 分数：", scores)
print("正确匹配的条目数：", LastGPT_texts_match_correct_total)
LastGPT_extracted_text_match_indices, LastGPT_extracted_text_match_scores, LastGPT_extracted_text_match_correct_total = retrieve(sim_embeddings.pooler_output.cpu().detach(), LastGPT_et_embeddings.cpu().detach(), K=K)

print("正确匹配的条目数：", LastGPT_extracted_text_match_correct_total)


In [ ]:
K=1

LastGPT_texts_k1_match_indices, LastGPT_texts_match_scores, LastGPT_texts_match_correct_total = retrieve(sim_embeddings.pooler_output.cpu().detach(), LastGPT_embeddings.cpu().detach(), K=K)

# 打印结果
# print("Top K 索引：", indices)
# print("Top K 分数：", scores)
print("正确匹配的条目数：", LastGPT_texts_match_correct_total)
LastGPT_extracted_k1_text_match_indices, LastGPT_extracted_text_match_scores, LastGPT_extracted_text_match_correct_total = retrieve(sim_embeddings.pooler_output.cpu().detach(), LastGPT_et_embeddings.cpu().detach(), K=K)

print("正确匹配的条目数：", LastGPT_extracted_text_match_correct_total)

### text

In [ ]:
LastGPT_texts_cls_NN=NN(embedding=LastGPT_embeddings)
LastGPT_extracted_text_cls_NN=NN(embedding=LastGPT_et_embeddings)

In [ ]:
LastGPT_texts_cls_NN_distance,LastGPT_texts_cls_NN_indes=LastGPT_texts_cls_NN.kneighbors(sim_embeddings.pooler_output.cpu().detach())

LastGPT_extracted_text_cls_NN_distance,LastGPT_extracted_text_cls_NN_indes=LastGPT_extracted_text_cls_NN.kneighbors(sim_embeddings.pooler_output.cpu().detach())

In [ ]:
overleap(LastGPT_texts_match_indices.cpu().numpy(), LastGPT_texts_cls_NN_indes)

#_____________cross should get low score_____________

overleap(LastGPT_extracted_text_match_indices.cpu().numpy(), LastGPT_extracted_text_cls_NN_indes)

In [ ]:
LastGPT_cls_KM=KM(LastGPT_embeddings.cpu().numpy())

LastGPT_extracted_text_cls_KM=KM(LastGPT_et_embeddings.cpu().numpy())

In [ ]:
LastGPT_cls_KM_index=LastGPT_cls_KM.predict(LastGPT_embeddings.cpu().numpy())

LastGPT_extracted_text_cls_KM_index=LastGPT_extracted_text_cls_KM.predict(LastGPT_et_embeddings.cpu().numpy())

In [ ]:
LastGPT_cls_match_index=LastGPT_cls_KM.predict(LastGPT_embeddings[LastGPT_texts_k1_match_indices].reshape(LastGPT_embeddings.shape).cpu().numpy())
LastGPT_et_cls_match_index=LastGPT_extracted_text_cls_KM.predict(LastGPT_et_embeddings[LastGPT_extracted_k1_text_match_indices].reshape(LastGPT_et_embeddings.shape).cpu().numpy())

In [ ]:
LastGPT_x_cls=LastGPT_cls_match_index==LastGPT_cls_KM_index
LastGPT_x_et_cls=LastGPT_extracted_text_cls_KM_index==LastGPT_et_cls_match_index

In [ ]:
np.sum(LastGPT_x_cls),np.sum(LastGPT_x_et_cls)

### SMIELS

In [ ]:
LastGPTSMIS_texts_cls_NN=NN(embedding=sim_embeddings.pooler_output)
LastGPTSMIS_extracted_text_cls_NN=NN(embedding=sim_embeddings.pooler_output)

In [ ]:
LastGPTSMIS_texts_cls_NN_distance,LastGPTSMIS_texts_cls_NN_indes=LastGPTSMIS_texts_cls_NN.kneighbors(LastGPT_embeddings.cpu().numpy())

LastGPTSMIS_extracted_text_cls_NN_distance,LastGPTSMIS_extracted_text_cls_NN_indes=LastGPTSMIS_extracted_text_cls_NN.kneighbors(LastGPT_et_embeddings.cpu().numpy())

In [ ]:
overleap(LastGPT_texts_match_indices.cpu().numpy(), LastGPTSMIS_texts_cls_NN_indes)

#_____________cross should get low score_____________

overleap(LastGPT_extracted_text_match_indices.cpu().numpy(), LastGPTSMIS_extracted_text_cls_NN_indes)

In [ ]:
LastGPTSMIS_cls_KM=KM(sim_embeddings.pooler_output.cpu().numpy().astype(np.float32))

LastGPTSMIS_extracted_text_cls_KM=KM(sim_embeddings.pooler_output.cpu().numpy().astype(np.float32))

In [ ]:
LastGPTSMIS_cls_KM_index=LastGPTSMIS_cls_KM.predict(sim_embeddings.pooler_output.cpu().numpy().astype(np.float32))

LastGPTSMIS_extracted_text_cls_KM_index=LastGPTSMIS_extracted_text_cls_KM.predict(sim_embeddings.pooler_output.cpu().numpy().astype(np.float32))

In [ ]:
LastGPTSMIS_cls_match_index=LastGPTSMIS_cls_KM.predict(sim_embeddings.pooler_output[LastGPT_texts_k1_match_indices].reshape(LastGPT_embeddings.shape).cpu().numpy().astype(np.float32))
LastGPTSMIS_et_cls_match_index=LastGPTSMIS_extracted_text_cls_KM.predict(sim_embeddings.pooler_output[LastGPT_extracted_k1_text_match_indices].reshape(LastGPT_et_embeddings.shape).cpu().numpy().astype(np.float32))
LastGPTSMIS_x_cls=LastGPTSMIS_cls_match_index==LastGPTSMIS_cls_KM_index
LastGPTSMIS_x_et_cls=LastGPTSMIS_extracted_text_cls_KM_index==LastGPTSMIS_et_cls_match_index

np.sum(LastGPTSMIS_x_cls),np.sum(LastGPTSMIS_x_et_cls)

In [ ]:
globals().keys()

In [ ]:
del AutoTokenizer,
del AutoModel,
del SCIBERT,
del SCIB_cls_embedding,
del SCIB_mean_embedding,
del SCIB_extracted_text_cls_embedding,
del SCIB_extracted_text_mean_embedding,
del SCIB_texts_cls_NN,
del SCIB_texts_mean_NN,
del SCIB_extracted_text_cls_NN,
del SCIB_extracted_text_mean_NN,
del SCIB_texts_cls_NN_distance,
del SCIB_texts_cls_NN_indes,
del SCIB_texts_mean_NN_distance,
del SCIB_texts_mean_NN_indes,
del SCIB_extracted_text_cls_NN_distance,
del SCIB_extracted_text_cls_NN_indes,
del SCIB_extracted_text_mean_NN_distance,
del SCIB_extracted_text_mean_NN_indes,
del SCIB_texts_match_indices,
del SCIB_texts_match_scores,
del SCIB_texts_match_correct_total,
del SCIB_texts_mean_match_indices,
del SCIB_texts_mean_match_scores,
del SCIB_texts_mean_match_correct_total,
del SCIB_extracted_text_match_indices,
del SCIB_extracted_text_match_scores,
del SCIB_extracted_text_match_correct_total,
del SCIB_extracted_text_mean_match_indices,
del SCIB_extracted_text_mean_match_scores,
del SCIB_extracted_text_mean_match_correct_total,
del SCIB_texts_k1_match_indices,
del SCIB_texts_k1_match_scores,
del SCIB_texts_k1_match_correct_total,
del SCIB_texts_K1_mean_match_indices,
del SCIB_texts_k1_mean_match_scores,
del SCIB_texts_k1_mean_match_correct_total,
del SCIB_extracted_k1_text_match_indices,
del SCIB_extracted_k1_text_match_scores,
del SCIB_extracted_k1_text_match_correct_total,
del SCIB_extracted_k1_text_mean_match_indices,
del SCIB_extracted_k1_text_mean_match_scores,
del SCIB_extracted_text_k1_mean_match_correct_total,
del SCIBSMIS_texts_cls_NN,
del SCIBSMIS_texts_mean_NN,
del SCIBSMIS_extracted_text_cls_NN,
del SCIBSMIS_extracted_text_mean_NN,
del SCIBSMIS_texts_cls_NN_distance,
del SCIBSMIS_texts_cls_NN_indes,
del SCIBSMIS_texts_mean_NN_distance,
del SCIBSMIS_texts_mean_NN_indes,
del SCIBSMIS_extracted_text_cls_NN_distance,
del SCIBSMIS_extracted_text_cls_NN_indes,
del SCIBSMIS_extracted_text_mean_NN_distance,
del SCIBSMIS_extracted_text_mean_NN_indes,
del SCIB_cls_KM,
del SCIB_mean_KM,
del SCIB_extracted_text_cls_KM,
del SCIB_extracted_text_mean_KM,
del SCIB_cls_KM_index,
del SCIB_mean_KM_index,
del SCIB_extracted_text_cls_KM_index,
del SCIB_extracted_text_mean_KM_index,
del SCIB_cls_match_index,
del SCIB_mean_match_index,
del SCIB_et_cls_match_index,
del SCIB_et_mean_match_index,
del SCIB_x_cls,
del SCIB_x_mean,
del SCIB_x_et_cls,
del SCIB_x_et_mean,
del SCIBSMIS_cls_KM,
del SCIBSMIS_mean_KM,
del SCIBSMIS_extracted_text_cls_KM,
del SCIBSMIS_extracted_text_mean_KM,
del SCIBSMIS_cls_KM_index,
del SCIBSMIS_mean_KM_index,
del SCIBSMIS_extracted_text_cls_KM_index,
del SCIBSMIS_extracted_text_mean_KM_index,
del SCIBSMIS_cls_match_index,
del SCIBSMIS_mean_match_index,
del SCIBSMIS_et_cls_match_index,
del SCIBSMIS_et_mean_match_index,
del SCIBSMIS_x_cls,
del SCIBSMIS_x_mean,
del SCIBSMIS_x_et_cls,
del SCIBSMIS_x_et_mean,
del BertTokenizer,
del BertModel,
del pureBert,
del PB_cls_embedding,
del PB_mean_embedding,
del PB_extracted_text_cls_embedding,
del PB_extracted_text_mean_embedding,
del PB_texts_match_indices,
del PB_texts_match_scores,
del PB_texts_match_correct_total,
del PB_texts_mean_match_indices,
del PB_texts_mean_match_scores,
del PB_texts_mean_match_correct_total,
del PB_extracted_text_match_indices,
del PB_extracted_text_match_scores,
del PB_extracted_text_match_correct_total,
del PB_extracted_text_mean_match_indices,
del PB_extracted_text_mean_match_scores,
del PB_extracted_text_mean_match_correct_total,
del PB_texts_k1_match_indices,
del PB_texts_k1_match_scores,
del PB_texts_k1_match_correct_total,
del texts_K1_mean_match_indices,
del texts_k1_mean_match_scores,
del texts_k1_mean_match_correct_total,
del PB_extracted_k1_text_match_indices,
del PB_extracted_k1_text_match_scores,
del PB_extracted_k1_text_match_correct_total,
del PB_extracted_k1_text_mean_match_indices,
del PB_extracted_k1_text_mean_match_scores,
del PB_extracted_text_k1_mean_match_correct_total,
del PB_texts_cls_NN,
del PB_texts_mean_NN,
del PB_extracted_text_cls_NN,
del PB_extracted_text_mean_NN,
del PB_texts_cls_NN_distance,
del PB_texts_cls_NN_indes,
del PB_,
del PB_texts_mean_NN_indes,
del PB_extracted_text_cls_NN_distance,
del PB_extracted_text_cls_NN_indes,
del PB_extracted_text_mean_NN_distance,
del PB_extracted_text_mean_NN_indes,
del PBSMIS_texts_cls_NN,
del PBSMIS_texts_mean_NN,
del PBSMIS_extracted_text_cls_NN,
del PBSMIS_extracted_text_mean_NN,
del PBSMIS_texts_cls_NN_distance,
del PBSMIS_texts_cls_NN_indes,
del PBSMIS_texts_mean_NN_distance,
del PBSMIS_texts_mean_NN_indes,
del PBSMIS_extracted_text_cls_NN_distance,
del PBSMIS_extracted_text_cls_NN_indes,
del PBSMIS_extracted_text_mean_NN_distance,
del PBSMIS_extracted_text_mean_NN_indes,
del PB_cls_KM,
del PB_mean_KM,
del PB_extracted_text_cls_KM,
del PB_extracted_text_mean_KM,
del PB_cls_KM_index,
del PB_mean_KM_index,
del PB_extracted_text_cls_KM_index,
del PB_extracted_text_mean_KM_index,
del PB_cls_match_index,
del PB_mean_match_index,
del PB_et_cls_match_index,
del PB_et_mean_match_index,
del PB_x_cls,
del PB_x_mean,
del PB_x_et_cls,
del PB_x_et_mean,
del PBSMIS_cls_KM,
del PBSMIS_mean_KM,
del PBSMIS_extracted_text_cls_KM,
del PBSMIS_extracted_text_mean_KM,
del PBSMIS_cls_KM_index,
del PBSMIS_mean_KM_index,
del PBSMIS_extracted_text_cls_KM_index,
del PBSMIS_extracted_text_mean_KM_index,
del PBSMIS_cls_match_index,
del PBSMIS_mean_match_index,
del PBSMIS_et_cls_match_index,
del PBSMIS_et_mean_match_index,
del PBSMIS_x_cls,
del PBSMIS_x_mean,
del PBSMIS_x_et_cls,
del PBSMIS_x_et_mean,
del SentenceTransformer,
del SB_embeddings,
del SB_extracted_embeddings,
del K,
del SB_texts_match_indices,
del SB_texts_match_scores,
del SB_texts_match_correct_total,
del SB_extracted_match_indices,
del SB_extracted_match_scores,
del SB_extracted_match_correct_total,
del SB_texts_k1_match_indices,
del SB_texts_k1_match_scores,
del SB_texts_k1_match_correct_total,
del SB_extracted_k1_match_indices,
del SB_extracted_k1_match_scores,
del SB_extracted_k1_match_correct_total,
del SB_texts_mean_NN,
del SB_extracted_text_mean_NN,
del SB_emb_distance,
del SB_emb_index,
del SB_et_distance,
del SB_et_index,
del SBSMIS_texts_mean_NN,
del SBSMIS_extracted_text_mean_NN,
del SBSMIS_emb_distance,
del SBSMIS_emb_index,
del SBSMIS_et_distance,
del SBSMIS_et_index,
del SB_cls_KM,
del SB_extracted_text_KM,
del SB_cls,
del SB_extracted_text_cls,
del SB_cls_index,
del SB_x_cls,
del SB_et,
del SBSMIS_cls_KM,
del SBSMIS_extracted_text_KM,
del SBSMIS_cls,
del SBSMIS_extracted_text_cls,
del SBSMIS_cls_index,
del SBSMIS_x_cls,
del SBSMIS_et,
del OpenAIGPTModel,
del get_GPT_embedding,
del GPT_cls_embedding1,
del GPT_mean_embedding1,
del GPT_cls_embedding2,
del GPT_mean_embedding2,
del GPT_cls_embedding3,
del GPT_mean_embedding3,
del GPT_cls_embedding4,
del GPT_mean_embedding4,
del GPT_cls_embedding,
del GPT_mean_embedding,
del GPT_extracted_text_cls_embedding1,
del GPT_extracted_text_mean_embedding1,
del GPT_extracted_text_cls_embedding2,
del GPT_extracted_text_mean_embedding2,
del GPT_extracted_text_cls_embedding3,
del GPT_extracted_text_mean_embedding3,
del GPT_extracted_text_cls_embedding4,
del GPT_extracted_text_mean_embedding4,
del GPT_extracted_text_cls_embedding,
del GPT_extracted_text_mean_embedding,
del GPT_texts_match_indices,
del GPT_texts_match_scores,
del GPT_texts_match_correct_total,
del GPT_texts_mean_match_indices,
del GPT_texts_mean_match_scores,
del GPT_texts_mean_match_correct_total,
del GPT_extracted_text_match_indices,
del GPT_extracted_text_match_scores,
del GPTextracted_text_match_correct_total,
del GPT_extracted_text_mean_match_indices,
del GPT_extracted_text_mean_match_scores,
del GPT_extracted_text_mean_match_correct_total,
del GPT_texts_k1_match_indices,
del GPT_texts_k1_match_scores,
del GPT_texts_k1_match_correct_total,
del GPT_texts_K1_mean_match_indices,
del GPT_texts_k1_mean_match_scores,
del GPT_texts_k1_mean_match_correct_total,
del GPT_extracted_k1_text_match_indices,
del GPT_extracted_k1_text_match_scores,
del GPT_extracted_k1_text_match_correct_total,
del GPT_extracted_k1_text_mean_match_indices,
del GPT_extracted_k1_text_mean_match_scores,
del GPT_extracted_text_k1_mean_match_correct_total,
del GPT_texts_cls_NN,
del GPT_texts_mean_NN,
del GPT_extracted_text_cls_NN,
del GPT_extracted_text_mean_NN,
del CPT_texts_cls_NN_distance,
del GPT_texts_cls_NN_indes,
del GPT_texts_mean_NN_distance,
del GPT_texts_mean_NN_indes,
del GPT_extracted_text_cls_NN_distance,
del GPT_extracted_text_cls_NN_indes,
del GPT_extracted_text_mean_NN_distance,
del GPT_extracted_text_mean_NN_indes,
del GPTSMIS_texts_cls_NN,
del GPTSMIS_texts_mean_NN,
del GPTSMIS_extracted_text_cls_NN,
del GPTSMIS_extracted_text_mean_NN,
del CPTSMIS_texts_cls_NN_distance,
del GPTSMIS_texts_cls_NN_indes,
del GPTSMIS_texts_mean_NN_distance,
del GPTSMIS_texts_mean_NN_indes,
del GPTSMIS_extracted_text_cls_NN_distance,
del GPTSMIS_extracted_text_cls_NN_indes,
del GPTSMIS_extracted_text_mean_NN_distance,
del GPTSMIS_extracted_text_mean_NN_indes,
del GPT_cls_KM,
del GPT_mean_KM,
del GPT_extracted_text_cls_KM,
del GPT_extracted_text_mean_KM,
del GPT_cls_KM_index,
del GPT_mean_KM_index,
del GPT_extracted_text_cls_KM_index,
del GPT_extracted_text_mean_KM_index,
del GPT_cls_match_index,
del GPT_mean_match_index,
del GPT_et_cls_match_index,
del GPT_et_mean_match_index,
del GPT_x_cls,
del GPT_x_mean,
del GPT_x_et_cls,
del GPT_x_et_mean,
del GPTSMIS_cls_KM,
del GPTSMIS_mean_KM,
del GPTSMIS_extracted_text_cls_KM,
del GPTSMIS_extracted_text_mean_KM,
del GPTSMIS_cls_KM_index,
del GPTSMIS_mean_KM_index,
del GPTSMIS_extracted_text_cls_KM_index,
del GPTSMIS_extracted_text_mean_KM_index,
del GPTSMIS_cls_match_index,
del GPTSMIS_mean_match_index,
del GPTSMIS_et_cls_match_index,
del GPTSMIS_et_mean_match_index,
del GPTSMIS_x_cls,
del GPTSMIS_x_mean,
del GPTSMIS_x_et_cls,
del GPTSMIS_x_et_mean,
del Last_GPTembedding,
del LastGPT_embeddings1,
del LastGPT_embeddings2,
del LastGPT_embeddings3,
del LastGPT_embeddings4,
del LastGPT_embeddings,
del LastGPT_et_embeddings1,
del LastGPT_et_embeddings2,
del LastGPT_et_embeddings3,
del LastGPT_et_embeddings4,
del LastGPT_et_embeddings,
del LastGPT_texts_match_indices,
del LastGPT_texts_match_scores,
del LastGPT_texts_match_correct_total,
del LastGPT_extracted_text_match_indices,
del LastGPT_extracted_text_match_scores,
del LastGPT_extracted_text_match_correct_total,
del LastGPT_texts_k1_match_indices,
del LastGPT_extracted_k1_text_match_indices,
del LastGPT_texts_cls_NN,
del LastGPT_extracted_text_cls_NN,
del LastGPT_texts_cls_NN_distance,
del LastGPT_texts_cls_NN_indes,
del LastGPT_extracted_text_cls_NN_distance,
del LastGPT_extracted_text_cls_NN_indes,
del LastGPT_cls_KM,
del LastGPT_extracted_text_cls_KM,
del LastGPT_cls_KM_index,
del LastGPT_extracted_text_cls_KM_index,
del LastGPT_cls_match_index,
del LastGPT_et_cls_match_index,
del LastGPT_x_cls,
del LastGPT_x_et_cls,
del LastGPTSMIS_texts_cls_NN,
del LastGPTSMIS_extracted_text_cls_NN,
del LastGPTSMIS_texts_cls_NN_distance,
del LastGPTSMIS_texts_cls_NN_indes,
del LastGPTSMIS_extracted_text_cls_NN_distance,
del LastGPTSMIS_extracted_text_cls_NN_indes,
del LastGPTSMIS_cls_KM,
del LastGPTSMIS_extracted_text_cls_KM,
del LastGPTSMIS_cls_KM_index,
del LastGPTSMIS_extracted_text_cls_KM_index,
del LastGPTSMIS_cls_match_index,
del LastGPTSMIS_et_cls_match_index,
del LastGPTSMIS_x_cls,
del LastGPTSMIS_x_et_cls,

In [ ]:
import gc

# 
gc.collect()


In [ ]:
torch.cuda.empty_cache()

## Summarise last word GPT

In [ ]:
import torch
from transformers import AutoTokenizer, OpenAIGPTModel

def Summarise_GPTembedding(texts):

    # Check if GPU is available and set the device
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

    # Load pre-trained model and tokenizer and move the model to the GPU
    tokenizer = AutoTokenizer.from_pretrained("openai-community/openai-gpt")
    model = OpenAIGPTModel.from_pretrained("openai-community/openai-gpt").to(device)
    
    if tokenizer.pad_token is None:
        tokenizer.add_special_tokens({'pad_token': '[PAD]'})
        model.resize_token_embeddings(len(tokenizer))
    # Input sentences
    # texts = ['Your sentence here1', 'Your sentence here2']

    # Tokenize input sentence and move input tensors to the GPU

    template = 'This_sentence_:_"The_molecule_is_a_nitrile_that_is_acetonitrile_where_one_of_the_methyl_hydrogens_is_substituted_by_a_2-methylphenyl_group."_summarise_in_one_word:"Acetonitrile".This_sentence_:_"*sent_0*"_summarise_in_one_word:"'
    inputs = tokenizer([template.replace('*sent_0*', i).replace('_', ' ') for i in texts], padding=True, truncation=True,  max_length=512,return_tensors="pt")['input_ids'].to(device)
    with torch.no_grad():
        embeddings = model(inputs, output_hidden_states=True, return_dict=True).hidden_states[-1][:, -1, :]
    print(embeddings.shape)
    return embeddings



In [ ]:
# LastSGPT_embeddings=Summarise_GPTembedding(texts)
# LastSGPT_et_embeddings=Summarise_GPTembedding(extracted_text)
LastSGPT_embeddings1=Summarise_GPTembedding(texts[:1000])
LastSGPT_embeddings2=Summarise_GPTembedding(texts[1000:2000])
LastSGPT_embeddings3=Summarise_GPTembedding(texts[2000:3000])
LastSGPT_embeddings4=Summarise_GPTembedding(texts[3000:])

LastSGPT_embeddings=torch.cat((LastSGPT_embeddings1,LastSGPT_embeddings2,LastSGPT_embeddings3,LastSGPT_embeddings4))

LastSGPT_et_embeddings1=Summarise_GPTembedding(extracted_text[:1000])
LastSGPT_et_embeddings2=Summarise_GPTembedding(extracted_text[1000:2000])
LastSGPT_et_embeddings3=Summarise_GPTembedding(extracted_text[2000:3000])
LastSGPT_et_embeddings4=Summarise_GPTembedding(extracted_text[3000:])

LastSGPT_et_embeddings=torch.cat((LastSGPT_et_embeddings1,LastSGPT_et_embeddings2,LastSGPT_et_embeddings3,LastSGPT_et_embeddings4))


LastSGPT_embeddings.shape,LastSGPT_et_embeddings.shape

In [ ]:
de_biased_cka(LastSGPT_embeddings,sim_embeddings.pooler_output),de_biased_cka(LastSGPT_et_embeddings,sim_embeddings.pooler_output)

In [ ]:
CKA(LastSGPT_embeddings,sim_embeddings.pooler_output.to(device)),CKA(LastSGPT_et_embeddings,sim_embeddings.pooler_output.to(device))

In [ ]:
K=10

LastSGPT_texts_match_indices, LastSGPT_texts_match_scores, LastSGPT_texts_match_correct_total = retrieve(sim_embeddings.pooler_output.cpu().detach(), LastSGPT_embeddings.cpu().detach(), K=K)

# 打印结果
# print("Top K 索引：", indices)
# print("Top K 分数：", scores)
print("正确匹配的条目数：", LastSGPT_texts_match_correct_total)
LastSGPT_extracted_text_match_indices, LastSGPT_extracted_text_match_scores, LastSGPT_extracted_text_match_correct_total = retrieve(sim_embeddings.pooler_output.cpu().detach(), LastSGPT_et_embeddings.cpu().detach(), K=K)

print("正确匹配的条目数：", LastSGPT_extracted_text_match_correct_total)

In [ ]:
K=1

LastSGPT_texts_k1_match_indices, LastSGPT_texts_match_scores, LastSGPT_texts_match_correct_total = retrieve(sim_embeddings.pooler_output.cpu().detach(), LastSGPT_embeddings.cpu().detach(), K=K)

# 打印结果
# print("Top K 索引：", indices)
# print("Top K 分数：", scores)
print("正确匹配的条目数：", LastSGPT_texts_match_correct_total)
LastSGPT_extracted_k1_text_match_indices, LastSGPT_extracted_text_match_scores, LastSGPT_extracted_text_match_correct_total = retrieve(sim_embeddings.pooler_output.cpu().detach(), LastSGPT_et_embeddings.cpu().detach(), K=K)

print("正确匹配的条目数：", LastSGPT_extracted_text_match_correct_total)

### text

In [ ]:
LastSGPT_texts_cls_NN=NN(embedding=LastSGPT_embeddings)
LastSGPT_extracted_text_cls_NN=NN(embedding=LastSGPT_et_embeddings)

In [ ]:
LastSGPT_texts_cls_NN_distance,LastSGPT_texts_cls_NN_indes=LastSGPT_texts_cls_NN.kneighbors(sim_embeddings.pooler_output.cpu().detach())

LastSGPT_extracted_text_cls_NN_distance,LastSGPT_extracted_text_cls_NN_indes=LastSGPT_extracted_text_cls_NN.kneighbors(sim_embeddings.pooler_output.cpu().detach())

In [ ]:
overleap(LastSGPT_texts_match_indices.cpu().numpy(), LastSGPT_texts_cls_NN_indes)

#_____________cross should get low score_____________

overleap(LastSGPT_extracted_text_match_indices.cpu().numpy(), LastSGPT_extracted_text_cls_NN_indes)

In [ ]:
LastSGPT_cls_KM=KM(LastSGPT_embeddings.cpu().numpy())

LastSGPT_extracted_text_cls_KM=KM(LastSGPT_et_embeddings.cpu().numpy())

In [ ]:
LastSGPT_cls_KM_index=LastSGPT_cls_KM.predict(LastSGPT_embeddings.cpu().numpy())

LastSGPT_extracted_text_cls_KM_index=LastSGPT_extracted_text_cls_KM.predict(LastSGPT_et_embeddings.cpu().numpy())

In [ ]:
LastSGPT_cls_match_index=LastSGPT_cls_KM.predict(LastSGPT_embeddings[LastSGPT_texts_k1_match_indices].reshape(LastSGPT_embeddings.shape).cpu().numpy())
LastSGPT_et_cls_match_index=LastSGPT_extracted_text_cls_KM.predict(LastSGPT_et_embeddings[LastSGPT_extracted_k1_text_match_indices].reshape(LastSGPT_et_embeddings.shape).cpu().numpy())

In [ ]:
LastSGPT_x_cls=LastSGPT_cls_match_index==LastSGPT_cls_KM_index
LastSGPT_x_et_cls=LastSGPT_extracted_text_cls_KM_index==LastSGPT_et_cls_match_index

np.sum(LastSGPT_x_cls),np.sum(LastSGPT_x_et_cls)

### SMIELS

In [ ]:
LastSGPTSMIS_texts_cls_NN=NN(embedding=sim_embeddings.pooler_output)
LastSGPTSMIS_extracted_text_cls_NN=NN(embedding=sim_embeddings.pooler_output)

In [ ]:
LastSGPTSMIS_texts_cls_NN_distance,LastSGPTSMIS_texts_cls_NN_indes=LastSGPTSMIS_texts_cls_NN.kneighbors(LastSGPT_embeddings.cpu().numpy())

LastSGPTSMIS_extracted_text_cls_NN_distance,LastSGPTSMIS_extracted_text_cls_NN_indes=LastSGPTSMIS_extracted_text_cls_NN.kneighbors(LastSGPT_et_embeddings.cpu().numpy())

In [ ]:
overleap(LastSGPT_texts_match_indices.cpu().numpy(), LastSGPTSMIS_texts_cls_NN_indes)

#_____________cross should get low score_____________

overleap(LastSGPT_extracted_text_match_indices.cpu().numpy(), LastSGPTSMIS_extracted_text_cls_NN_indes)

In [ ]:
LastSGPTSMIS_cls_KM=KM(sim_embeddings.pooler_output.cpu().numpy().astype(np.float32))

LastSGPTSMIS_extracted_text_cls_KM=KM(sim_embeddings.pooler_output.cpu().numpy().astype(np.float32))

In [ ]:
LastSGPTSMIS_cls_KM_index=LastSGPTSMIS_cls_KM.predict(sim_embeddings.pooler_output.cpu().numpy().astype(np.float32))

LastSGPTSMIS_extracted_text_cls_KM_index=LastSGPTSMIS_extracted_text_cls_KM.predict(sim_embeddings.pooler_output.cpu().numpy().astype(np.float32))

In [ ]:
LastSGPTSMIS_cls_match_index=LastSGPTSMIS_cls_KM.predict(sim_embeddings.pooler_output[LastSGPT_texts_k1_match_indices].reshape(LastSGPT_embeddings.shape).cpu().numpy().astype(np.float32))
LastSGPTSMIS_et_cls_match_index=LastSGPTSMIS_extracted_text_cls_KM.predict(sim_embeddings.pooler_output[LastSGPT_extracted_k1_text_match_indices].reshape(LastSGPT_et_embeddings.shape).cpu().numpy().astype(np.float32))

In [ ]:
LastSGPTSMIS_x_cls=LastSGPTSMIS_cls_match_index==LastSGPTSMIS_cls_KM_index
LastSGPTSMIS_x_et_cls=LastSGPTSMIS_extracted_text_cls_KM_index==LastSGPTSMIS_et_cls_match_index

np.sum(LastSGPTSMIS_x_cls),np.sum(LastSGPTSMIS_x_et_cls)

In [ ]:
globals().keys()

In [ ]:
# del  AutoTokenizer,
# del  AutoModel,
# del  SCIBERT,
# del  SCIB_cls_embedding,
# del  SCIB_mean_embedding,
# del  SCIB_extracted_text_cls_embedding,
# del  SCIB_extracted_text_mean_embedding,
# del  SCIB_texts_cls_NN,
# del  SCIB_texts_mean_NN,
# del  SCIB_extracted_text_cls_NN,
# del  SCIB_extracted_text_mean_NN,
# del  SCIB_texts_cls_NN_distance,
# del  SCIB_texts_cls_NN_indes,
# del  SCIB_texts_mean_NN_distance,
# del  SCIB_texts_mean_NN_indes,
# del  SCIB_extracted_text_cls_NN_distance,
# del  SCIB_extracted_text_cls_NN_indes,
# del  SCIB_extracted_text_mean_NN_distance,
# del  SCIB_extracted_text_mean_NN_indes,
# del  SCIB_texts_match_indices,
# del  SCIB_texts_match_scores,
# del  SCIB_texts_match_correct_total,
# del  SCIB_texts_mean_match_indices,
# del  SCIB_texts_mean_match_scores,
# del  SCIB_texts_mean_match_correct_total,
# del  SCIB_extracted_text_match_indices,
# del  SCIB_extracted_text_match_scores,
# del  SCIB_extracted_text_match_correct_total,
# del  SCIB_extracted_text_mean_match_indices,
# del  SCIB_extracted_text_mean_match_scores,
# del  SCIB_extracted_text_mean_match_correct_total,
# del  SCIB_texts_k1_match_indices,
# del  SCIB_texts_k1_match_scores,
# del  SCIB_texts_k1_match_correct_total,
# del  SCIB_texts_K1_mean_match_indices,
# del  SCIB_texts_k1_mean_match_scores,
# del  SCIB_texts_k1_mean_match_correct_total,
# del  SCIB_extracted_k1_text_match_indices,
# del  SCIB_extracted_k1_text_match_scores,
# del  SCIB_extracted_k1_text_match_correct_total,
# del  SCIB_extracted_k1_text_mean_match_indices,
# del  SCIB_extracted_k1_text_mean_match_scores,
# del  SCIB_extracted_text_k1_mean_match_correct_total,
# del  SCIBSMIS_texts_cls_NN,
# del  SCIBSMIS_texts_mean_NN,
# del  SCIBSMIS_extracted_text_cls_NN,
# del  SCIBSMIS_extracted_text_mean_NN,
# del  SCIBSMIS_texts_cls_NN_distance,
# del  SCIBSMIS_texts_cls_NN_indes,
# del  SCIBSMIS_texts_mean_NN_distance,
# del  SCIBSMIS_texts_mean_NN_indes,
# del  SCIBSMIS_extracted_text_cls_NN_distance,
# del  SCIBSMIS_extracted_text_cls_NN_indes,
# del  SCIBSMIS_extracted_text_mean_NN_distance,
# del  SCIBSMIS_extracted_text_mean_NN_indes,
# del  SCIB_cls_KM,
# del  SCIB_mean_KM,
# del  SCIB_extracted_text_cls_KM,
# del  SCIB_extracted_text_mean_KM,
# del  SCIB_cls_KM_index,
# del  SCIB_mean_KM_index,
# del  SCIB_extracted_text_cls_KM_index,
# del  SCIB_extracted_text_mean_KM_index,
# del  SCIB_cls_match_index,
# del  SCIB_mean_match_index,
# del  SCIB_et_cls_match_index,
# del  SCIB_et_mean_match_index,
# del  SCIB_x_cls,
# del  SCIB_x_mean,
# del  SCIB_x_et_cls,
# del  SCIB_x_et_mean,
# del  SCIBSMIS_cls_KM,
# del  SCIBSMIS_mean_KM,
# del  SCIBSMIS_extracted_text_cls_KM,
# del  SCIBSMIS_extracted_text_mean_KM,
# del  SCIBSMIS_cls_KM_index,
# del  SCIBSMIS_mean_KM_index,
# del  SCIBSMIS_extracted_text_cls_KM_index,
# del  SCIBSMIS_extracted_text_mean_KM_index,
# del  SCIBSMIS_cls_match_index,
# del  SCIBSMIS_mean_match_index,
# del  SCIBSMIS_et_cls_match_index,
# del  SCIBSMIS_et_mean_match_index,
# del  SCIBSMIS_x_cls,
# del  SCIBSMIS_x_mean,
# del  SCIBSMIS_x_et_cls,
# del  SCIBSMIS_x_et_mean,
# del  BertTokenizer,
# del  BertModel,
# del  pureBert,
# del  PB_cls_embedding,
# del  PB_mean_embedding,
# del  PB_extracted_text_cls_embedding,
# del  PB_extracted_text_mean_embedding,
# del  PB_texts_match_indices,
# del  PB_texts_match_scores,
# del  PB_texts_match_correct_total,
# del  PB_texts_mean_match_indices,
# del  PB_texts_mean_match_scores,
# del  PB_texts_mean_match_correct_total,
# del  PB_extracted_text_match_indices,
# del  PB_extracted_text_match_scores,
# del  PB_extracted_text_match_correct_total,
# del  PB_extracted_text_mean_match_indices,
# del  PB_extracted_text_mean_match_scores,
# del  PB_extracted_text_mean_match_correct_total,
# del  PB_texts_k1_match_indices,
# del  PB_texts_k1_match_scores,
# del  PB_texts_k1_match_correct_total,
# del  texts_K1_mean_match_indices,
# del  texts_k1_mean_match_scores,
# del  texts_k1_mean_match_correct_total,
# del  PB_extracted_k1_text_match_indices,
# del  PB_extracted_k1_text_match_scores,
# del  PB_extracted_k1_text_match_correct_total,
# del  PB_extracted_k1_text_mean_match_indices,
# del  PB_extracted_k1_text_mean_match_scores,
# del  PB_extracted_text_k1_mean_match_correct_total,
# del  PB_texts_cls_NN,
# del  PB_texts_mean_NN,
# del  PB_extracted_text_cls_NN,
# del  PB_extracted_text_mean_NN,
# del  PB_texts_cls_NN_distance,
# del  PB_texts_cls_NN_indes,
# del  PB_,
# del  PB_texts_mean_NN_indes,
# del  PB_extracted_text_cls_NN_distance,
# del  PB_extracted_text_cls_NN_indes,
# del  PB_extracted_text_mean_NN_distance,
# del  PB_extracted_text_mean_NN_indes,
# del  PBSMIS_texts_cls_NN,
# del  PBSMIS_texts_mean_NN,
# del  PBSMIS_extracted_text_cls_NN,
# del  PBSMIS_extracted_text_mean_NN,
# del  PBSMIS_texts_cls_NN_distance,
# del  PBSMIS_texts_cls_NN_indes,
# del  PBSMIS_texts_mean_NN_distance,
# del  PBSMIS_texts_mean_NN_indes,
# del  PBSMIS_extracted_text_cls_NN_distance,
# del  PBSMIS_extracted_text_cls_NN_indes,
# del  PBSMIS_extracted_text_mean_NN_distance,
# del  PBSMIS_extracted_text_mean_NN_indes,
# del  PB_cls_KM,
# del  PB_mean_KM,
# del  PB_extracted_text_cls_KM,
# del  PB_extracted_text_mean_KM,
# del  PB_cls_KM_index,
# del  PB_mean_KM_index,
# del  PB_extracted_text_cls_KM_index,
# del  PB_extracted_text_mean_KM_index,
# del  PB_cls_match_index,
# del  PB_mean_match_index,
# del  PB_et_cls_match_index,
# del  PB_et_mean_match_index,
# del  PB_x_cls,
# del  PB_x_mean,
# del  PB_x_et_cls,
# del  PB_x_et_mean,
# del  PBSMIS_cls_KM,
# del  PBSMIS_mean_KM,
# del  PBSMIS_extracted_text_cls_KM,
# del  PBSMIS_extracted_text_mean_KM,
# del  PBSMIS_cls_KM_index,
# del  PBSMIS_mean_KM_index,
# del  PBSMIS_extracted_text_cls_KM_index,
# del  PBSMIS_extracted_text_mean_KM_index,
# del  PBSMIS_cls_match_index,
# del  PBSMIS_mean_match_index,
# del  PBSMIS_et_cls_match_index,
# del  PBSMIS_et_mean_match_index,
# del  PBSMIS_x_cls,
# del  PBSMIS_x_mean,
# del  PBSMIS_x_et_cls,
# del  PBSMIS_x_et_mean,
# del  SentenceTransformer,
# del  SB_embeddings,
# del  SB_extracted_embeddings,
# del  K,
# del  SB_texts_match_indices,
# del  SB_texts_match_scores,
# del  SB_texts_match_correct_total,
# del  SB_extracted_match_indices,
# del  SB_extracted_match_scores,
# del  SB_extracted_match_correct_total,
# del  SB_texts_k1_match_indices,
# del  SB_texts_k1_match_scores,
# del  SB_texts_k1_match_correct_total,
# del  SB_extracted_k1_match_indices,
# del  SB_extracted_k1_match_scores,
# del  SB_extracted_k1_match_correct_total,
# del  SB_texts_mean_NN,
# del  SB_extracted_text_mean_NN,
# del  SB_emb_distance,
# del  SB_emb_index,
# del  SB_et_distance,
# del  SB_et_index,
# del  SBSMIS_texts_mean_NN,
# del  SBSMIS_extracted_text_mean_NN,
# del  SBSMIS_emb_distance,
# del  SBSMIS_emb_index,
# del  SBSMIS_et_distance,
# del  SBSMIS_et_index,
# del  SB_cls_KM,
# del  SB_extracted_text_KM,
# del  SB_cls,
# del  SB_extracted_text_cls,
# del  SB_cls_index,
# del  SB_x_cls,
# del  SB_et,
# del  SBSMIS_cls_KM,
# del  SBSMIS_extracted_text_KM,
# del  SBSMIS_cls,
# del  SBSMIS_extracted_text_cls,
# del  SBSMIS_cls_index,
# del  SBSMIS_x_cls,
# del  SBSMIS_et,
# del  OpenAIGPTModel,
# del  get_GPT_embedding,
# del  GPT_cls_embedding1,
# del  GPT_mean_embedding1,
# del  GPT_cls_embedding2,
# del  GPT_mean_embedding2,
# del  GPT_cls_embedding3,
# del  GPT_mean_embedding3,
# del  GPT_cls_embedding4,
# del  GPT_mean_embedding4,
# del  GPT_cls_embedding,
# del  GPT_mean_embedding,
# del  GPT_extracted_text_cls_embedding1,
# del  GPT_extracted_text_mean_embedding1,
# del  GPT_extracted_text_cls_embedding2,
# del  GPT_extracted_text_mean_embedding2,
# del  GPT_extracted_text_cls_embedding3,
# del  GPT_extracted_text_mean_embedding3,
# del  GPT_extracted_text_cls_embedding4,
# del  GPT_extracted_text_mean_embedding4,
# del  GPT_extracted_text_cls_embedding,
# del  GPT_extracted_text_mean_embedding,
# del  GPT_texts_match_indices,
# del  GPT_texts_match_scores,
# del  GPT_texts_match_correct_total,
# del  GPT_texts_mean_match_indices,
# del  GPT_texts_mean_match_scores,
# del  GPT_texts_mean_match_correct_total,
# del  GPT_extracted_text_match_indices,
# del  GPT_extracted_text_match_scores,
# del  GPTextracted_text_match_correct_total,
# del  GPT_extracted_text_mean_match_indices,
# del  GPT_extracted_text_mean_match_scores,
# del  GPT_extracted_text_mean_match_correct_total,
# del  GPT_texts_k1_match_indices,
# del  GPT_texts_k1_match_scores,
# del  GPT_texts_k1_match_correct_total,
# del  GPT_texts_K1_mean_match_indices,
# del  GPT_texts_k1_mean_match_scores,
# del  GPT_texts_k1_mean_match_correct_total,
# del  GPT_extracted_k1_text_match_indices,
# del  GPT_extracted_k1_text_match_scores,
# del  GPT_extracted_k1_text_match_correct_total,
# del  GPT_extracted_k1_text_mean_match_indices,
# del  GPT_extracted_k1_text_mean_match_scores,
# del  GPT_extracted_text_k1_mean_match_correct_total,
# del  GPT_texts_cls_NN,
# del  GPT_texts_mean_NN,
# del  GPT_extracted_text_cls_NN,
# del  GPT_extracted_text_mean_NN,
# del  CPT_texts_cls_NN_distance,
# del  GPT_texts_cls_NN_indes,
# del  GPT_texts_mean_NN_distance,
# del  GPT_texts_mean_NN_indes,
# del  GPT_extracted_text_cls_NN_distance,
# del  GPT_extracted_text_cls_NN_indes,
# del  GPT_extracted_text_mean_NN_distance,
# del  GPT_extracted_text_mean_NN_indes,
# del  GPTSMIS_texts_cls_NN,
# del  GPTSMIS_texts_mean_NN,
# del  GPTSMIS_extracted_text_cls_NN,
# del  GPTSMIS_extracted_text_mean_NN,
# del  CPTSMIS_texts_cls_NN_distance,
# del  GPTSMIS_texts_cls_NN_indes,
# del  GPTSMIS_texts_mean_NN_distance,
# del  GPTSMIS_texts_mean_NN_indes,
# del  GPTSMIS_extracted_text_cls_NN_distance,
# del  GPTSMIS_extracted_text_cls_NN_indes,
# del  GPTSMIS_extracted_text_mean_NN_distance,
# del  GPTSMIS_extracted_text_mean_NN_indes,
# del  GPT_cls_KM,
# del  GPT_mean_KM,
# del  GPT_extracted_text_cls_KM,
# del  GPT_extracted_text_mean_KM,
# del  GPT_cls_KM_index,
# del  GPT_mean_KM_index,
# del  GPT_extracted_text_cls_KM_index,
# del  GPT_extracted_text_mean_KM_index,
# del  GPT_cls_match_index,
# del  GPT_mean_match_index,
# del  GPT_et_cls_match_index,
# del  GPT_et_mean_match_index,
# del  GPT_x_cls,
# del  GPT_x_mean,
# del  GPT_x_et_cls,
# del  GPT_x_et_mean,
# del  GPTSMIS_cls_KM,
# del  GPTSMIS_mean_KM,
# del  GPTSMIS_extracted_text_cls_KM,
# del  GPTSMIS_extracted_text_mean_KM,
# del  GPTSMIS_cls_KM_index,
# del  GPTSMIS_mean_KM_index,
# del  GPTSMIS_extracted_text_cls_KM_index,
# del  GPTSMIS_extracted_text_mean_KM_index,
# del  GPTSMIS_cls_match_index,
# del  GPTSMIS_mean_match_index,
# del  GPTSMIS_et_cls_match_index,
# del  GPTSMIS_et_mean_match_index,
# del  GPTSMIS_x_cls,
# del  GPTSMIS_x_mean,
# del  GPTSMIS_x_et_cls,
# del  GPTSMIS_x_et_mean,
# del  Last_GPTembedding,
# del  LastGPT_embeddings1,
# del  LastGPT_embeddings2,
# del  LastGPT_embeddings3,
# del  LastGPT_embeddings4,
# del  LastGPT_embeddings,
# del  LastGPT_et_embeddings1,
# del  LastGPT_et_embeddings2,
# del  LastGPT_et_embeddings3,
# del  LastGPT_et_embeddings4,
# del  LastGPT_et_embeddings,
# del  LastGPT_texts_match_indices,
# del  LastGPT_texts_match_scores,
# del  LastGPT_texts_match_correct_total,
# del  LastGPT_extracted_text_match_indices,
# del  LastGPT_extracted_text_match_scores,
# del  LastGPT_extracted_text_match_correct_total,
# del  LastGPT_texts_k1_match_indices,
# del  LastGPT_extracted_k1_text_match_indices,
# del  LastGPT_texts_cls_NN,
# del  LastGPT_extracted_text_cls_NN,
# del  LastGPT_texts_cls_NN_distance,
# del  LastGPT_texts_cls_NN_indes,
# del  LastGPT_extracted_text_cls_NN_distance,
# del  LastGPT_extracted_text_cls_NN_indes,
# del  LastGPT_cls_KM,
# del  LastGPT_extracted_text_cls_KM,
# del  LastGPT_cls_KM_index,
# del  LastGPT_extracted_text_cls_KM_index,
# del  LastGPT_cls_match_index,
# del  LastGPT_et_cls_match_index,
# del  LastGPT_x_cls,
# del  LastGPT_x_et_cls,
# del  LastGPTSMIS_texts_cls_NN,
# del  LastGPTSMIS_extracted_text_cls_NN,
# del  LastGPTSMIS_texts_cls_NN_distance,
# del  LastGPTSMIS_texts_cls_NN_indes,
# del  LastGPTSMIS_extracted_text_cls_NN_distance,
# del  LastGPTSMIS_extracted_text_cls_NN_indes,
# del  LastGPTSMIS_cls_KM,
# del  LastGPTSMIS_extracted_text_cls_KM,
# del  LastGPTSMIS_cls_KM_index,
# del  LastGPTSMIS_extracted_text_cls_KM_index,
# del  LastGPTSMIS_cls_match_index,
# del  LastGPTSMIS_et_cls_match_index,
# del  LastGPTSMIS_x_cls,
# del  LastGPTSMIS_x_et_cls,
# del  Summarise_GPTembedding,
# del  LastSGPT_embeddings1,
# del  LastSGPT_embeddings2,
# del  LastSGPT_embeddings3,
# del  LastSGPT_embeddings4,
# del  LastSGPT_embeddings,
# del  LastSGPT_et_embeddings1,
# del  LastSGPT_et_embeddings2,
# del  LastSGPT_et_embeddings3,
# del  LastSGPT_et_embeddings4,
# del  LastSGPT_et_embeddings,
# del  LastSGPT_texts_match_indices,
# del  LastSGPT_texts_match_scores,
# del  LastSGPT_texts_match_correct_total,
# del  LastSGPT_extracted_text_match_indices,
# del  LastSGPT_extracted_text_match_scores,
# del  LastSGPT_extracted_text_match_correct_total,
# del  LastSGPT_texts_k1_match_indices,
# del  LastSGPT_extracted_k1_text_match_indices,
# del  LastSGPT_texts_cls_NN,
# del  LastSGPT_extracted_text_cls_NN,
# del  LastSGPT_texts_cls_NN_distance,
# del  LastSGPT_texts_cls_NN_indes,
# del  LastSGPT_extracted_text_cls_NN_distance,
# del  LastSGPT_extracted_text_cls_NN_indes,
# del  LastSGPT_cls_KM,
# del  LastSGPT_extracted_text_cls_KM,
# del  LastSGPT_cls_KM_index,
# del  LastSGPT_extracted_text_cls_KM_index,
# del  LastSGPT_cls_match_index,
# del  LastSGPT_et_cls_match_index,
# del  LastSGPT_x_cls,
# del  LastSGPT_x_et_cls,
# del  LastSGPTSMIS_texts_cls_NN,
# del  LastSGPTSMIS_extracted_text_cls_NN,
# del  LastSGPTSMIS_texts_cls_NN_distance,
# del  LastSGPTSMIS_texts_cls_NN_indes,
# del  LastSGPTSMIS_extracted_text_cls_NN_distance,
# del  LastSGPTSMIS_extracted_text_cls_NN_indes,
# del  LastSGPTSMIS_cls_KM,
# del  LastSGPTSMIS_extracted_text_cls_KM,
# del  LastSGPTSMIS_cls_KM_index,
# del  LastSGPTSMIS_extracted_text_cls_KM_index,
# del  LastSGPTSMIS_cls_match_index,
# del  LastSGPTSMIS_et_cls_match_index,
# del  LastSGPTSMIS_x_cls,
# del  LastSGPTSMIS_x_et_cls,

In [ ]:
del AutoTokenizer,
del OpenAIGPTModel,
del Summarise_GPTembedding,
del LastSGPT_embeddings1,
del LastSGPT_embeddings2,
del LastSGPT_embeddings3,
del LastSGPT_embeddings4,
del LastSGPT_embeddings,
del LastSGPT_et_embeddings1,
del LastSGPT_et_embeddings2,
del LastSGPT_et_embeddings3,
del LastSGPT_et_embeddings4,
del LastSGPT_et_embeddings,
del K,
del LastSGPT_texts_match_indices,
del LastSGPT_texts_match_scores,
del LastSGPT_texts_match_correct_total,
del LastSGPT_extracted_text_match_indices,
del LastSGPT_extracted_text_match_scores,
del LastSGPT_extracted_text_match_correct_total,
del LastSGPT_texts_k1_match_indices,
del LastSGPT_extracted_k1_text_match_indices,
del LastSGPT_texts_cls_NN,
del LastSGPT_extracted_text_cls_NN,
del LastSGPT_texts_cls_NN_distance,
del LastSGPT_texts_cls_NN_indes,
del LastSGPT_extracted_text_cls_NN_distance,
del LastSGPT_extracted_text_cls_NN_indes,
del LastSGPT_cls_KM,
del LastSGPT_extracted_text_cls_KM,
del LastSGPT_cls_KM_index,
del LastSGPT_extracted_text_cls_KM_index,
del LastSGPT_cls_match_index,
del LastSGPT_et_cls_match_index,
del LastSGPT_x_cls,
del LastSGPT_x_et_cls,
del LastSGPTSMIS_texts_cls_NN,
del LastSGPTSMIS_extracted_text_cls_NN,
del LastSGPTSMIS_texts_cls_NN_distance,
del LastSGPTSMIS_texts_cls_NN_indes,
del LastSGPTSMIS_extracted_text_cls_NN_distance,
del LastSGPTSMIS_extracted_text_cls_NN_indes,
del LastSGPTSMIS_cls_KM,
del LastSGPTSMIS_extracted_text_cls_KM,
del LastSGPTSMIS_cls_KM_index,
del LastSGPTSMIS_extracted_text_cls_KM_index,
del LastSGPTSMIS_cls_match_index,
del LastSGPTSMIS_et_cls_match_index,
del LastSGPTSMIS_x_cls,
del LastSGPTSMIS_x_et_cls,

gc.collect()
torch.cuda.empty_cache()

## GPT 2

In [ ]:
import torch
from transformers import GPT2Model, GPT2Tokenizer

def GPT2embedding(texts):

    # Check if GPU is available and set the device
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    # Load pre-trained model and tokenizer and move the model to the GPU
    model = GPT2Model.from_pretrained('gpt2').to(device)
    tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
    
    if tokenizer.pad_token is None:
        tokenizer.add_special_tokens({'pad_token': '[PAD]'})
        model.resize_token_embeddings(len(tokenizer))
    # Input sentences
    # texts = ['Your sentence here1', 'Your sentence here2']

    # Tokenize input sentence and move input tensors to the GPU
    input_ids = tokenizer(texts, return_tensors='pt', padding=True, truncation=True)['input_ids'].to(device)

    # Get the hidden states (outputs from each layer)
    with torch.no_grad():
        outputs = model(input_ids)

    # # Optionally, move the result back to the CPU if needed
    # sentence_embedding = sentence_embedding.cpu().detach()

    # print(sentence_embedding.shape)
    # return sentence_embedding

    # Access last hidden states
    last_hidden_states = outputs.last_hidden_state
    print(last_hidden_states.shape)

    # 获取最后的隐藏状态 [batch_size, seq_len, hidden_size]
    last_hidden_states = outputs.last_hidden_state

    # 获取第一个 token (<s> token) 的 embedding（相当于 CLS token 的句子嵌入）
    cls_embedding = last_hidden_states[:, 0, :]

    # 也可以取所有 token 的平均值作为句子嵌入
    mean_embedding = torch.mean(last_hidden_states, dim=1)

    print(f"CLS token embedding size: {cls_embedding.shape}")
    print(f"Mean token embedding size: {mean_embedding.shape}")
    return cls_embedding,mean_embedding

In [ ]:
# GPT2_cls_embedding,GPT2_mean_embedding=GPT2embedding(texts)
# GPT2_extracted_text_cls_embedding,GPT2_extracted_text_mean_embedding=GPT2embedding(extracted_text)
GPT2_cls_embedding1,GPT2_mean_embedding1=GPT2embedding(texts[:1000])
GPT2_cls_embedding2,GPT2_mean_embedding2=GPT2embedding(texts[1000:2000])
GPT2_cls_embedding3,GPT2_mean_embedding3=GPT2embedding(texts[2000:3000])
GPT2_cls_embedding4,GPT2_mean_embedding4=GPT2embedding(texts[3000:])

GPT2_cls_embedding=torch.cat((GPT2_cls_embedding1,GPT2_cls_embedding2,GPT2_cls_embedding3,GPT2_cls_embedding4))
GPT2_mean_embedding=torch.cat((GPT2_mean_embedding1,GPT2_mean_embedding2,GPT2_mean_embedding3,GPT2_mean_embedding4))

GPT2_extracted_text_cls_embedding1,GPT2_extracted_text_mean_embedding1=GPT2embedding(extracted_text[:1000])
GPT2_extracted_text_cls_embedding2,GPT2_extracted_text_mean_embedding2=GPT2embedding(extracted_text[1000:2000])
GPT2_extracted_text_cls_embedding3,GPT2_extracted_text_mean_embedding3=GPT2embedding(extracted_text[2000:3000])
GPT2_extracted_text_cls_embedding4,GPT2_extracted_text_mean_embedding4=GPT2embedding(extracted_text[3000:])

GPT2_extracted_text_cls_embedding=torch.cat((GPT2_extracted_text_cls_embedding1,GPT2_extracted_text_cls_embedding2,GPT2_extracted_text_cls_embedding3,GPT2_extracted_text_cls_embedding4))
GPT2_extracted_text_mean_embedding=torch.cat((GPT2_extracted_text_mean_embedding1,GPT2_extracted_text_mean_embedding2,GPT2_extracted_text_mean_embedding3,GPT2_extracted_text_mean_embedding4))

### CKA

In [ ]:
de_biased_cka(GPT2_cls_embedding,sim_embeddings.pooler_output),de_biased_cka(GPT2_mean_embedding,sim_embeddings.pooler_output),de_biased_cka(GPT2_extracted_text_cls_embedding,sim_embeddings.pooler_output),de_biased_cka(GPT2_extracted_text_mean_embedding,sim_embeddings.pooler_output)

In [ ]:
CKA(GPT2_cls_embedding.cpu(),sim_embeddings.pooler_output),CKA(GPT2_mean_embedding.cpu(),sim_embeddings.pooler_output),CKA(GPT2_extracted_text_cls_embedding.cpu(),sim_embeddings.pooler_output),CKA(GPT2_extracted_text_mean_embedding.cpu(),sim_embeddings.pooler_output)

### Now, we have cls mean embedding of texts and extracted_text
cls_embedding,

mean_embedding,

extracted_text_cls_embedding,

extracted_text_mean_embedding,

#### exact match

In [ ]:
GPT2_texts_match_indices, GPT2_texts_match_scores, GPT2_texts_match_correct_total = retrieve(sim_embeddings.pooler_output, GPT2_cls_embedding.cpu().detach(), K=10)
GPT2_texts_mean_match_indices, GPT2_texts_mean_match_scores, GPT2_texts_mean_match_correct_total = retrieve(sim_embeddings.pooler_output, GPT2_mean_embedding.cpu().detach(), K=10)

# 打印结果
# print("Top K 索引：", indices)
# print("Top K 分数：", scores)
print("正确匹配的条目数：", GPT2_texts_match_correct_total)
print("正确匹配的条目数：", GPT2_texts_mean_match_correct_total)
GPT2_extracted_text_match_indices, GPT2_extracted_text_match_scores, GPT2_extracted_text_match_correct_total = retrieve(sim_embeddings.pooler_output, GPT2_extracted_text_cls_embedding.cpu().detach(), K=10)
GPT2_extracted_text_mean_match_indices, GPT2_extracted_text_mean_match_scores, GPT2_extracted_text_mean_match_correct_total = retrieve(sim_embeddings.pooler_output, GPT2_extracted_text_mean_embedding.cpu().detach(), K=10)

print("正确匹配的条目数：", GPT2_extracted_text_match_correct_total)
print("正确匹配的条目数：", GPT2_extracted_text_mean_match_correct_total)

In [ ]:
GPT2_texts_k1_match_indices, GPT2_texts_k1_match_scores, GPT2_texts_k1_match_correct_total = retrieve(sim_embeddings.pooler_output.cpu().detach(), GPT2_cls_embedding.cpu().detach(), K=1)
GPT2_texts_K1_mean_match_indices, GPT2_texts_k1_mean_match_scores, GPT2_texts_k1_mean_match_correct_total = retrieve(sim_embeddings.pooler_output.cpu().detach(), GPT2_mean_embedding.cpu().detach(), K=1)

# 打印结果
# print("Top K 索引：", indices)
# print("Top K 分数：", scores)
print("正确匹配的条目数：", GPT2_texts_k1_match_correct_total)
print("正确匹配的条目数：", GPT2_texts_k1_mean_match_correct_total)


extracted_k1_text_match_indices, extracted_k1_text_match_scores, extracted_k1_text_match_correct_total = retrieve(sim_embeddings.pooler_output.cpu().detach(), GPT2_extracted_text_cls_embedding.cpu().detach(), K=1)
extracted_k1_text_mean_match_indices, extracted_k1_text_mean_match_scores, extracted_text_k1_mean_match_correct_total = retrieve(sim_embeddings.pooler_output.cpu().detach(), GPT2_extracted_text_mean_embedding.cpu().detach(), K=1)

print("正确匹配的条目数：", extracted_k1_text_match_correct_total)
print("正确匹配的条目数：", extracted_text_k1_mean_match_correct_total)

#### text NN

In [ ]:
GPT2_texts_cls_NN=NN(embedding=GPT2_cls_embedding)
GPT2_texts_mean_NN=NN(embedding=GPT2_mean_embedding)
GPT2_extracted_text_cls_NN=NN(embedding=GPT2_extracted_text_cls_embedding)
GPT2_extracted_text_mean_NN=NN(embedding=GPT2_extracted_text_mean_embedding)

In [ ]:
GPT2_texts_cls_NN_distance,GPT2_texts_cls_NN_indes=GPT2_texts_cls_NN.kneighbors(sim_embeddings.pooler_output)
GPT2_texts_mean_NN_distance,GPT2_texts_mean_NN_indes=GPT2_texts_mean_NN.kneighbors(sim_embeddings.pooler_output)

GPT2_extracted_text_cls_NN_distance,GPT2_extracted_text_cls_NN_indes=GPT2_extracted_text_cls_NN.kneighbors(sim_embeddings.pooler_output)
GPT2_extracted_text_mean_NN_distance,GPT2_extracted_text_mean_NN_indes=GPT2_extracted_text_mean_NN.kneighbors(sim_embeddings.pooler_output)


In [ ]:
overleap(GPT2_texts_match_indices.cpu().numpy(), GPT2_texts_cls_NN_indes)

overleap(GPT2_texts_mean_match_indices.cpu().numpy(), GPT2_texts_mean_NN_indes)
#_____________cross should get low score_____________

overleap(GPT2_texts_match_indices.cpu().numpy(), GPT2_texts_mean_NN_indes)
overleap(GPT2_texts_mean_match_indices.cpu().numpy(), GPT2_texts_cls_NN_indes)
print('='*10)
overleap(GPT2_extracted_text_match_indices.cpu().numpy(), GPT2_extracted_text_cls_NN_indes)
overleap(GPT2_extracted_text_mean_match_indices.cpu().numpy(), GPT2_extracted_text_mean_NN_indes)
#_____________cross should get low score_____________

overleap(GPT2_extracted_text_mean_match_indices.cpu().numpy(), GPT2_extracted_text_cls_NN_indes)
overleap(GPT2_extracted_text_match_indices.cpu().numpy(), GPT2_extracted_text_mean_NN_indes)


#### SMILES NN

In [ ]:
GPT2SMIS_texts_cls_NN=NN(embedding=sim_embeddings.pooler_output)
GPT2SMIS_texts_mean_NN=NN(embedding=sim_embeddings.pooler_output)
GPT2SMIS_extracted_text_cls_NN=NN(embedding=sim_embeddings.pooler_output)
GPT2SMIS_extracted_text_mean_NN=NN(embedding=sim_embeddings.pooler_output)

In [ ]:
GPT2SMIS_texts_cls_NN_distance,GPT2SMIS_texts_cls_NN_indes=GPT2SMIS_texts_cls_NN.kneighbors(GPT2_cls_embedding.cpu().numpy())
GPT2SMIS_texts_mean_NN_distance,GPT2SMIS_texts_mean_NN_indes=GPT2SMIS_texts_mean_NN.kneighbors(GPT2_mean_embedding.cpu().numpy())

GPT2SMIS_extracted_text_cls_NN_distance,GPT2SMIS_extracted_text_cls_NN_indes=GPT2SMIS_extracted_text_cls_NN.kneighbors(GPT2_extracted_text_cls_embedding.cpu().numpy())
GPT2SMIS_extracted_text_mean_NN_distance,GPT2SMIS_extracted_text_mean_NN_indes=GPT2SMIS_extracted_text_mean_NN.kneighbors(GPT2_extracted_text_mean_embedding.cpu().numpy())

In [ ]:
GPT2_texts_match_indices.shape,GPT2SMIS_texts_cls_NN_indes.shape

In [ ]:
overleap(GPT2_texts_match_indices.cpu().numpy(), GPT2SMIS_texts_cls_NN_indes)
overleap(GPT2_texts_mean_match_indices.cpu().numpy(), GPT2SMIS_texts_mean_NN_indes)
#_____________cross should get low score_____________

overleap(GPT2_texts_match_indices.cpu().numpy(), GPT2SMIS_texts_mean_NN_indes)
overleap(GPT2_texts_mean_match_indices.cpu().numpy(), GPT2SMIS_texts_cls_NN_indes)
print('='*20)
overleap(GPT2_extracted_text_match_indices.cpu().numpy(), GPT2SMIS_extracted_text_cls_NN_indes)
overleap(GPT2_extracted_text_mean_match_indices.cpu().numpy(), GPT2SMIS_extracted_text_mean_NN_indes)

#_____________cross should get low score_____________
overleap(GPT2_extracted_text_mean_match_indices.cpu().numpy(), GPT2SMIS_extracted_text_cls_NN_indes)
overleap(GPT2_extracted_text_match_indices.cpu().numpy(), GPT2SMIS_extracted_text_mean_NN_indes)

### text K-MEANS

In [ ]:
GPT2_cls_KM=KM(GPT2_cls_embedding.cpu().numpy())
GPT2_mean_KM=KM(GPT2_mean_embedding.cpu().numpy())

GPT2_extracted_text_cls_KM=KM(GPT2_extracted_text_cls_embedding.cpu().numpy())
GPT2_extracted_text_mean_KM=KM(GPT2_extracted_text_mean_embedding.cpu().numpy())

In [ ]:
GPT2_cls_KM_index=GPT2_cls_KM.predict(GPT2_cls_embedding.cpu().numpy())
GPT2_mean_KM_index=GPT2_mean_KM.predict(GPT2_mean_embedding.cpu().numpy())

GPT2_extracted_text_cls_KM_index=GPT2_extracted_text_cls_KM.predict(GPT2_extracted_text_cls_embedding.cpu().numpy())
GPT2_extracted_text_mean_KM_index=GPT2_extracted_text_mean_KM.predict(GPT2_extracted_text_mean_embedding.cpu().numpy())

In [ ]:
GPT2_cls_match_index=GPT2_cls_KM.predict(GPT2_cls_embedding[GPT2_texts_k1_match_indices].reshape(GPT2_cls_embedding.shape).cpu().numpy())
GPT2_mean_match_index=GPT2_mean_KM.predict(GPT2_mean_embedding[GPT2_texts_K1_mean_match_indices].reshape(GPT2_cls_embedding.shape).cpu().numpy())
GPT2_et_cls_match_index=GPT2_extracted_text_cls_KM.predict(GPT2_extracted_text_cls_embedding[extracted_k1_text_match_indices].reshape(GPT2_cls_embedding.shape).cpu().numpy())
GPT2_et_mean_match_index=GPT2_extracted_text_mean_KM.predict(GPT2_extracted_text_mean_embedding[extracted_k1_text_mean_match_indices].reshape(GPT2_cls_embedding.shape).cpu().numpy())

In [ ]:
GPT2_x_cls=GPT2_cls_match_index==GPT2_cls_KM_index
GPT2_x_mean=GPT2_mean_KM_index==GPT2_mean_match_index
GPT2_x_et_cls=GPT2_extracted_text_cls_KM_index==GPT2_et_cls_match_index
GPT2_x_et_mean=GPT2_et_mean_match_index==GPT2_extracted_text_mean_KM_index
np.sum(GPT2_x_cls),np.sum(GPT2_x_mean),np.sum(GPT2_x_et_cls),np.sum(GPT2_x_et_mean)

##### Smis K-MEANS

In [ ]:
GPT2SMIS_cls_KM=KM(sim_embeddings.pooler_output.cpu().numpy().astype(np.float32))
GPT2SMIS_mean_KM=KM(sim_embeddings.pooler_output.cpu().numpy().astype(np.float32))

GPT2SMIS_extracted_text_cls_KM=KM(sim_embeddings.pooler_output.cpu().numpy().astype(np.float32))
GPT2SMIS_extracted_text_mean_KM=KM(sim_embeddings.pooler_output.cpu().numpy().astype(np.float32))

In [ ]:
GPT2SMIS_cls_KM_index=GPT2SMIS_cls_KM.predict(sim_embeddings.pooler_output.cpu().numpy().astype(np.float32))
GPT2SMIS_mean_KM_index=GPT2SMIS_mean_KM.predict(sim_embeddings.pooler_output.cpu().numpy().astype(np.float32))

GPT2SMIS_extracted_text_cls_KM_index=GPT2SMIS_extracted_text_cls_KM.predict(sim_embeddings.pooler_output.cpu().numpy().astype(np.float32))
GPT2SMIS_extracted_text_mean_KM_index=GPT2SMIS_extracted_text_mean_KM.predict(sim_embeddings.pooler_output.cpu().numpy().astype(np.float32))

In [ ]:
GPT2SMIS_cls_match_index=GPT2SMIS_cls_KM.predict(sim_embeddings.pooler_output[GPT2_texts_k1_match_indices].reshape(GPT2_cls_embedding.shape).cpu().numpy())
GPT2SMIS_mean_match_index=GPT2SMIS_mean_KM.predict(sim_embeddings.pooler_output[GPT2_texts_K1_mean_match_indices].reshape(GPT2_cls_embedding.shape).cpu().numpy())
GPT2SMIS_et_cls_match_index=GPT2SMIS_extracted_text_cls_KM.predict(sim_embeddings.pooler_output[extracted_k1_text_match_indices].reshape(GPT2_cls_embedding.shape).cpu().numpy())
GPT2SMIS_et_mean_match_index=GPT2SMIS_extracted_text_mean_KM.predict(sim_embeddings.pooler_output[extracted_k1_text_mean_match_indices].reshape(GPT2_cls_embedding.shape).cpu().numpy())

In [ ]:
GPT2SMIS_x_cls=GPT2SMIS_cls_match_index==GPT2SMIS_cls_KM_index
GPT2SMIS_x_mean=GPT2SMIS_mean_KM_index==GPT2SMIS_mean_match_index
GPT2SMIS_x_et_cls=GPT2SMIS_extracted_text_cls_KM_index==GPT2SMIS_et_cls_match_index
GPT2SMIS_x_et_mean=GPT2SMIS_et_mean_match_index==GPT2SMIS_extracted_text_mean_KM_index
np.sum(GPT2SMIS_x_cls),np.sum(GPT2SMIS_x_mean),np.sum(GPT2SMIS_x_et_cls),np.sum(GPT2SMIS_x_et_mean)

In [ ]:
list(globals().keys())

In [ ]:
del GPT2embedding,
del GPT2_cls_embedding1,
del GPT2_mean_embedding1,
del GPT2_cls_embedding2,
del GPT2_mean_embedding2,
del GPT2_cls_embedding3,
del GPT2_mean_embedding3,
del GPT2_cls_embedding4,
del GPT2_mean_embedding4,
del GPT2_cls_embedding,
del GPT2_mean_embedding,
del GPT2_extracted_text_cls_embedding1,
del GPT2_extracted_text_mean_embedding1,
del GPT2_extracted_text_cls_embedding2,
del GPT2_extracted_text_mean_embedding2,
del GPT2_extracted_text_cls_embedding3,
del GPT2_extracted_text_mean_embedding3,
del GPT2_extracted_text_cls_embedding4,
del GPT2_extracted_text_mean_embedding4,
del GPT2_extracted_text_cls_embedding,
del GPT2_extracted_text_mean_embedding,

gc.collect()
torch.cuda.empty_cache()

## last word gpt2

In [ ]:
import torch
from transformers import GPT2Model, GPT2Tokenizer

def LastGPT2embedding(texts):

    # Check if GPU is available and set the device
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

    # Load pre-trained model and tokenizer and move the model to the GPU
    model = GPT2Model.from_pretrained('gpt2').to(device)
    tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
    
    if tokenizer.pad_token is None:
        tokenizer.add_special_tokens({'pad_token': '[PAD]'})
        model.resize_token_embeddings(len(tokenizer))
    # Input sentences
    # texts = ['Your sentence here1', 'Your sentence here2']

    # Tokenize input sentence and move input tensors to the GPU

    template = 'This_sentence_:_"The_molecule_is_a_nitrile_that_is_acetonitrile_where_one_of_the_methyl_hydrogens_is_substituted_by_a_2-methylphenyl_group."_means_in_one_word:"Acetonitrile".This_sentence_:_"*sent_0*"_means_in_one_word:"'
    inputs = tokenizer([template.replace('*sent_0*', i).replace('_', ' ') for i in texts], padding=True,  return_tensors="pt")['input_ids'].to(device)
    with torch.no_grad():
        embeddings = model(inputs, output_hidden_states=True, return_dict=True).hidden_states[-1][:, -1, :]
    print(embeddings.shape)
    return embeddings



In [ ]:
# LastGPT2_embeddings=LastGPT2embedding(texts)
LastGPT2_embeddings1=LastGPT2embedding(texts[:1000])
LastGPT2_embeddings2=LastGPT2embedding(texts[1000:2000])
LastGPT2_embeddings3=LastGPT2embedding(texts[2000:3000])
LastGPT2_embeddings4=LastGPT2embedding(texts[3000:])

LastGPT2_embeddings=torch.cat((LastGPT2_embeddings1,LastGPT2_embeddings2,LastGPT2_embeddings3,LastGPT2_embeddings4))
LastGPT2_embeddings.shape

In [ ]:
# LastGPT2_et_embeddings=LastGPT2embedding(extracted_text)
LastGPT2_et_embeddings1=LastGPT2embedding(extracted_text[:1000])
LastGPT2_et_embeddings2=LastGPT2embedding(extracted_text[1000:2000])
LastGPT2_et_embeddings3=LastGPT2embedding(extracted_text[2000:3000])
LastGPT2_et_embeddings4=LastGPT2embedding(extracted_text[3000:])

LastGPT2_et_embeddings=torch.cat((LastGPT2_et_embeddings1,LastGPT2_et_embeddings2,LastGPT2_et_embeddings3,LastGPT2_et_embeddings4))

LastGPT2_et_embeddings.shape

In [ ]:
LastGPT2_embeddings.shape,LastGPT2_et_embeddings.shape

In [ ]:
de_biased_cka(LastGPT2_embeddings,sim_embeddings.pooler_output),de_biased_cka(LastGPT2_et_embeddings,sim_embeddings.pooler_output)

In [ ]:
CKA(LastGPT2_embeddings,sim_embeddings.pooler_output.to(device)),CKA(LastGPT2_et_embeddings,sim_embeddings.pooler_output.to(device))

In [ ]:
K=10

LastGPT2_texts_match_indices, LastGPT2_texts_match_scores, LastGPT2_texts_match_correct_total = retrieve(sim_embeddings.pooler_output.cpu().detach(), LastGPT2_embeddings.cpu().detach(), K=K)

# 打印结果
# print("Top K 索引：", indices)
# print("Top K 分数：", scores)
print("正确匹配的条目数：", LastGPT2_texts_match_correct_total)
LastGPT2_extracted_text_match_indices, LastGPT2_extracted_text_match_scores, LastGPT2_extracted_text_match_correct_total = retrieve(sim_embeddings.pooler_output.cpu().detach(), LastGPT2_et_embeddings.cpu().detach(), K=K)

print("正确匹配的条目数：", LastGPT2_extracted_text_match_correct_total)

In [ ]:
K=1

LastGPT2_texts_k1_match_indices, LastGPT2_texts_match_scores, LastGPT2_texts_k1_match_correct_total = retrieve(sim_embeddings.pooler_output.cpu().detach(), LastGPT2_embeddings.cpu().detach(), K=K)

# 打印结果
# print("Top K 索引：", indices)
# print("Top K 分数：", scores)
print("正确匹配的条目数：", LastGPT2_texts_k1_match_correct_total)
LastGPT2_extracted_k1_text_match_indices, LastGPT2_extracted_text_match_scores, LastGPT2_extracted_text_k1_match_correct_total = retrieve(sim_embeddings.pooler_output.cpu().detach(), LastGPT2_et_embeddings.cpu().detach(), K=K)

print("正确匹配的条目数：", LastGPT2_extracted_text_k1_match_correct_total)

### text

In [ ]:
LastGPT2_texts_cls_NN=NN(embedding=LastGPT2_embeddings)
LastGPT2_extracted_text_cls_NN=NN(embedding=LastGPT2_et_embeddings)

In [ ]:
LastGPT2_texts_cls_NN_distance,LastGPT2_texts_cls_NN_indes=LastGPT2_texts_cls_NN.kneighbors(sim_embeddings.pooler_output.cpu().detach())

LastGPT2_extracted_text_cls_NN_distance,LastGPT2_extracted_text_cls_NN_indes=LastGPT2_extracted_text_cls_NN.kneighbors(sim_embeddings.pooler_output.cpu().detach())

In [ ]:
overleap(LastGPT2_texts_match_indices.cpu().numpy(), LastGPT2_texts_cls_NN_indes)

#_____________cross should get low score_____________

overleap(LastGPT2_extracted_text_match_indices.cpu().numpy(), LastGPT2_extracted_text_cls_NN_indes)

In [ ]:
LastGPT2_cls_KM=KM(LastGPT2_embeddings.cpu().numpy())
LastGPT2_extracted_text_cls_KM=KM(LastGPT2_et_embeddings.cpu().numpy())

In [ ]:
LastGPT2_cls_KM_index=LastGPT2_cls_KM.predict(LastGPT2_embeddings.cpu().numpy())
LastGPT2_extracted_text_cls_KM_index=LastGPT2_extracted_text_cls_KM.predict(LastGPT2_et_embeddings.cpu().numpy())

In [ ]:
LastGPT2_cls_match_index=LastGPT2_cls_KM.predict(LastGPT2_embeddings[LastGPT2_texts_k1_match_indices].reshape(LastGPT2_embeddings.shape).cpu().numpy())
LastGPT2_et_cls_match_index=LastGPT2_extracted_text_cls_KM.predict(LastGPT2_et_embeddings[LastGPT2_extracted_k1_text_match_indices].reshape(LastGPT2_et_embeddings.shape).cpu().numpy())

In [ ]:
LastGPT2_x_cls=LastGPT2_cls_match_index==LastGPT2_cls_KM_index
LastGPT2_x_et_cls=LastGPT2_extracted_text_cls_KM_index==LastGPT2_et_cls_match_index

np.sum(LastGPT2_x_cls),np.sum(LastGPT2_x_et_cls)

In [ ]:
# np.sum(x_cls),np.sum(x_et_cls)

### SMIELS

In [ ]:
LastGPT2SMIS_texts_cls_NN=NN(embedding=sim_embeddings.pooler_output)
LastGPT2SMIS_extracted_text_cls_NN=NN(embedding=sim_embeddings.pooler_output)

In [ ]:
LastGPT2SMIS_texts_cls_NN_distance,LastGPT2SMIS_texts_cls_NN_indes=LastGPT2SMIS_texts_cls_NN.kneighbors(LastGPT2_embeddings.cpu().numpy())

LastGPT2SMIS_extracted_text_cls_NN_distance,LastGPT2SMIS_extracted_text_cls_NN_indes=LastGPT2SMIS_extracted_text_cls_NN.kneighbors(LastGPT2_et_embeddings.cpu().numpy())

In [ ]:
overleap(LastGPT2_texts_match_indices.cpu().numpy(), LastGPT2SMIS_texts_cls_NN_indes)

#_____________cross should get low score_____________

overleap(LastGPT2_extracted_text_match_indices.cpu().numpy(), LastGPT2SMIS_extracted_text_cls_NN_indes)

In [ ]:
LastGPT2SMIS_cls_KM=KM(sim_embeddings.pooler_output.cpu().numpy().astype(np.float32))
LastGPT2SMIS_extracted_text_cls_KM=KM(sim_embeddings.pooler_output.cpu().numpy().astype(np.float32))

In [ ]:
LastGPT2SMIS_cls_KM_index=LastGPT2SMIS_cls_KM.predict(sim_embeddings.pooler_output.cpu().numpy().astype(np.float32))
LastGPT2SMIS_extracted_text_cls_KM_index=LastGPT2SMIS_extracted_text_cls_KM.predict(sim_embeddings.pooler_output.cpu().numpy().astype(np.float32))

In [ ]:
LastGPT2SMIS_cls_match_index=LastGPT2SMIS_cls_KM.predict(sim_embeddings.pooler_output[LastGPT2_texts_k1_match_indices].reshape(LastGPT2_embeddings.shape).cpu().numpy().astype(np.float32))
LastGPT2SMIS_et_cls_match_index=LastGPT2SMIS_extracted_text_cls_KM.predict(sim_embeddings.pooler_output[LastGPT2_extracted_k1_text_match_indices].reshape(LastGPT2_et_embeddings.shape).cpu().numpy().astype(np.float32))

In [ ]:
LastGPT2SMIS_x_cls=LastGPT2SMIS_cls_match_index==LastGPT2SMIS_cls_KM_index
LastGPT2SMIS_x_et_cls=LastGPT2SMIS_extracted_text_cls_KM_index==LastGPT2SMIS_et_cls_match_index

np.sum(LastGPT2SMIS_x_cls),np.sum(LastGPT2SMIS_x_et_cls)

In [ ]:
# np.sum(x_cls),np.sum(x_et_cls)

In [ ]:
del LastGPT2embedding,
del LastGPT2_embeddings1,
del LastGPT2_embeddings2,
del LastGPT2_embeddings3,
del LastGPT2_embeddings4,
del LastGPT2_embeddings,
del LastGPT2_et_embeddings1,
del LastGPT2_et_embeddings2,
del LastGPT2_et_embeddings3,
del LastGPT2_et_embeddings4,
del LastGPT2_et_embeddings,


import gc
gc.collect()
torch.cuda.empty_cache()

In [ ]:
list(globals().keys())

In [ ]:
del GPT2Model,
del GPT2Tokenizer,
del GPT2_texts_match_indices,
del GPT2_texts_match_scores,
del GPT2_texts_match_correct_total,
del GPT2_texts_mean_match_indices,
del GPT2_texts_mean_match_scores,
del GPT2_texts_mean_match_correct_total,
del GPT2_extracted_text_match_indices,
del GPT2_extracted_text_match_scores,
del GPT2_extracted_text_match_correct_total,
del GPT2_extracted_text_mean_match_indices,
del GPT2_extracted_text_mean_match_scores,
del GPT2_extracted_text_mean_match_correct_total,
del GPT2_texts_k1_match_indices,
del GPT2_texts_k1_match_scores,
del GPT2_texts_k1_match_correct_total,
del GPT2_texts_K1_mean_match_indices,
del GPT2_texts_k1_mean_match_scores,
del GPT2_texts_k1_mean_match_correct_total,
del extracted_k1_text_match_indices,
del extracted_k1_text_match_scores,
del extracted_k1_text_match_correct_total,
del extracted_k1_text_mean_match_indices,
del extracted_k1_text_mean_match_scores,
del extracted_text_k1_mean_match_correct_total,
del GPT2_texts_cls_NN,
del GPT2_texts_mean_NN,
del GPT2_extracted_text_cls_NN,
del GPT2_extracted_text_mean_NN,
del GPT2_texts_cls_NN_distance,
del GPT2_texts_cls_NN_indes,
del GPT2_texts_mean_NN_distance,
del GPT2_texts_mean_NN_indes,
del GPT2_extracted_text_cls_NN_distance,
del GPT2_extracted_text_cls_NN_indes,
del GPT2_extracted_text_mean_NN_distance,
del GPT2_extracted_text_mean_NN_indes,
del GPT2SMIS_texts_cls_NN,
del GPT2SMIS_texts_mean_NN,
del GPT2SMIS_extracted_text_cls_NN,
del GPT2SMIS_extracted_text_mean_NN,
del GPT2SMIS_texts_cls_NN_distance,
del GPT2SMIS_texts_cls_NN_indes,
del GPT2SMIS_texts_mean_NN_distance,
del GPT2SMIS_texts_mean_NN_indes,
del GPT2SMIS_extracted_text_cls_NN_distance,
del GPT2SMIS_extracted_text_cls_NN_indes,
del GPT2SMIS_extracted_text_mean_NN_distance,
del GPT2SMIS_extracted_text_mean_NN_indes,
del GPT2_cls_KM,
del GPT2_mean_KM,
del GPT2_extracted_text_cls_KM,
del GPT2_extracted_text_mean_KM,
del GPT2_cls_KM_index,
del GPT2_mean_KM_index,
del GPT2_extracted_text_cls_KM_index,
del GPT2_extracted_text_mean_KM_index,
del GPT2_cls_match_index,
del GPT2_mean_match_index,
del GPT2_et_cls_match_index,
del GPT2_et_mean_match_index,
del GPT2_x_cls,
del GPT2_x_mean,
del GPT2_x_et_cls,
del GPT2_x_et_mean,
del GPT2SMIS_cls_KM,
del GPT2SMIS_mean_KM,
del GPT2SMIS_extracted_text_cls_KM,
del GPT2SMIS_extracted_text_mean_KM,
del GPT2SMIS_cls_KM_index,
del GPT2SMIS_mean_KM_index,
del GPT2SMIS_extracted_text_cls_KM_index,
del GPT2SMIS_extracted_text_mean_KM_index,
del GPT2SMIS_cls_match_index,
del GPT2SMIS_mean_match_index,
del GPT2SMIS_et_cls_match_index,
del GPT2SMIS_et_mean_match_index,
del GPT2SMIS_x_cls,
del GPT2SMIS_x_mean,
del GPT2SMIS_x_et_cls,
del GPT2SMIS_x_et_mean,
del K,
del LastGPT2_texts_match_indices,
del LastGPT2_texts_match_scores,
del LastGPT2_texts_match_correct_total,
del LastGPT2_extracted_text_match_indices,
del LastGPT2_extracted_text_match_scores,
del LastGPT2_extracted_text_match_correct_total,
del LastGPT2_texts_k1_match_indices,
del LastGPT2_texts_k1_match_correct_total,
del LastGPT2_extracted_k1_text_match_indices,
del LastGPT2_extracted_text_k1_match_correct_total,
del LastGPT2_texts_cls_NN,
del LastGPT2_extracted_text_cls_NN,
del LastGPT2_texts_cls_NN_distance,
del LastGPT2_texts_cls_NN_indes,
del LastGPT2_extracted_text_cls_NN_distance,
del LastGPT2_extracted_text_cls_NN_indes,
del LastGPT2_cls_KM,
del LastGPT2_extracted_text_cls_KM,
del LastGPT2_cls_KM_index,
del LastGPT2_extracted_text_cls_KM_index,
del LastGPT2_cls_match_index,
del LastGPT2_et_cls_match_index,
del LastGPT2_x_cls,
del LastGPT2_x_et_cls,
del LastGPT2SMIS_texts_cls_NN,
del LastGPT2SMIS_extracted_text_cls_NN,
del LastGPT2SMIS_texts_cls_NN_distance,
del LastGPT2SMIS_texts_cls_NN_indes,
del LastGPT2SMIS_extracted_text_cls_NN_distance,
del LastGPT2SMIS_extracted_text_cls_NN_indes,
del LastGPT2SMIS_cls_KM,
del LastGPT2SMIS_extracted_text_cls_KM,
del LastGPT2SMIS_cls_KM_index,
del LastGPT2SMIS_extracted_text_cls_KM_index,
del LastGPT2SMIS_cls_match_index,
del LastGPT2SMIS_et_cls_match_index,
del LastGPT2SMIS_x_cls,
del LastGPT2SMIS_x_et_cls,

import gc
gc.collect()
torch.cuda.empty_cache()

## Summarise last word gpt2

In [ ]:
import torch
from transformers import GPT2Model, GPT2Tokenizer

def Summarise_GPT2embedding(texts):

    # Check if GPU is available and set the device
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

    # Load pre-trained model and tokenizer and move the model to the GPU
    model = GPT2Model.from_pretrained('gpt2').to(device)
    tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
    
    if tokenizer.pad_token is None:
        tokenizer.add_special_tokens({'pad_token': '[PAD]'})
        model.resize_token_embeddings(len(tokenizer))
    # Input sentences
    # texts = ['Your sentence here1', 'Your sentence here2']

    # Tokenize input sentence and move input tensors to the GPU

    template = 'This_sentence_:_"The_molecule_is_a_nitrile_that_is_acetonitrile_where_one_of_the_methyl_hydrogens_is_substituted_by_a_2-methylphenyl_group."_summarise_in_one_word:"Acetonitrile".This_sentence_:_"*sent_0*"_summarise_in_one_word:"'
    inputs = tokenizer([template.replace('*sent_0*', i).replace('_', ' ') for i in texts], padding=True,  return_tensors="pt")['input_ids'].to(device)
    with torch.no_grad():
        embeddings = model(inputs, output_hidden_states=True, return_dict=True).hidden_states[-1][:, -1, :]
    print(embeddings.shape)
    return embeddings



In [ ]:
# LastSGPT2_embeddings=Summarise_GPT2embedding(texts)
LastSGPT2_embeddings1=Summarise_GPT2embedding(texts[:1000])
LastSGPT2_embeddings2=Summarise_GPT2embedding(texts[1000:2000])
LastSGPT2_embeddings3=Summarise_GPT2embedding(texts[2000:3000])
LastSGPT2_embeddings4=Summarise_GPT2embedding(texts[3000:])

LastSGPT2_embeddings=torch.cat((LastSGPT2_embeddings1,LastSGPT2_embeddings2,LastSGPT2_embeddings3,LastSGPT2_embeddings4))

In [ ]:
# LastSGPT2_et_embeddings=Summarise_GPT2embedding(extracted_text)
LastSGPT2_et_embeddings1=Summarise_GPT2embedding(extracted_text[:1000])
LastSGPT2_et_embeddings2=Summarise_GPT2embedding(extracted_text[1000:2000])
LastSGPT2_et_embeddings3=Summarise_GPT2embedding(extracted_text[2000:3000])
LastSGPT2_et_embeddings4=Summarise_GPT2embedding(extracted_text[3000:])

LastSGPT2_et_embeddings=torch.cat((LastSGPT2_et_embeddings1,LastSGPT2_et_embeddings2,LastSGPT2_et_embeddings3,LastSGPT2_et_embeddings4))

In [ ]:
LastSGPT2_embeddings.shape,LastSGPT2_et_embeddings.shape

In [ ]:
de_biased_cka(LastSGPT2_embeddings,sim_embeddings.pooler_output),de_biased_cka(LastSGPT2_et_embeddings,sim_embeddings.pooler_output)

In [ ]:
CKA(LastSGPT2_embeddings,sim_embeddings.pooler_output.to(device)),CKA(LastSGPT2_et_embeddings,sim_embeddings.pooler_output.to(device))

In [ ]:
K=10

LastSGPT2_texts_match_indices, LastSGPT2_texts_match_scores, LastSGPT2_texts_match_correct_total = retrieve(sim_embeddings.pooler_output.cpu().detach(), LastSGPT2_embeddings.cpu().detach(), K=K)

print("正确匹配的条目数：", LastSGPT2_texts_match_correct_total)
LastSGPT2_extracted_text_match_indices, LastSGPT2_extracted_text_match_scores, LastSGPT2_extracted_text_match_correct_total = retrieve(sim_embeddings.pooler_output.cpu().detach(), LastSGPT2_et_embeddings.cpu().detach(), K=K)

print("正确匹配的条目数：", LastSGPT2_extracted_text_match_correct_total)

In [ ]:
K=1

LastSGPT2_texts_k1_match_indices, LastSGPT2_texts_k1_match_scores, LastSGPT2_texts_k1_match_correct_total = retrieve(sim_embeddings.pooler_output.cpu().detach(), LastSGPT2_embeddings.cpu().detach(), K=K)

# 打印结果
# print("Top K 索引：", indices)
# print("Top K 分数：", scores)
print("正确匹配的条目数：", LastSGPT2_texts_k1_match_correct_total)
LastSGPT2_extracted_k1_text_match_indices, LastSGPT2_extracted_k1_text_match_scores, LastSGPT2_extracted_k1_text_match_correct_total = retrieve(sim_embeddings.pooler_output.cpu().detach(), LastSGPT2_et_embeddings.cpu().detach(), K=K)

print("正确匹配的条目数：", LastSGPT2_extracted_k1_text_match_correct_total)

### text

In [ ]:
LastSGPT2_texts_cls_NN=NN(embedding=LastSGPT2_embeddings)
LastSGPT2_extracted_text_cls_NN=NN(embedding=LastSGPT2_et_embeddings)

In [ ]:
LastSGPT2_texts_cls_NN_distance,LastSGPT2_texts_cls_NN_indes=LastSGPT2_texts_cls_NN.kneighbors(sim_embeddings.pooler_output.cpu().detach())
LastSGPT2_extracted_text_cls_NN_distance,LastSGPT2_extracted_text_cls_NN_indes=LastSGPT2_extracted_text_cls_NN.kneighbors(sim_embeddings.pooler_output.cpu().detach())

In [ ]:
overleap(LastSGPT2_texts_match_indices.cpu().numpy(), LastSGPT2_texts_cls_NN_indes)
#_____________cross should get low score_____________
overleap(LastSGPT2_extracted_text_match_indices.cpu().numpy(), LastSGPT2_extracted_text_cls_NN_indes)

In [ ]:
LastSGPT2_cls_KM=KM(LastSGPT2_embeddings.cpu().numpy())
LastSGPT2_extracted_text_cls_KM=KM(LastSGPT2_et_embeddings.cpu().numpy())

In [ ]:
LastSGPT2_cls_KM_index=LastSGPT2_cls_KM.predict(LastSGPT2_embeddings.cpu().numpy())
LastSGPT2_extracted_text_cls_KM_index=LastSGPT2_extracted_text_cls_KM.predict(LastSGPT2_et_embeddings.cpu().numpy())

In [ ]:
LastSGPT2_cls_match_index=LastSGPT2_cls_KM.predict(LastSGPT2_embeddings[LastSGPT2_texts_k1_match_indices].reshape(LastSGPT2_embeddings.shape).cpu().numpy())
LastSGPT2_et_cls_match_index=LastSGPT2_extracted_text_cls_KM.predict(LastSGPT2_et_embeddings[LastSGPT2_extracted_k1_text_match_indices].reshape(LastSGPT2_et_embeddings.shape).cpu().numpy())

In [ ]:
LastSGPT2_x_cls=LastSGPT2_cls_match_index==LastSGPT2_cls_KM_index
LastSGPT2_x_et_cls=LastSGPT2_extracted_text_cls_KM_index==LastSGPT2_et_cls_match_index

np.sum(LastSGPT2_x_cls),np.sum(LastSGPT2_x_et_cls)

In [ ]:
# np.sum(x_cls),np.sum(x_et_cls)

### SMIELS

In [ ]:
LastSGPT2SMIS_texts_cls_NN=NN(embedding=sim_embeddings.pooler_output)
LastSGPT2SMIS_extracted_text_cls_NN=NN(embedding=sim_embeddings.pooler_output)

In [ ]:
LastSGPT2SMIS_texts_cls_NN_distance,LastSGPT2SMIS_texts_cls_NN_indes=LastSGPT2SMIS_texts_cls_NN.kneighbors(LastSGPT2_embeddings.cpu().numpy())
LastSGPT2SMIS_extracted_text_cls_NN_distance,LastSGPT2SMIS_extracted_text_cls_NN_indes=LastSGPT2SMIS_extracted_text_cls_NN.kneighbors(LastSGPT2_et_embeddings.cpu().numpy())

In [ ]:
overleap(LastSGPT2_texts_match_indices.cpu().numpy(), LastSGPT2SMIS_texts_cls_NN_indes)
#_____________cross should get low score_____________
overleap(LastSGPT2_extracted_text_match_indices.cpu().numpy(), LastSGPT2SMIS_extracted_text_cls_NN_indes)

In [ ]:
LastSGPT2SMIS_cls_KM=KM(sim_embeddings.pooler_output.cpu().numpy().astype(np.float32))
LastSGPT2SMIS_extracted_text_cls_KM=KM(sim_embeddings.pooler_output.cpu().numpy().astype(np.float32))

In [ ]:
LastSGPT2SMIS_cls_KM_index=LastSGPT2SMIS_cls_KM.predict(sim_embeddings.pooler_output.cpu().numpy().astype(np.float32))
LastSGPT2SMIS_extracted_text_cls_KM_index=LastSGPT2SMIS_extracted_text_cls_KM.predict(sim_embeddings.pooler_output.cpu().numpy().astype(np.float32))

In [ ]:
LastSGPT2SMIS_cls_match_index=LastSGPT2SMIS_cls_KM.predict(sim_embeddings.pooler_output[LastSGPT2_texts_k1_match_indices].reshape(LastSGPT2_embeddings.shape).cpu().numpy().astype(np.float32))
LastSGPT2SMIS_et_cls_match_index=LastSGPT2SMIS_extracted_text_cls_KM.predict(sim_embeddings.pooler_output[LastSGPT2_extracted_k1_text_match_indices].reshape(LastSGPT2_et_embeddings.shape).cpu().numpy().astype(np.float32))

In [ ]:
LastSGPT2SMIS_x_cls=LastSGPT2SMIS_cls_match_index==LastSGPT2SMIS_cls_KM_index
LastSGPT2SMIS_x_et_cls=LastSGPT2SMIS_extracted_text_cls_KM_index==LastSGPT2SMIS_et_cls_match_index

np.sum(LastSGPT2SMIS_x_cls),np.sum(LastSGPT2SMIS_x_et_cls)

In [ ]:
del LastSGPT2_embeddings1,
del LastSGPT2_embeddings2,
del LastSGPT2_embeddings3,
del LastSGPT2_embeddings4,
del LastSGPT2_embeddings,
del LastSGPT2_et_embeddings1,
del LastSGPT2_et_embeddings2,
del LastSGPT2_et_embeddings3,
del LastSGPT2_et_embeddings4,
del LastSGPT2_et_embeddings,

import gc
gc.collect()
torch.cuda.empty_cache()

## MolCLR

In [ ]:
# gcn_embeddings=torch.load('gcn_embeddings.pt')
# gcn_out_embeddings=torch.load('gcn_out_embeddings.pt')
# gin_embeddings=torch.load('gin_embeddings.pt')
# gin_out_embeddings=torch.load('gin_out_embeddings.pt')

In [ ]:
# gcn_embeddings.shape,gcn_out_embeddings.shape,gin_embeddings.shape,gin_out_embeddings.shape,

In [ ]:
import torch
from torch_geometric.data import Data
from rdkit import Chem
from rdkit.Chem import AllChem
from MolCLR.models.ginet_molclr import GINet
from MolCLR.models.gcn_molclr import GCN

# Convert a SMILES string into a graph representation
def smiles_to_graph(smiles):
    mol = Chem.MolFromSmiles(smiles)
    Chem.SanitizeMol(mol)
    
    # Extract atom features and edge features
    atom_features = torch.tensor([[atom.GetAtomicNum(), atom.GetChiralTag()] for atom in mol.GetAtoms()], dtype=torch.long)
    bond_features = torch.tensor([[bond.GetBondTypeAsDouble(), bond.GetStereo()] for bond in mol.GetBonds()], dtype=torch.long)
    
    # Build edge_index (source, target) pairs for the molecular graph
    edge_index = torch.tensor([[bond.GetBeginAtomIdx(), bond.GetEndAtomIdx()] for bond in mol.GetBonds()], dtype=torch.long).t().contiguous()
    
    # Create a batch (for a single molecule, the batch index will be all 0s)
    data = Data(x=atom_features, edge_index=edge_index, edge_attr=bond_features, batch=torch.tensor([0] * atom_features.size(0)))
    return data

# Load the GINet model based on the configuration from config.yaml
def load_model(path='',type='gin'):
    # Initialize the GINet model with the exact settings from config.yaml
    if type=='gin':
        model = GINet(num_layer=5, emb_dim=300, feat_dim=512, drop_ratio=0, pool='mean')
    elif type=='gcn':
        model=GCN(num_layer=5, emb_dim=300, feat_dim=512, drop_ratio=0, pool='mean')
    
    # Optionally load pre-trained weights from the "pretrained_gin" model
    pretrained_model_path = path  # Ensure this path exists and is correct
    try:
        model.load_state_dict(torch.load(pretrained_model_path,map_location='cuda:0'))
        print("Loaded pre-trained model from:", pretrained_model_path)
    except FileNotFoundError:
        print("Pre-trained weights not found, training from scratch.")

    model.eval()  # Set model to evaluation mode
    return model


gin_model = load_model(path='./../MolCLR/ckpt/pretrained_gzin/checkpoints/model.pth',type='gin')
gcn_model = load_model(path='./../MolCLR/ckpt/pretrained_gcn/checkpoints/model.pth',type='gcn')


In [ ]:
gcn_h_list=[]
gcn_out_list=[]
gin_h_list=[]
gin_out_list=[]
text_list=[]
et_list=[]
pass_list=[]
compute_list=[]
for index,i in enumerate(smis) :
    # print(index)
    # print(i)
    graph_data = smiles_to_graph(i)
    # print(graph_data)
    try:
        gin_h, gin_out = gin_model(graph_data)
        gin_h_list.append(gin_h)
        gin_out_list.append(gin_out)
        gcn_h, gcn_out = gcn_model(graph_data)
        gcn_h_list.append(gcn_h)
        gcn_out_list.append(gcn_out)
        text_list.append(texts[index])
        et_list.append(extracted_text[index])
        compute_list.append(index)
    except IndexError:
        # print("PASS:",index)
        pass_list.append(index)

In [ ]:
len(compute_list),len(pass_list),len(gin_h_list),len(gin_out_list),len(gcn_h_list),len(gcn_out_list),len(text_list),len(et_list)

In [ ]:
# with open('compute_list.txt','w') as f:
#     for i in compute_list:
#         f.writelines(str(i)+'\n')

In [ ]:
# with open('pass_list.txt','w') as f:
#     for i in pass_list:
#         f.writelines(str(i)+'\n')

In [ ]:
gcn_embeddings = torch.cat(gcn_h_list, dim=0)
gcn_out_embeddings= torch.cat(gcn_out_list, dim=0)
gin_embeddings = torch.cat(gin_h_list, dim=0)
gin_out_embeddings= torch.cat(gin_out_list, dim=0)
gcn_embeddings.shape,gin_embeddings.shape,gcn_out_embeddings.shape,gin_out_embeddings.shape

## SCI-BERT

In [ ]:
from transformers import AutoTokenizer,AutoModel

def SCIBERT(texts=texts):

    tokenizer = AutoTokenizer.from_pretrained('allenai/scibert_scivocab_uncased')
    model = AutoModel.from_pretrained('allenai/scibert_scivocab_uncased')
    model = model.to(device)

    # 将句子 token 化，并将输入转移到 GPU
    inputs = tokenizer(texts, return_tensors='pt', truncation=True, padding=True, max_length=512)
    inputs = {key: value.to(device) for key, value in inputs.items()}  # 转移到 GPU

    # 获取BERT的输出
    with torch.no_grad():  # 禁用梯度计算，节省内存
        outputs = model(**inputs)

    # BERT的最后隐藏层的输出 [batch_size, seq_len, hidden_size]
    last_hidden_states = outputs.last_hidden_state

    # 取 [CLS] token 的 embedding（句子嵌入）
    cls_embedding = last_hidden_states[:, 0, :]

    # 也可以取所有 token 的平均值作为句子嵌入
    mean_embedding = torch.mean(last_hidden_states, dim=1)

    print(f"CLS token embedding size: {cls_embedding.shape}")
    print(f"Mean token embedding size: {mean_embedding.shape}")

    return cls_embedding,mean_embedding

In [ ]:
cls_embedding,mean_embedding=SCIBERT(texts)

In [ ]:
extracted_text_cls_embedding,extracted_text_mean_embedding=SCIBERT(extracted_text)

In [ ]:
de_biased_cka(cls_embedding[compute_list], gcn_embeddings),de_biased_cka(cls_embedding[compute_list], gcn_out_embeddings),de_biased_cka(cls_embedding[compute_list],gin_embeddings),de_biased_cka(cls_embedding[compute_list],gin_out_embeddings)

In [ ]:
de_biased_cka(mean_embedding[compute_list],gcn_embeddings),de_biased_cka(mean_embedding[compute_list],gcn_out_embeddings),de_biased_cka(mean_embedding[compute_list],gin_embeddings),de_biased_cka(mean_embedding[compute_list],gin_out_embeddings)

In [ ]:
de_biased_cka(extracted_text_cls_embedding[compute_list], gcn_embeddings),de_biased_cka(extracted_text_cls_embedding[compute_list],gcn_out_embeddings),de_biased_cka(extracted_text_cls_embedding[compute_list],gin_embeddings),de_biased_cka(extracted_text_cls_embedding[compute_list],gin_out_embeddings)

In [ ]:
de_biased_cka(extracted_text_mean_embedding[compute_list], gcn_embeddings),de_biased_cka(extracted_text_mean_embedding[compute_list],gcn_out_embeddings),de_biased_cka(extracted_text_mean_embedding[compute_list],gin_embeddings),de_biased_cka(extracted_text_mean_embedding[compute_list],gin_out_embeddings)

In [ ]:
print(CKA(cls_embedding[compute_list], gcn_embeddings.to(device)),CKA(cls_embedding[compute_list], gcn_out_embeddings.to(device)),CKA(cls_embedding[compute_list],gin_embeddings.to(device)),CKA(cls_embedding[compute_list],gin_out_embeddings.to(device)))
print(CKA(mean_embedding[compute_list],gcn_embeddings.to(device)),CKA(mean_embedding[compute_list],gcn_out_embeddings.to(device)),CKA(mean_embedding[compute_list],gin_embeddings.to(device)),CKA(mean_embedding[compute_list],gin_out_embeddings.to(device)))
print(CKA(extracted_text_cls_embedding[compute_list], gcn_embeddings.to(device)),CKA(extracted_text_cls_embedding[compute_list],gcn_out_embeddings.to(device)),CKA(extracted_text_cls_embedding[compute_list],gin_embeddings.to(device)),CKA(extracted_text_cls_embedding[compute_list],gin_out_embeddings.to(device)))
print(CKA(extracted_text_mean_embedding[compute_list], gcn_embeddings.to(device)),CKA(extracted_text_mean_embedding[compute_list],gcn_out_embeddings.to(device)),CKA(extracted_text_mean_embedding[compute_list],gin_embeddings.to(device)),CKA(extracted_text_mean_embedding[compute_list],gin_out_embeddings.to(device)))

In [ ]:
len(text_list),len(et_list)

In [ ]:
del SCIBERT,
del cls_embedding,
del mean_embedding,
del extracted_text_cls_embedding,
del extracted_text_mean_embedding,

import gc
gc.collect()
torch.cuda.empty_cache()

## Sentence Bert

In [ ]:
torch.cuda.empty_cache()
from sentence_transformers import SentenceTransformer

# 1. Load a pretrained Sentence Transformer model
model = SentenceTransformer("distiluse-base-multilingual-cased-v2").to(device)
# 2. Calculate embeddings by calling model.encode()
embeddings = model.encode(text_list)
print('embeddings',embeddings.shape)

extracted_embeddings = model.encode(et_list)
print('extracted_embeddings',extracted_embeddings.shape)

In [ ]:
de_biased_cka(torch.from_numpy(embeddings)[compute_list],gcn_embeddings),de_biased_cka(torch.from_numpy(extracted_embeddings)[compute_list],gcn_embeddings),de_biased_cka(torch.from_numpy(embeddings)[compute_list],gcn_out_embeddings),de_biased_cka(torch.from_numpy(extracted_embeddings)[compute_list],gcn_out_embeddings)

In [ ]:
de_biased_cka(torch.from_numpy(embeddings)[compute_list],gin_embeddings),de_biased_cka(torch.from_numpy(extracted_embeddings)[compute_list],gin_embeddings),de_biased_cka(torch.from_numpy(embeddings)[compute_list],gin_out_embeddings),de_biased_cka(torch.from_numpy(extracted_embeddings)[compute_list],gin_out_embeddings)

In [ ]:
print(CKA(torch.from_numpy(embeddings)[compute_list],gcn_embeddings),CKA(torch.from_numpy(extracted_embeddings)[compute_list],gcn_embeddings),CKA(torch.from_numpy(embeddings)[compute_list],gcn_out_embeddings),CKA(torch.from_numpy(extracted_embeddings)[compute_list],gcn_out_embeddings))
print(CKA(torch.from_numpy(embeddings)[compute_list],gin_embeddings),CKA(torch.from_numpy(extracted_embeddings)[compute_list],gin_embeddings),CKA(torch.from_numpy(embeddings)[compute_list],gin_out_embeddings),CKA(torch.from_numpy(extracted_embeddings)[compute_list],gin_out_embeddings))

In [ ]:
print(CKA(torch.from_numpy(embeddings)[compute_list],gcn_embeddings),CKA(torch.from_numpy(extracted_embeddings)[compute_list],gcn_embeddings),CKA(torch.from_numpy(embeddings)[compute_list],gcn_out_embeddings),CKA(torch.from_numpy(extracted_embeddings)[compute_list],gcn_out_embeddings))
print(CKA(torch.from_numpy(embeddings)[compute_list],gin_embeddings),CKA(torch.from_numpy(extracted_embeddings)[compute_list],gin_embeddings),CKA(torch.from_numpy(embeddings)[compute_list],gin_out_embeddings),CKA(torch.from_numpy(extracted_embeddings)[compute_list],gin_out_embeddings))

### match

In [ ]:
K=10
# GCN
gcn_texts_match_indices, texts_match_scores, texts_match_correct_total = retrieve(gcn_embeddings.cpu(), torch.from_numpy(embeddings).cpu().detach()[compute_list], K=10)
gcn_extracted_match_indices, extracted_match_scores, extracted_match_correct_total = retrieve(gcn_embeddings.cpu(), torch.from_numpy(extracted_embeddings).cpu().detach()[compute_list], K=10)

print(texts_match_correct_total,extracted_match_correct_total)

In [ ]:
K=10
# GIN
gin_texts_match_indices, texts_match_scores, texts_match_correct_total = retrieve(gin_embeddings.cpu(), torch.from_numpy(embeddings).cpu().detach()[compute_list], K=10)
gin_extracted_match_indices, extracted_match_scores, extracted_match_correct_total = retrieve(gin_embeddings.cpu(), torch.from_numpy(extracted_embeddings).cpu().detach()[compute_list], K=10)

print(texts_match_correct_total,extracted_match_correct_total)

In [ ]:
K=1
# GCN
gcn_texts_k1_match_indices, texts_k1_match_scores, texts_k1_match_correct_total = retrieve(gcn_embeddings.cpu(), torch.from_numpy(embeddings).cpu().detach()[compute_list], K=K)
gcn_extracte_k1_match_indices, extracted_k1_match_scores, extracted_k1_match_correct_total = retrieve(gcn_embeddings.cpu(), torch.from_numpy(extracted_embeddings).cpu().detach()[compute_list], K=K)

print(texts_k1_match_correct_total,extracted_k1_match_correct_total)

In [ ]:
K=1
# GIN
gin_texts_k1_match_indices, texts_match_scores, texts_match_correct_total = retrieve(gin_embeddings.cpu(), torch.from_numpy(embeddings).cpu().detach()[compute_list], K=K)
gin_extracted_k1_match_indices, extracted_match_scores, extracted_match_correct_total = retrieve(gin_embeddings.cpu(), torch.from_numpy(extracted_embeddings).cpu().detach()[compute_list], K=K)

print(texts_match_correct_total,extracted_match_correct_total)

### text NN

In [ ]:
texts_mean_NN=NN(embedding=torch.from_numpy(embeddings)[compute_list])
extracted_text_mean_NN=NN(embedding=torch.from_numpy(extracted_embeddings)[compute_list])

In [ ]:
emb_distance,gcn_emb_index=texts_mean_NN.kneighbors(embeddings[compute_list])
et_distance,gcn_et_index=extracted_text_mean_NN.kneighbors(extracted_embeddings[compute_list])

emb_distance,gin_emb_index=texts_mean_NN.kneighbors(embeddings[compute_list])
et_distance,gin_et_index=extracted_text_mean_NN.kneighbors(extracted_embeddings[compute_list])

In [ ]:
overleap(gcn_texts_match_indices.cpu().numpy(), gcn_emb_index)

overleap(gcn_extracted_match_indices.cpu().numpy(), gcn_et_index)

overleap(gin_texts_match_indices.cpu().numpy(), gin_emb_index)

overleap(gin_extracted_match_indices.cpu().numpy(), gin_et_index)


### SMILES NN

In [ ]:
GCN_NN=NN(embedding=gcn_embeddings[compute_list])
GIN_NN=NN(embedding=gin_embeddings[compute_list])

In [ ]:
emb_distance,gcn_emb_index=GCN_NN.kneighbors(gcn_embeddings.detach().numpy()[compute_list])
et_distance,gcn_et_index=GCN_NN.kneighbors(gcn_embeddings.detach().numpy()[compute_list])

emb_distance,gin_emb_index=GIN_NN.kneighbors(gin_embeddings.detach().numpy()[compute_list])
et_distance,gin_et_index=GIN_NN.kneighbors(gin_embeddings.detach().numpy()[compute_list])

In [ ]:
overleap(gcn_texts_match_indices.cpu().numpy(), gcn_emb_index)

overleap(gcn_extracted_match_indices.cpu().numpy(), gcn_et_index)

overleap(gin_texts_match_indices.cpu().numpy(), gin_emb_index)

overleap(gin_extracted_match_indices.cpu().numpy(), gin_et_index)


### text K-means

In [ ]:
cls_KM=KM(embeddings[compute_list])

extracted_text_KM=KM(extracted_embeddings[compute_list])

In [ ]:
cls=cls_KM.predict(embeddings[compute_list])
extracted_text_cls=extracted_text_KM.predict(extracted_embeddings[compute_list])

In [ ]:
cls_index=cls_KM.predict(embeddings[gcn_texts_k1_match_indices].reshape(embeddings.shape)[compute_list])
et_index=extracted_text_KM.predict(extracted_embeddings[gcn_extracte_k1_match_indices].reshape(extracted_embeddings.shape)[compute_list])

In [ ]:
x_cls=cls_index==cls
et=et_index==extracted_text_cls
np.sum(x_cls),np.sum(et)

In [ ]:
cls_index=cls_KM.predict(embeddings[gin_texts_k1_match_indices].reshape(embeddings.shape)[compute_list])
et_index=extracted_text_KM.predict(extracted_embeddings[gin_extracted_k1_match_indices].reshape(extracted_embeddings.shape)[compute_list])

In [ ]:
x_cls=cls_index==cls
et=et_index==extracted_text_cls
np.sum(x_cls),np.sum(et)

### SMIS K-means

In [ ]:
cls_KM=KM(gcn_embeddings.detach().numpy())

extracted_text_KM=KM(gcn_embeddings.detach().numpy())

In [ ]:
cls=cls_KM.predict(gcn_embeddings.detach().numpy())
extracted_text_cls=extracted_text_KM.predict(gcn_embeddings.detach().numpy())

In [ ]:
cls_index=cls_KM.predict(gcn_embeddings[gcn_texts_k1_match_indices].reshape(gcn_embeddings.shape).detach().numpy() )
et_index=extracted_text_KM.predict(gcn_embeddings[gcn_extracte_k1_match_indices].reshape(gcn_embeddings.shape).detach().numpy() )

In [ ]:
x_cls=cls_index==cls
et=et_index==extracted_text_cls
np.sum(x_cls),np.sum(et)

### GIN

In [ ]:
cls_KM=KM(gin_embeddings.detach().numpy())

extracted_text_KM=KM(gin_embeddings.detach().numpy())

In [ ]:
cls=cls_KM.predict(gin_embeddings.detach().numpy())
extracted_text_cls=extracted_text_KM.predict(gin_embeddings.detach().numpy())

In [ ]:
cls_index=cls_KM.predict(gin_embeddings[gin_texts_k1_match_indices].reshape(gin_embeddings.shape).detach().numpy())
et_index=extracted_text_KM.predict(gin_embeddings[gin_extracted_k1_match_indices].reshape(gin_embeddings.shape).detach().numpy() )

In [ ]:
x_cls=cls_index==cls
et=et_index==extracted_text_cls
np.sum(x_cls),np.sum(et)

In [ ]:
del embeddings
del extracted_embeddings
import gc
gc.collect()
torch.cuda.empty_cache()

## Pure BERT

In [ ]:
import torch
from transformers import BertTokenizer, BertModel
def pureBert(texts=texts):
    # 检查是否有可用的 GPU
    device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
    # device = torch.device('cuda:1')
    # 加载预训练的BERT模型和tokenizer
    model_name = 'bert-base-uncased'  # 你可以使用其他的BERT模型，例如 'bert-large-uncased'
    tokenizer = BertTokenizer.from_pretrained(model_name)
    model = BertModel.from_pretrained(model_name)

    # 将模型转移到 GPU
    model = model.to(device)

    # 将句子 token 化，并将输入转移到 GPU
    inputs = tokenizer(texts, return_tensors='pt', truncation=True, padding=True)
    inputs = {key: value.to(device) for key, value in inputs.items()}  # 转移到 GPU

    # 获取BERT的输出
    with torch.no_grad():  # 禁用梯度计算，节省内存
        outputs = model(**inputs)

    # BERT的最后隐藏层的输出 [batch_size, seq_len, hidden_size]
    last_hidden_states = outputs.last_hidden_state

    # 取 [CLS] token 的 embedding（句子嵌入）
    cls_embedding = last_hidden_states[:, 0, :]

    # 也可以取所有 token 的平均值作为句子嵌入
    mean_embedding = torch.mean(last_hidden_states, dim=1)

    print(f"CLS token embedding size: {cls_embedding.shape}")
    print(f"Mean token embedding size: {mean_embedding.shape}")

    return cls_embedding,mean_embedding


In [ ]:
# cls_embedding,mean_embedding=pureBert(texts)
cls_embedding1,mean_embedding1=pureBert(texts[:1000])
cls_embedding2,mean_embedding2=pureBert(texts[1000:2000])
cls_embedding3,mean_embedding3=pureBert(texts[2000:3000])
cls_embedding4,mean_embedding4=pureBert(texts[3000:])

cls_embedding=torch.cat((cls_embedding1,cls_embedding2,cls_embedding3,cls_embedding4))
mean_embedding=torch.cat((mean_embedding1,mean_embedding2,mean_embedding3,mean_embedding4))

In [ ]:
# extracted_text_cls_embedding,extracted_text_mean_embedding=pureBert(extracted_text)
extracted_text_cls_embedding1,extracted_text_mean_embedding1=pureBert(extracted_text[:1000])
extracted_text_cls_embedding2,extracted_text_mean_embedding2=pureBert(extracted_text[1000:2000])
extracted_text_cls_embedding3,extracted_text_mean_embedding3=pureBert(extracted_text[2000:3000])
extracted_text_cls_embedding4,extracted_text_mean_embedding4=pureBert(extracted_text[3000:])

extracted_text_cls_embedding=torch.cat((extracted_text_cls_embedding1,extracted_text_cls_embedding2,extracted_text_cls_embedding3,extracted_text_cls_embedding4))
extracted_text_mean_embedding=torch.cat((extracted_text_mean_embedding1,extracted_text_mean_embedding2,extracted_text_mean_embedding3,extracted_text_mean_embedding4))

In [ ]:
de_biased_cka(cls_embedding[compute_list], gcn_embeddings),de_biased_cka(cls_embedding[compute_list], gcn_out_embeddings),de_biased_cka(cls_embedding[compute_list],gin_embeddings),de_biased_cka(cls_embedding[compute_list],gin_out_embeddings)

In [ ]:
de_biased_cka(mean_embedding[compute_list],gcn_embeddings),de_biased_cka(mean_embedding[compute_list],gcn_out_embeddings),de_biased_cka(mean_embedding[compute_list],gin_embeddings),de_biased_cka(mean_embedding[compute_list],gin_out_embeddings)

In [ ]:
de_biased_cka(extracted_text_cls_embedding[compute_list], gcn_embeddings),de_biased_cka(extracted_text_cls_embedding[compute_list],gcn_out_embeddings),de_biased_cka(extracted_text_cls_embedding[compute_list],gin_embeddings),de_biased_cka(extracted_text_cls_embedding[compute_list],gin_out_embeddings)

In [ ]:
de_biased_cka(extracted_text_mean_embedding[compute_list], gcn_embeddings),de_biased_cka(extracted_text_mean_embedding[compute_list],gcn_out_embeddings),de_biased_cka(extracted_text_mean_embedding[compute_list],gin_embeddings),de_biased_cka(extracted_text_mean_embedding[compute_list],gin_out_embeddings)

In [ ]:
print(CKA(cls_embedding[compute_list], gcn_embeddings.to(device)),CKA(cls_embedding[compute_list], gcn_out_embeddings.to(device)),CKA(cls_embedding[compute_list],gin_embeddings.to(device)),CKA(cls_embedding[compute_list],gin_out_embeddings.to(device)))
print(CKA(mean_embedding[compute_list],gcn_embeddings.to(device)),CKA(mean_embedding[compute_list],gcn_out_embeddings.to(device)),CKA(mean_embedding[compute_list],gin_embeddings.to(device)),CKA(mean_embedding[compute_list],gin_out_embeddings.to(device)))
print(CKA(extracted_text_cls_embedding[compute_list], gcn_embeddings.to(device)),CKA(extracted_text_cls_embedding[compute_list],gcn_out_embeddings.to(device)),CKA(extracted_text_cls_embedding[compute_list],gin_embeddings.to(device)),CKA(extracted_text_cls_embedding[compute_list],gin_out_embeddings.to(device)))
print(CKA(extracted_text_mean_embedding[compute_list], gcn_embeddings.to(device)),CKA(extracted_text_mean_embedding[compute_list],gcn_out_embeddings.to(device)),CKA(extracted_text_mean_embedding[compute_list],gin_embeddings.to(device)),CKA(extracted_text_mean_embedding[compute_list],gin_out_embeddings.to(device)))

In [ ]:
len(text_list),len(et_list)

In [ ]:
del cls_embedding1,mean_embedding1,
del cls_embedding2,mean_embedding2,
del cls_embedding3,mean_embedding3,
del cls_embedding4,mean_embedding4,
del cls_embedding,
del mean_embedding,
del extracted_text_cls_embedding1,extracted_text_mean_embedding1,
del extracted_text_cls_embedding2,extracted_text_mean_embedding2,
del extracted_text_cls_embedding3,extracted_text_mean_embedding3,
del extracted_text_cls_embedding4,extracted_text_mean_embedding4,
del extracted_text_cls_embedding,
del extracted_text_mean_embedding,

import gc
gc.collect()
torch.cuda.empty_cache()

## GPT

In [ ]:
from transformers import AutoTokenizer, OpenAIGPTModel
import torch
def get_embedding(texts=texts):
    # Load tokenizer and model
    tokenizer = AutoTokenizer.from_pretrained("openai-community/openai-gpt")
    model = OpenAIGPTModel.from_pretrained("openai-community/openai-gpt")

    # Set the padding token
    if tokenizer.pad_token is None:
        tokenizer.add_special_tokens({'pad_token': '[PAD]'})
        model.resize_token_embeddings(len(tokenizer))

    # Move model to GPU if available
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    # Tokenize input texts
    inputs = tokenizer(texts, return_tensors="pt", truncation=True, padding=True)

    # Move inputs to the same device as the model
    inputs = {key: value.to(device) for key, value in inputs.items()}

    # Run model to get hidden states
    with torch.no_grad():  # Disable gradient calculation for inference
        outputs = model(**inputs)

    # Access last hidden states
    last_hidden_states = outputs.last_hidden_state
    print(last_hidden_states.shape)

    # 获取最后的隐藏状态 [batch_size, seq_len, hidden_size]
    last_hidden_states = outputs.last_hidden_state

    # 获取第一个 token (<s> token) 的 embedding（相当于 CLS token 的句子嵌入）
    cls_embedding = last_hidden_states[:, 0, :]

    # 也可以取所有 token 的平均值作为句子嵌入
    mean_embedding = torch.mean(last_hidden_states, dim=1)

    print(f"CLS token embedding size: {cls_embedding.shape}")
    print(f"Mean token embedding size: {mean_embedding.shape}")
    return cls_embedding,mean_embedding


In [ ]:
# cls_embedding,mean_embedding=get_embedding(texts)
cls_embedding1,mean_embedding1=get_embedding(texts[:1000])
cls_embedding2,mean_embedding2=get_embedding(texts[1000:2000])
cls_embedding3,mean_embedding3=get_embedding(texts[2000:3000])
cls_embedding4,mean_embedding4=get_embedding(texts[3000:])

cls_embedding=torch.cat((cls_embedding1,cls_embedding2,cls_embedding3,cls_embedding4))
mean_embedding=torch.cat((mean_embedding1,mean_embedding2,mean_embedding3,mean_embedding4))

In [ ]:
cls_embedding.shape,mean_embedding.shape

In [ ]:
# extracted_text_cls_embedding,extracted_text_mean_embedding=get_embedding(extracted_text)
extracted_text_cls_embedding1,extracted_text_mean_embedding1=get_embedding(extracted_text[:1000])
extracted_text_cls_embedding2,extracted_text_mean_embedding2=get_embedding(extracted_text[1000:2000])
extracted_text_cls_embedding3,extracted_text_mean_embedding3=get_embedding(extracted_text[2000:3000])
extracted_text_cls_embedding4,extracted_text_mean_embedding4=get_embedding(extracted_text[3000:])

extracted_text_cls_embedding=torch.cat((extracted_text_cls_embedding1,extracted_text_cls_embedding2,extracted_text_cls_embedding3,extracted_text_cls_embedding4))
extracted_text_mean_embedding=torch.cat((extracted_text_mean_embedding1,extracted_text_mean_embedding2,extracted_text_mean_embedding3,extracted_text_mean_embedding4))
extracted_text_cls_embedding.shape,extracted_text_mean_embedding.shape

### CKA

In [ ]:
de_biased_cka(cls_embedding[compute_list], gcn_embeddings),de_biased_cka(cls_embedding[compute_list], gcn_out_embeddings),de_biased_cka(cls_embedding[compute_list],gin_embeddings),de_biased_cka(cls_embedding[compute_list],gin_out_embeddings)

In [ ]:
de_biased_cka(mean_embedding[compute_list],gcn_embeddings),de_biased_cka(mean_embedding[compute_list],gcn_out_embeddings),de_biased_cka(mean_embedding[compute_list],gin_embeddings),de_biased_cka(mean_embedding[compute_list],gin_out_embeddings)

In [ ]:
de_biased_cka(extracted_text_cls_embedding[compute_list], gcn_embeddings),de_biased_cka(extracted_text_cls_embedding[compute_list],gcn_out_embeddings),de_biased_cka(extracted_text_cls_embedding[compute_list],gin_embeddings),de_biased_cka(extracted_text_cls_embedding[compute_list],gin_out_embeddings)

In [ ]:
de_biased_cka(extracted_text_mean_embedding[compute_list], gcn_embeddings),de_biased_cka(extracted_text_mean_embedding[compute_list],gcn_out_embeddings),de_biased_cka(extracted_text_mean_embedding[compute_list],gin_embeddings),de_biased_cka(extracted_text_mean_embedding[compute_list],gin_out_embeddings)

In [ ]:
print(CKA(cls_embedding[compute_list], gcn_embeddings.to(device)),CKA(cls_embedding[compute_list], gcn_out_embeddings.to(device)),CKA(cls_embedding[compute_list],gin_embeddings.to(device)),CKA(cls_embedding[compute_list],gin_out_embeddings.to(device)))
print(CKA(mean_embedding[compute_list],gcn_embeddings.to(device)),CKA(mean_embedding[compute_list],gcn_out_embeddings.to(device)),CKA(mean_embedding[compute_list],gin_embeddings.to(device)),CKA(mean_embedding[compute_list],gin_out_embeddings.to(device)))
print(CKA(extracted_text_cls_embedding[compute_list], gcn_embeddings.to(device)),CKA(extracted_text_cls_embedding[compute_list],gcn_out_embeddings.to(device)),CKA(extracted_text_cls_embedding[compute_list],gin_embeddings.to(device)),CKA(extracted_text_cls_embedding[compute_list],gin_out_embeddings.to(device)))
print(CKA(extracted_text_mean_embedding[compute_list], gcn_embeddings.to(device)),CKA(extracted_text_mean_embedding[compute_list],gcn_out_embeddings.to(device)),CKA(extracted_text_mean_embedding[compute_list],gin_embeddings.to(device)),CKA(extracted_text_mean_embedding[compute_list],gin_out_embeddings.to(device)))

In [ ]:
del cls_embedding1,
del mean_embedding1,
del cls_embedding2,
del mean_embedding2,
del cls_embedding3,
del mean_embedding3,
del cls_embedding4,
del mean_embedding4,
del cls_embedding,
del mean_embedding,
del extracted_text_cls_embedding1,
del extracted_text_mean_embedding1,
del extracted_text_cls_embedding2,
del extracted_text_mean_embedding2,
del extracted_text_cls_embedding3,
del extracted_text_mean_embedding3,
del extracted_text_cls_embedding4,
del extracted_text_mean_embedding4,
del extracted_text_cls_embedding,
del extracted_text_mean_embedding,


import gc
gc.collect()
torch.cuda.empty_cache()

## last word GPT

In [ ]:
import torch
from transformers import AutoTokenizer, OpenAIGPTModel

def GPTembedding(texts):

    # Check if GPU is available and set the device
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

    # Load pre-trained model and tokenizer and move the model to the GPU
    tokenizer = AutoTokenizer.from_pretrained("openai-community/openai-gpt")
    model = OpenAIGPTModel.from_pretrained("openai-community/openai-gpt").to(device)
    
    if tokenizer.pad_token is None:
        tokenizer.add_special_tokens({'pad_token': '[PAD]'})
        model.resize_token_embeddings(len(tokenizer))
    # Input sentences
    # texts = ['Your sentence here1', 'Your sentence here2']

    # Tokenize input sentence and move input tensors to the GPU

    template = 'This_sentence_:_"The_molecule_is_a_nitrile_that_is_acetonitrile_where_one_of_the_methyl_hydrogens_is_substituted_by_a_2-methylphenyl_group."_means_in_one_word:"Acetonitrile".This_sentence_:_"*sent_0*"_means_in_one_word:"'
    inputs = tokenizer([template.replace('*sent_0*', i).replace('_', ' ') for i in texts], padding=True, truncation=True,  max_length=512,return_tensors="pt")['input_ids'].to(device)
    with torch.no_grad():
        embeddings = model(inputs, output_hidden_states=True, return_dict=True).hidden_states[-1][:, -1, :]
    print(embeddings.shape)
    return embeddings



In [ ]:
# cls_embedding=GPTembedding(texts)
cls_embedding1=GPTembedding(texts[:1000])
cls_embedding2=GPTembedding(texts[1000:2000])
cls_embedding3=GPTembedding(texts[2000:3000])
cls_embedding4=GPTembedding(texts[3000:])

cls_embedding=torch.cat((cls_embedding1,cls_embedding2,cls_embedding3,cls_embedding4))

In [ ]:
# extracted_text_cls_embedding=GPTembedding(extracted_text)
extracted_text_cls_embedding1=GPTembedding(extracted_text[:1000])
extracted_text_cls_embedding2=GPTembedding(extracted_text[1000:2000])
extracted_text_cls_embedding3=GPTembedding(extracted_text[2000:3000])
extracted_text_cls_embedding4=GPTembedding(extracted_text[3000:])

extracted_text_cls_embedding=torch.cat((extracted_text_cls_embedding1,extracted_text_cls_embedding2,extracted_text_cls_embedding3,extracted_text_cls_embedding4))

### CKA

In [ ]:
de_biased_cka(cls_embedding[compute_list], gcn_embeddings),de_biased_cka(cls_embedding[compute_list], gcn_out_embeddings),de_biased_cka(cls_embedding[compute_list],gin_embeddings),de_biased_cka(cls_embedding[compute_list],gin_out_embeddings)

In [ ]:
de_biased_cka(extracted_text_cls_embedding[compute_list], gcn_embeddings),de_biased_cka(extracted_text_cls_embedding[compute_list],gcn_out_embeddings),de_biased_cka(extracted_text_cls_embedding[compute_list],gin_embeddings),de_biased_cka(extracted_text_cls_embedding[compute_list],gin_out_embeddings)

In [ ]:
print(CKA(cls_embedding[compute_list], gcn_embeddings.to(device)),CKA(cls_embedding[compute_list], gcn_out_embeddings.to(device)),CKA(cls_embedding[compute_list],gin_embeddings.to(device)),CKA(cls_embedding[compute_list],gin_out_embeddings.to(device)))
print(CKA(extracted_text_cls_embedding[compute_list], gcn_embeddings.to(device)),CKA(extracted_text_cls_embedding[compute_list],gcn_out_embeddings.to(device)),CKA(extracted_text_cls_embedding[compute_list],gin_embeddings.to(device)),CKA(extracted_text_cls_embedding[compute_list],gin_out_embeddings.to(device)))


In [ ]:
del cls_embedding1,
del cls_embedding2,
del cls_embedding3,
del cls_embedding4,
del cls_embedding,
del extracted_text_cls_embedding1,
del extracted_text_cls_embedding2,
del extracted_text_cls_embedding3,
del extracted_text_cls_embedding4,
del extracted_text_cls_embedding,

import gc
gc.collect()
torch.cuda.empty_cache()

## Summarise last word GPT

In [ ]:
import torch
from transformers import AutoTokenizer, OpenAIGPTModel

def GPTembedding(texts):

    # Check if GPU is available and set the device
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

    # Load pre-trained model and tokenizer and move the model to the GPU
    tokenizer = AutoTokenizer.from_pretrained("openai-community/openai-gpt")
    model = OpenAIGPTModel.from_pretrained("openai-community/openai-gpt").to(device)
    
    if tokenizer.pad_token is None:
        tokenizer.add_special_tokens({'pad_token': '[PAD]'})
        model.resize_token_embeddings(len(tokenizer))
    # Input sentences
    # texts = ['Your sentence here1', 'Your sentence here2']

    # Tokenize input sentence and move input tensors to the GPU

    template = 'This_sentence_:_"The_molecule_is_a_nitrile_that_is_acetonitrile_where_one_of_the_methyl_hydrogens_is_substituted_by_a_2-methylphenyl_group."_summarise_in_one_word:"Acetonitrile".This_sentence_:_"*sent_0*"_summarise_in_one_word:"'
    inputs = tokenizer([template.replace('*sent_0*', i).replace('_', ' ') for i in texts], padding=True, truncation=True,  max_length=512,return_tensors="pt")['input_ids'].to(device)
    with torch.no_grad():
        embeddings = model(inputs, output_hidden_states=True, return_dict=True).hidden_states[-1][:, -1, :]
    print(embeddings.shape)
    return embeddings



In [ ]:
# cls_embedding=GPTembedding(texts)
cls_embedding1=GPTembedding(texts[:1000])
cls_embedding2=GPTembedding(texts[1000:2000])
cls_embedding3=GPTembedding(texts[2000:3000])
cls_embedding4=GPTembedding(texts[3000:])

cls_embedding=torch.cat((cls_embedding1,cls_embedding2,cls_embedding3,cls_embedding4))

In [ ]:
# extracted_text_cls_embedding=GPTembedding(extracted_text)
# extracted_text_cls_embedding=GPTembedding(extracted_text)
extracted_text_cls_embedding1=GPTembedding(extracted_text[:1000])
extracted_text_cls_embedding2=GPTembedding(extracted_text[1000:2000])
extracted_text_cls_embedding3=GPTembedding(extracted_text[2000:3000])
extracted_text_cls_embedding4=GPTembedding(extracted_text[3000:])

extracted_text_cls_embedding=torch.cat((extracted_text_cls_embedding1,extracted_text_cls_embedding2,extracted_text_cls_embedding3,extracted_text_cls_embedding4))

### CKA

In [ ]:
de_biased_cka(cls_embedding[compute_list], gcn_embeddings),de_biased_cka(cls_embedding[compute_list], gcn_out_embeddings),de_biased_cka(cls_embedding[compute_list],gin_embeddings),de_biased_cka(cls_embedding[compute_list],gin_out_embeddings)

In [ ]:
de_biased_cka(extracted_text_cls_embedding[compute_list], gcn_embeddings),de_biased_cka(extracted_text_cls_embedding[compute_list],gcn_out_embeddings),de_biased_cka(extracted_text_cls_embedding[compute_list],gin_embeddings),de_biased_cka(extracted_text_cls_embedding[compute_list],gin_out_embeddings)

In [ ]:
print(CKA(cls_embedding[compute_list], gcn_embeddings.to(device)),CKA(cls_embedding[compute_list], gcn_out_embeddings.to(device)),CKA(cls_embedding[compute_list],gin_embeddings.to(device)),CKA(cls_embedding[compute_list],gin_out_embeddings.to(device)))
print(CKA(extracted_text_cls_embedding[compute_list], gcn_embeddings.to(device)),CKA(extracted_text_cls_embedding[compute_list],gcn_out_embeddings.to(device)),CKA(extracted_text_cls_embedding[compute_list],gin_embeddings.to(device)),CKA(extracted_text_cls_embedding[compute_list],gin_out_embeddings.to(device)))


In [ ]:
del cls_embedding1,
del cls_embedding2,
del cls_embedding3,
del cls_embedding4,
del cls_embedding,
del extracted_text_cls_embedding1,
del extracted_text_cls_embedding2,
del extracted_text_cls_embedding3,
del extracted_text_cls_embedding4,
del extracted_text_cls_embedding,

import gc
gc.collect()
torch.cuda.empty_cache()

## GPT 2

In [ ]:
import torch
from transformers import GPT2Model, GPT2Tokenizer

def GPT2embedding(texts):

    # Check if GPU is available and set the device
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    # Load pre-trained model and tokenizer and move the model to the GPU
    model = GPT2Model.from_pretrained('gpt2').to(device)
    tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
    
    if tokenizer.pad_token is None:
        tokenizer.add_special_tokens({'pad_token': '[PAD]'})
        model.resize_token_embeddings(len(tokenizer))
    # Input sentences
    # texts = ['Your sentence here1', 'Your sentence here2']

    # Tokenize input sentence and move input tensors to the GPU
    input_ids = tokenizer(texts, return_tensors='pt', padding=True, truncation=True)['input_ids'].to(device)

    # Get the hidden states (outputs from each layer)
    with torch.no_grad():
        outputs = model(input_ids)

    # # Optionally, move the result back to the CPU if needed
    # sentence_embedding = sentence_embedding.cpu().detach()

    # print(sentence_embedding.shape)
    # return sentence_embedding

    # Access last hidden states
    last_hidden_states = outputs.last_hidden_state
    print(last_hidden_states.shape)

    # 获取最后的隐藏状态 [batch_size, seq_len, hidden_size]
    last_hidden_states = outputs.last_hidden_state

    # 获取第一个 token (<s> token) 的 embedding（相当于 CLS token 的句子嵌入）
    cls_embedding = last_hidden_states[:, 0, :]

    # 也可以取所有 token 的平均值作为句子嵌入
    mean_embedding = torch.mean(last_hidden_states, dim=1)

    print(f"CLS token embedding size: {cls_embedding.shape}")
    print(f"Mean token embedding size: {mean_embedding.shape}")
    return cls_embedding,mean_embedding

In [ ]:
# cls_embedding,mean_embedding=GPT2embedding(texts)
cls_embedding1,mean_embedding1=GPT2embedding(texts[:1000])
cls_embedding2,mean_embedding2=GPT2embedding(texts[1000:2000])
cls_embedding3,mean_embedding3=GPT2embedding(texts[2000:3000])
cls_embedding4,mean_embedding4=GPT2embedding(texts[3000:])

cls_embedding=torch.cat((cls_embedding1,cls_embedding2,cls_embedding3,cls_embedding4))
mean_embedding=torch.cat((mean_embedding1,mean_embedding2,mean_embedding3,mean_embedding4))

In [ ]:
# extracted_text_cls_embedding,extracted_text_mean_embedding=GPT2embedding(extracted_text)
extracted_text_cls_embedding1,extracted_text_mean_embedding1=GPT2embedding(extracted_text[:1000])
extracted_text_cls_embedding2,extracted_text_mean_embedding2=GPT2embedding(extracted_text[1000:2000])
extracted_text_cls_embedding3,extracted_text_mean_embedding3=GPT2embedding(extracted_text[2000:3000])
extracted_text_cls_embedding4,extracted_text_mean_embedding4=GPT2embedding(extracted_text[3000:])

extracted_text_cls_embedding=torch.cat((extracted_text_cls_embedding1,extracted_text_cls_embedding2,extracted_text_cls_embedding3,extracted_text_cls_embedding4))
extracted_text_mean_embedding=torch.cat((extracted_text_mean_embedding1,extracted_text_mean_embedding2,extracted_text_mean_embedding3,extracted_text_mean_embedding4))

### CKA

In [ ]:
de_biased_cka(cls_embedding[compute_list], gcn_embeddings),de_biased_cka(cls_embedding[compute_list], gcn_out_embeddings),de_biased_cka(cls_embedding[compute_list],gin_embeddings),de_biased_cka(cls_embedding[compute_list],gin_out_embeddings)

In [ ]:
de_biased_cka(mean_embedding[compute_list],gcn_embeddings),de_biased_cka(mean_embedding[compute_list],gcn_out_embeddings),de_biased_cka(mean_embedding[compute_list],gin_embeddings),de_biased_cka(mean_embedding[compute_list],gin_out_embeddings)

In [ ]:
de_biased_cka(extracted_text_cls_embedding[compute_list], gcn_embeddings),de_biased_cka(extracted_text_cls_embedding[compute_list],gcn_out_embeddings),de_biased_cka(extracted_text_cls_embedding[compute_list],gin_embeddings),de_biased_cka(extracted_text_cls_embedding[compute_list],gin_out_embeddings)

In [ ]:
de_biased_cka(extracted_text_mean_embedding[compute_list], gcn_embeddings),de_biased_cka(extracted_text_mean_embedding[compute_list],gcn_out_embeddings),de_biased_cka(extracted_text_mean_embedding[compute_list],gin_embeddings),de_biased_cka(extracted_text_mean_embedding[compute_list],gin_out_embeddings)

In [ ]:
print(CKA(cls_embedding[compute_list], gcn_embeddings.to(device)),CKA(cls_embedding[compute_list], gcn_out_embeddings.to(device)),CKA(cls_embedding[compute_list],gin_embeddings.to(device)),CKA(cls_embedding[compute_list],gin_out_embeddings.to(device)))
print(CKA(mean_embedding[compute_list],gcn_embeddings.to(device)),CKA(mean_embedding[compute_list],gcn_out_embeddings.to(device)),CKA(mean_embedding[compute_list],gin_embeddings.to(device)),CKA(mean_embedding[compute_list],gin_out_embeddings.to(device)))
print(CKA(extracted_text_cls_embedding[compute_list], gcn_embeddings.to(device)),CKA(extracted_text_cls_embedding[compute_list],gcn_out_embeddings.to(device)),CKA(extracted_text_cls_embedding[compute_list],gin_embeddings.to(device)),CKA(extracted_text_cls_embedding[compute_list],gin_out_embeddings.to(device)))
print(CKA(extracted_text_mean_embedding[compute_list], gcn_embeddings.to(device)),CKA(extracted_text_mean_embedding[compute_list],gcn_out_embeddings.to(device)),CKA(extracted_text_mean_embedding[compute_list],gin_embeddings.to(device)),CKA(extracted_text_mean_embedding[compute_list],gin_out_embeddings.to(device)))

In [ ]:
del cls_embedding1,
del mean_embedding1,
del cls_embedding2,
del mean_embedding2,
del cls_embedding3,
del mean_embedding3,
del cls_embedding4,
del mean_embedding4,
del cls_embedding,
del mean_embedding,
del extracted_text_cls_embedding1,
del extracted_text_mean_embedding1,
del extracted_text_cls_embedding2,
del extracted_text_mean_embedding2,
del extracted_text_cls_embedding3,
del extracted_text_mean_embedding3,
del extracted_text_cls_embedding4,
del extracted_text_mean_embedding4,
del extracted_text_cls_embedding,
del extracted_text_mean_embedding,

import gc
gc.collect()
torch.cuda.empty_cache()

## last word GPT 2

In [ ]:
import torch
from transformers import GPT2Model, GPT2Tokenizer

def GPT2embedding(texts):

    # Check if GPU is available and set the device
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

    # Load pre-trained model and tokenizer and move the model to the GPU
    model = GPT2Model.from_pretrained('gpt2').to(device)
    tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
    
    if tokenizer.pad_token is None:
        tokenizer.add_special_tokens({'pad_token': '[PAD]'})
        model.resize_token_embeddings(len(tokenizer))
    # Input sentences
    # texts = ['Your sentence here1', 'Your sentence here2']

    # Tokenize input sentence and move input tensors to the GPU

    template = 'This_sentence_:_"The_molecule_is_a_nitrile_that_is_acetonitrile_where_one_of_the_methyl_hydrogens_is_substituted_by_a_2-methylphenyl_group."_means_in_one_word:"Acetonitrile".This_sentence_:_"*sent_0*"_means_in_one_word:"'
    inputs = tokenizer([template.replace('*sent_0*', i).replace('_', ' ') for i in texts], padding=True,  return_tensors="pt")['input_ids'].to(device)
    with torch.no_grad():
        embeddings = model(inputs, output_hidden_states=True, return_dict=True).hidden_states[-1][:, -1, :]
    print(embeddings.shape)
    return embeddings



In [ ]:
# cls_embedding=GPT2embedding(texts)
cls_embedding1=GPT2embedding(texts[:1000])
cls_embedding2=GPT2embedding(texts[1000:2000])
cls_embedding3=GPT2embedding(texts[2000:3000])
cls_embedding4=GPT2embedding(texts[3000:])

cls_embedding=torch.cat((cls_embedding1,cls_embedding2,cls_embedding3,cls_embedding4))

In [ ]:
# extracted_text_cls_embedding=GPT2embedding(extracted_text)
extracted_text_cls_embedding1=GPT2embedding(extracted_text[:1000])
extracted_text_cls_embedding2=GPT2embedding(extracted_text[1000:2000])
extracted_text_cls_embedding3=GPT2embedding(extracted_text[2000:3000])
extracted_text_cls_embedding4=GPT2embedding(extracted_text[3000:])

extracted_text_cls_embedding=torch.cat((extracted_text_cls_embedding1,extracted_text_cls_embedding2,extracted_text_cls_embedding3,extracted_text_cls_embedding4))

### CKA

In [ ]:
de_biased_cka(cls_embedding[compute_list], gcn_embeddings),de_biased_cka(cls_embedding[compute_list], gcn_out_embeddings),de_biased_cka(cls_embedding[compute_list],gin_embeddings),de_biased_cka(cls_embedding[compute_list],gin_out_embeddings)

In [ ]:
de_biased_cka(extracted_text_cls_embedding[compute_list], gcn_embeddings),de_biased_cka(extracted_text_cls_embedding[compute_list],gcn_out_embeddings),de_biased_cka(extracted_text_cls_embedding[compute_list],gin_embeddings),de_biased_cka(extracted_text_cls_embedding[compute_list],gin_out_embeddings)

In [ ]:
print(CKA(cls_embedding[compute_list], gcn_embeddings.to(device)),CKA(cls_embedding[compute_list], gcn_out_embeddings.to(device)),CKA(cls_embedding[compute_list],gin_embeddings.to(device)),CKA(cls_embedding[compute_list],gin_out_embeddings.to(device)))
print(CKA(extracted_text_cls_embedding[compute_list], gcn_embeddings.to(device)),CKA(extracted_text_cls_embedding[compute_list],gcn_out_embeddings.to(device)),CKA(extracted_text_cls_embedding[compute_list],gin_embeddings.to(device)),CKA(extracted_text_cls_embedding[compute_list],gin_out_embeddings.to(device)))


In [ ]:
del cls_embedding1,
del cls_embedding2,
del cls_embedding3,
del cls_embedding4,
del cls_embedding,
del extracted_text_cls_embedding1,
del extracted_text_cls_embedding2,
del extracted_text_cls_embedding3,
del extracted_text_cls_embedding4,
del extracted_text_cls_embedding,

import gc
gc.collect()
torch.cuda.empty_cache()

## Summarise last word GPT 2

In [ ]:
import torch
from transformers import GPT2Model, GPT2Tokenizer

def GPT2embedding(texts):

    # Check if GPU is available and set the device
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

    # Load pre-trained model and tokenizer and move the model to the GPU
    model = GPT2Model.from_pretrained('gpt2').to(device)
    tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
    
    if tokenizer.pad_token is None:
        tokenizer.add_special_tokens({'pad_token': '[PAD]'})
        model.resize_token_embeddings(len(tokenizer))
    # Input sentences
    # texts = ['Your sentence here1', 'Your sentence here2']

    # Tokenize input sentence and move input tensors to the GPU

    template = 'This_sentence_:_"The_molecule_is_a_nitrile_that_is_acetonitrile_where_one_of_the_methyl_hydrogens_is_substituted_by_a_2-methylphenyl_group."_summarise_in_one_word:"Acetonitrile".This_sentence_:_"*sent_0*"_summarise_in_one_word:"'
    inputs = tokenizer([template.replace('*sent_0*', i).replace('_', ' ') for i in texts], padding=True,  return_tensors="pt")['input_ids'].to(device)
    with torch.no_grad():
        embeddings = model(inputs, output_hidden_states=True, return_dict=True).hidden_states[-1][:, -1, :]
    print(embeddings.shape)
    return embeddings



In [ ]:
# cls_embedding=GPT2embedding(texts)
cls_embedding1=GPT2embedding(texts[:1000])
cls_embedding2=GPT2embedding(texts[1000:2000])
cls_embedding3=GPT2embedding(texts[2000:3000])
cls_embedding4=GPT2embedding(texts[3000:])

cls_embedding=torch.cat((cls_embedding1,cls_embedding2,cls_embedding3,cls_embedding4))

In [ ]:
# extracted_text_cls_embedding=GPT2embedding(extracted_text)
extracted_text_cls_embedding1=GPT2embedding(extracted_text[:1000])
extracted_text_cls_embedding2=GPT2embedding(extracted_text[1000:2000])
extracted_text_cls_embedding3=GPT2embedding(extracted_text[2000:3000])
extracted_text_cls_embedding4=GPT2embedding(extracted_text[3000:])

extracted_text_cls_embedding=torch.cat((extracted_text_cls_embedding1,extracted_text_cls_embedding2,extracted_text_cls_embedding3,extracted_text_cls_embedding4))


### CKA

In [ ]:
de_biased_cka(cls_embedding[compute_list], gcn_embeddings),de_biased_cka(cls_embedding[compute_list], gcn_out_embeddings),de_biased_cka(cls_embedding[compute_list],gin_embeddings),de_biased_cka(cls_embedding[compute_list],gin_out_embeddings)

In [ ]:
de_biased_cka(extracted_text_cls_embedding[compute_list], gcn_embeddings),de_biased_cka(extracted_text_cls_embedding[compute_list],gcn_out_embeddings),de_biased_cka(extracted_text_cls_embedding[compute_list],gin_embeddings),de_biased_cka(extracted_text_cls_embedding[compute_list],gin_out_embeddings)

In [ ]:
print(CKA(cls_embedding[compute_list], gcn_embeddings.to(device)),CKA(cls_embedding[compute_list], gcn_out_embeddings.to(device)),CKA(cls_embedding[compute_list],gin_embeddings.to(device)),CKA(cls_embedding[compute_list],gin_out_embeddings.to(device)))
print(CKA(extracted_text_cls_embedding[compute_list], gcn_embeddings.to(device)),CKA(extracted_text_cls_embedding[compute_list],gcn_out_embeddings.to(device)),CKA(extracted_text_cls_embedding[compute_list],gin_embeddings.to(device)),CKA(extracted_text_cls_embedding[compute_list],gin_out_embeddings.to(device)))


In [ ]:
del cls_embedding1,
del cls_embedding2,
del cls_embedding3,
del cls_embedding4,
del cls_embedding,
del extracted_text_cls_embedding1,
del extracted_text_cls_embedding2,
del extracted_text_cls_embedding3,
del extracted_text_cls_embedding4,
del extracted_text_cls_embedding,

import gc
gc.collect()
torch.cuda.empty_cache()